In [27]:
# ═══════════════════════════════════════════════════════════════════
# BLOCO 1: SELETOR DE PASTA COM TIMER + MIGRAÇÃO
# Versão: 4.4 - COM MELHORIAS DE OBSERVABILIDADE E CONSISTÊNCIA
# Data: 2025-10-17
# ═══════════════════════════════════════════════════════════════════
# MELHORIAS v4.4:
# ✅ MELHORIA 1: Salvar estado local .bloco_1_state.json
# ✅ MELHORIA 2: Registro de versão do código
# ✅ MELHORIA 3: Validação de dependências
# ═══════════════════════════════════════════════════════════════════

# ═══════════════════════════════════════════════════════════════════
# METADADOS DA VERSÃO
# ═══════════════════════════════════════════════════════════════════

VERSAO_BLOCO1 = '4.4'
DATA_VERSAO = '2025-10-17'
CHANGELOG_V44 = {
    'v4.4': {
        'data': '2025-10-17',
        'melhorias': [
            'Salvar estado local em .bloco_1_state.json',
            'Registro de versão do código',
            'Validação de dependências no início'
        ],
        'compatibilidade': 'v4.3'
    }
}

print("="*70)
print(f" 🔍 PROCESSADOR DE ARQUIVOS DESCONHECIDOS v{VERSAO_BLOCO1}")
print("="*70)
print(f" Versão: {VERSAO_BLOCO1} | Data: {DATA_VERSAO}")
print(" Timer | Migração | Dicionários | Validações | Logs | Estado")
print("="*70)

# ═══════════════════════════════════════════════════════════════════
# MELHORIA 3: VALIDAÇÃO DE DEPENDÊNCIAS
# ═══════════════════════════════════════════════════════════════════

def validar_dependencias():
    """
    Valida que todas as bibliotecas necessárias estão instaladas.

    Evita erros confusos mais tarde na execução.
    """
    print("\n🔍 Validando dependências...")

    dependencias = {
        'pandas': 'pandas',
        'numpy': 'numpy',
        'openpyxl': 'openpyxl',
        'xlrd': 'xlrd',
        'tkinter': 'tkinter (built-in Python)'
    }

    faltando = []
    instaladas = []

    for modulo, nome_pip in dependencias.items():
        try:
            __import__(modulo)
            instaladas.append(f"✅ {modulo}")
        except ImportError:
            faltando.append(nome_pip)

    # Mostrar resultado
    for lib in instaladas:
        print(f"   {lib}")

    if faltando:
        print("\n❌ ERRO: Bibliotecas faltando!")
        print("─" * 70)
        for lib in faltando:
            print(f"   ❌ {lib}")
        print("\n💡 Solução:")
        libs_pip = [lib for lib in faltando if 'built-in' not in lib]
        if libs_pip:
            print(f"   Execute: pip install {' '.join(libs_pip)}")
        print()
        raise ImportError(
            f"Bibliotecas necessárias não instaladas: {', '.join(faltando)}"
        )

    print("✅ Todas as dependências instaladas!\n")

# Validar ANTES de importar
validar_dependencias()

# ═══════════════════════════════════════════════════════════════════
# IMPORTS
# ═══════════════════════════════════════════════════════════════════

import pandas as pd
import numpy as np
import xlrd
import re
import json
import os
import subprocess
import platform
import shutil
from pathlib import Path
from datetime import datetime
from collections import Counter
import warnings
warnings.filterwarnings('ignore')

import tkinter as tk
from tkinter import filedialog, messagebox, ttk

print("✅ Imports carregados")

# ═══════════════════════════════════════════════════════════════════
# CLASSE: LocalizadorDicionario (SISTEMA DE PERSISTÊNCIA GLOBAL)
# ═══════════════════════════════════════════════════════════════════

class LocalizadorDicionario:
    """
    Sistema de localização persistente de dicionários entre sessões.

    Mantém log global em: ~/.processador_dicionario_localizador.json

    Métodos Públicos:
    - obter_dicionario_atual() -> Path  # Para BLOCO 2+
    - obter_pasta_base_atual() -> Path  # Para FileManager
    - obter_timestamp_atual() -> str    # Para recuperar timestamp
    - registrar_mudanca()               # Chamado por BLOCO 1
    """

    LOG_FILE = Path.home() / '.processador_dicionario_localizador.json'

    @classmethod
    def carregar_log(cls):
        """Carrega log global com fallback para encoding"""
        if cls.LOG_FILE.exists():
            for encoding in ['utf-8', 'utf-8-sig', 'latin-1']:
                try:
                    with open(cls.LOG_FILE, 'r', encoding=encoding) as f:
                        return json.load(f)
                except (UnicodeDecodeError, json.JSONDecodeError):
                    continue
        return {
            'versao': '2.1',
            'dicionario_atual': None,
            'pasta_base_atual': None,
            'timestamp': None,
            'historico': []
        }

    @classmethod
    def salvar_log(cls, log):
        """Salva log com backup automático"""
        # Backup do log anterior
        if cls.LOG_FILE.exists():
            backup_file = cls.LOG_FILE.with_suffix('.json.bak')
            shutil.copy2(cls.LOG_FILE, backup_file)

        # Salvar novo log
        with open(cls.LOG_FILE, 'w', encoding='utf-8') as f:
            json.dump(log, f, indent=2, ensure_ascii=False)

    @classmethod
    def obter_dicionario_atual(cls):
        """Retorna Path do dicionário atual (para BLOCO 2+)"""
        log = cls.carregar_log()
        if not log['dicionario_atual']:
            raise FileNotFoundError(
                "❌ Dicionário não encontrado! Execute BLOCO 1."
            )

        dicionario_path = Path(log['dicionario_atual'])
        if not dicionario_path.exists():
            raise FileNotFoundError(
                f"❌ Dicionário não existe: {dicionario_path}"
            )

        return dicionario_path

    @classmethod
    def obter_pasta_base_atual(cls):
        """Retorna Path da pasta base atual (para FileManager)"""
        log = cls.carregar_log()
        if not log['pasta_base_atual']:
            raise FileNotFoundError(
                "❌ Pasta base não encontrada! Execute BLOCO 1."
            )

        pasta_base = Path(log['pasta_base_atual'])
        if not pasta_base.exists():
            raise FileNotFoundError(
                f"❌ Pasta base não existe: {pasta_base}"
            )

        return pasta_base

    @classmethod
    def obter_timestamp_atual(cls):
        """Retorna timestamp da execução atual (para BLOCO 2+)"""
        log = cls.carregar_log()
        if not log.get('timestamp'):
            raise FileNotFoundError(
                "❌ Timestamp não encontrado! Execute BLOCO 1."
            )
        return log['timestamp']

    @classmethod
    def registrar_mudanca(cls, pasta_base, timestamp, dicionario_path=None,
                         migrado_de=None, versao_bloco1=None):
        """
        Registra mudança de localização no log global.

        IMPORTANTE: Agora aceita timestamp e registra no LOG GLOBAL.
        dicionario_path é OPCIONAL - só registra se existir.

        Args:
            pasta_base: Path da pasta container
            timestamp: String timestamp da execução
            dicionario_path: Path do dicionário (opcional)
            migrado_de: Path de onde migrou (opcional)
            versao_bloco1: Versão do BLOCO 1 que criou (opcional)
        """
        log = cls.carregar_log()

        entrada = {
            'timestamp': datetime.now().isoformat(),
            'pasta_base': str(pasta_base),
            'dicionario_path': str(dicionario_path) if dicionario_path else None,
            'timestamp_execucao': timestamp,
            'existe': dicionario_path.exists() if dicionario_path else False,
            'versao_bloco1': versao_bloco1 or 'desconhecida'
        }

        if migrado_de:
            entrada['migrado_de'] = str(migrado_de)

        log['pasta_base_atual'] = str(pasta_base)
        log['timestamp'] = timestamp

        # Só registrar dicionário se ele REALMENTE existir
        if dicionario_path and dicionario_path.exists():
            log['dicionario_atual'] = str(dicionario_path)
        else:
            log['dicionario_atual'] = None

        log['historico'].append(entrada)
        log['ultima_atualizacao'] = datetime.now().isoformat()

        cls.salvar_log(log)

        print(f"\n📍 Localizador atualizado:")
        print(f"   Container: {pasta_base.name}")
        print(f"   Timestamp: {timestamp}")
        print(f"   Versão BLOCO 1: {versao_bloco1 or 'desconhecida'}")
        if dicionario_path and dicionario_path.exists():
            print(f"   Dicionário: {dicionario_path.name}")
        print(f"   Log: {cls.LOG_FILE}")

# ═══════════════════════════════════════════════════════════════════
# CLASSE: SeletorPastaComTimer (GUI COM TIMER)
# ═══════════════════════════════════════════════════════════════════

class SeletorPastaComTimer:
    """Seletor de pasta destino com timer de 10s e memória"""

    CONFIG_FILE = Path.home() / '.processador_last_directory.json'

    def __init__(self):
        self.resultado = {'path': None, 'acao': None}
        self.timeout_ocorreu = False

    def carregar_ultima_pasta(self):
        """Carrega última pasta usada"""
        if self.CONFIG_FILE.exists():
            try:
                with open(self.CONFIG_FILE, 'r', encoding='utf-8') as f:
                    config = json.load(f)
                ultima_pasta = Path(config.get('last_directory', ''))
                if ultima_pasta.exists():
                    return ultima_pasta
            except:
                pass
        return None

    def salvar_escolha(self, pasta):
        """Salva escolha para próxima execução"""
        config = {
            'last_directory': str(pasta),
            'timestamp': datetime.now().isoformat()
        }
        with open(self.CONFIG_FILE, 'w', encoding='utf-8') as f:
            json.dump(config, f, indent=2)

    def validar_pasta_destino(self, pasta):
        """Valida se pasta tem permissões adequadas"""
        pasta = Path(pasta)

        # Verificar permissão de escrita
        if not os.access(pasta, os.W_OK):
            return False, "❌ Sem permissão de escrita"

        # Verificar espaço em disco (mínimo 100MB)
        stat = os.statvfs(pasta) if hasattr(os, 'statvfs') else None
        if stat:
            espaco_livre = stat.f_bavail * stat.f_frsize
            if espaco_livre < 100 * 1024 * 1024:  # 100MB
                return False, (
                    f"❌ Espaço insuficiente "
                    f"({espaco_livre/1024/1024:.1f}MB)"
                )

        return True, "✅ Pasta válida"

    def selecionar_com_timer(self):
        """Exibe GUI com timer de 10s"""
        ultima_pasta = self.carregar_ultima_pasta()

        root = tk.Tk()
        root.title("Processador - Pasta Destino")
        root.geometry("650x450")

        # Centralizar janela
        x = (root.winfo_screenwidth() // 2) - 325
        y = (root.winfo_screenheight() // 2) - 225
        root.geometry(f"+{x}+{y}")

        frame = tk.Frame(root, padx=20, pady=20, bg='white')
        frame.pack(fill=tk.BOTH, expand=True)

        # Título
        tk.Label(
            frame,
            text="📂 Pasta DESTINO",
            font=('Arial', 14, 'bold'),
            bg='white'
        ).pack(pady=(0, 15))

        # Mensagem
        if ultima_pasta:
            msg = f"Timer de 10s para usar:\n\n{ultima_pasta}"
        else:
            msg = "Primeira execução - selecione pasta"

        tk.Label(
            frame,
            text=msg,
            justify=tk.LEFT,
            font=('Arial', 9),
            bg='white',
            wraplength=600
        ).pack(pady=(0, 15))

        # Timer
        contador = [10]
        if ultima_pasta:
            label_timer = tk.Label(
                frame,
                text=f"{contador[0]}s",
                font=('Arial', 24, 'bold'),
                fg='#FF4444',
                bg='white'
            )
            label_timer.pack(pady=(5, 20))

            def countdown():
                if contador[0] > 0 and not self.timeout_ocorreu:
                    contador[0] -= 1
                    label_timer.config(text=f"{contador[0]}s")
                    root.after(1000, countdown)
                elif contador[0] == 0:
                    self.timeout_ocorreu = True
                    self.resultado['path'] = ultima_pasta
                    self.resultado['acao'] = 'TIMEOUT'
                    root.quit()
                    root.destroy()

            root.after(1000, countdown)

        # Botões
        def escolher_nova():
            self.timeout_ocorreu = True
            root.withdraw()
            nova_pasta = filedialog.askdirectory(
                title="Pasta DESTINO",
                initialdir=ultima_pasta if ultima_pasta else None
            )

            if nova_pasta:
                valido, msg = self.validar_pasta_destino(nova_pasta)
                if not valido:
                    messagebox.showerror("Pasta Inválida", msg)
                    self.resultado['path'] = ultima_pasta
                    self.resultado['acao'] = 'CANCELADO'
                else:
                    self.resultado['path'] = Path(nova_pasta)
                    self.resultado['acao'] = 'NOVA'
            else:
                self.resultado['path'] = ultima_pasta
                self.resultado['acao'] = 'CANCELADO'

            root.quit()
            root.destroy()

        def usar_ultima():
            self.timeout_ocorreu = True
            self.resultado['path'] = ultima_pasta
            self.resultado['acao'] = 'MANTEVE'
            root.quit()
            root.destroy()

        frame_btns = tk.Frame(frame, bg='white')
        frame_btns.pack(side=tk.BOTTOM, pady=15)

        tk.Button(
            frame_btns,
            text="📁 Nova Pasta",
            command=escolher_nova,
            width=20,
            height=2,
            font=('Arial', 10, 'bold'),
            bg='#4CAF50',
            fg='white'
        ).pack(side=tk.LEFT, padx=10)

        if ultima_pasta:
            tk.Button(
                frame_btns,
                text="✅ Usar Última",
                command=usar_ultima,
                width=20,
                height=2,
                font=('Arial', 10),
                bg='#2196F3',
                fg='white'
            ).pack(side=tk.LEFT, padx=10)

        root.mainloop()
        return self.resultado

# ═══════════════════════════════════════════════════════════════════
# CLASSE: SeletorOrigemComTimer (GUI MIGRAÇÃO)
# ═══════════════════════════════════════════════════════════════════

class SeletorOrigemComTimer:
    """Pergunta se deseja copiar arquivos de execução anterior"""

    def __init__(self):
        self.resultado = {'copiar': False, 'path': None}
        self.timeout_ocorreu = False

    def perguntar_origem(self):
        """GUI com timer de 5s (default: NÃO)"""
        root = tk.Tk()
        root.title("Processador - Copiar Arquivos Anteriores?")
        root.geometry("650x350")

        x = (root.winfo_screenwidth() // 2) - 325
        y = (root.winfo_screenheight() // 2) - 175
        root.geometry(f"+{x}+{y}")

        frame = tk.Frame(root, padx=20, pady=20, bg='white')
        frame.pack(fill=tk.BOTH, expand=True)

        tk.Label(
            frame,
            text="📂 Copiar arquivos de execução anterior?",
            font=('Arial', 14, 'bold'),
            bg='white'
        ).pack(pady=(0, 15))

        msg = (
            "Se houver dicionários, logs ou outputs anteriores,\n"
            "você pode copiá-los para a nova estrutura."
        )
        tk.Label(
            frame,
            text=msg,
            justify=tk.LEFT,
            font=('Arial', 9),
            bg='white',
            wraplength=600
        ).pack(pady=(0, 15))

        contador = [5]
        label_timer = tk.Label(
            frame,
            text=f"{contador[0]}s (auto: NÃO)",
            font=('Arial', 18, 'bold'),
            fg='#FF6600',
            bg='white'
        )
        label_timer.pack(pady=(5, 20))

        def countdown():
            if contador[0] > 0 and not self.timeout_ocorreu:
                contador[0] -= 1
                label_timer.config(text=f"{contador[0]}s (auto: NÃO)")
                root.after(1000, countdown)
            elif contador[0] == 0:
                self.timeout_ocorreu = True
                self.resultado['copiar'] = False
                root.quit()
                root.destroy()

        root.after(1000, countdown)

        def sim_copiar():
            self.timeout_ocorreu = True
            root.withdraw()
            pasta_origem = filedialog.askdirectory(
                title="Selecione pasta ORIGEM (execução anterior)"
            )
            if pasta_origem:
                self.resultado['copiar'] = True
                self.resultado['path'] = Path(pasta_origem)
            else:
                self.resultado['copiar'] = False
            root.quit()
            root.destroy()

        def nao_copiar():
            self.timeout_ocorreu = True
            self.resultado['copiar'] = False
            root.quit()
            root.destroy()

        frame_btns = tk.Frame(frame, bg='white')
        frame_btns.pack(side=tk.BOTTOM, pady=15)

        tk.Button(
            frame_btns,
            text="✅ SIM - Selecionar Origem",
            command=sim_copiar,
            width=25,
            height=2,
            font=('Arial', 10, 'bold'),
            bg='#4CAF50',
            fg='white'
        ).pack(side=tk.LEFT, padx=10)

        tk.Button(
            frame_btns,
            text="❌ NÃO - Começar do Zero",
            command=nao_copiar,
            width=25,
            height=2,
            font=('Arial', 10),
            bg='#757575',
            fg='white'
        ).pack(side=tk.LEFT, padx=10)

        root.mainloop()
        return self.resultado

# ═══════════════════════════════════════════════════════════════════
# CLASSE: LimpadorRoot (DETECÇÃO DE POLUIÇÃO)
# ═══════════════════════════════════════════════════════════════════

class LimpadorRoot:
    """Detecta e limpa pastas antigas no root"""

    def __init__(self, pasta_root):
        self.pasta_root = Path(pasta_root)

    def detectar_poluicao(self):
        """Encontra pastas numeradas antigas"""
        pastas_numeradas = [
            p for p in self.pasta_root.iterdir()
            if p.is_dir() and p.name[:2].isdigit() and '_' in p.name
        ]
        return pastas_numeradas

    def perguntar_limpeza(self, pastas):
        """GUI para decidir o que fazer com pastas antigas"""
        root = tk.Tk()
        root.title("Processador - Limpar Root?")
        root.geometry("650x400")

        x = (root.winfo_screenwidth() // 2) - 325
        y = (root.winfo_screenheight() // 2) - 200
        root.geometry(f"+{x}+{y}")

        frame = tk.Frame(root, padx=20, pady=20, bg='white')
        frame.pack(fill=tk.BOTH, expand=True)

        tk.Label(
            frame,
            text="⚠️  Pastas antigas detectadas no root",
            font=('Arial', 14, 'bold'),
            bg='white',
            fg='#FF6600'
        ).pack(pady=(0, 10))

        msg = f"Encontradas {len(pastas)} pastas soltas:\n\n"
        msg += "\n".join([f"• {p.name}" for p in pastas[:5]])
        if len(pastas) > 5:
            msg += f"\n... e mais {len(pastas)-5}"

        tk.Label(
            frame,
            text=msg,
            justify=tk.LEFT,
            font=('Arial', 9),
            bg='white',
            wraplength=600
        ).pack(pady=(0, 15))

        resultado = {'acao': None}

        def mover():
            resultado['acao'] = 'MOVER'
            root.quit()
            root.destroy()

        def deletar():
            resultado['acao'] = 'DELETAR'
            root.quit()
            root.destroy()

        def ignorar():
            resultado['acao'] = 'IGNORAR'
            root.quit()
            root.destroy()

        frame_btns = tk.Frame(frame, bg='white')
        frame_btns.pack(side=tk.BOTTOM, pady=15)

        tk.Button(
            frame_btns,
            text="📦 Mover p/ Estrutura",
            command=mover,
            width=20,
            height=2,
            font=('Arial', 9, 'bold'),
            bg='#4CAF50',
            fg='white'
        ).pack(side=tk.LEFT, padx=5)

        tk.Button(
            frame_btns,
            text="🗑️  Deletar",
            command=deletar,
            width=15,
            height=2,
            font=('Arial', 9),
            bg='#F44336',
            fg='white'
        ).pack(side=tk.LEFT, padx=5)

        tk.Button(
            frame_btns,
            text="⏭️  Ignorar",
            command=ignorar,
            width=15,
            height=2,
            font=('Arial', 9),
            bg='#757575',
            fg='white'
        ).pack(side=tk.LEFT, padx=5)

        root.mainloop()
        return resultado['acao']

# ═══════════════════════════════════════════════════════════════════
# CLASSE: GerenciadorMigracao (CÓPIA COMPLETA COM LOG)
# ═══════════════════════════════════════════════════════════════════

class GerenciadorMigracao:
    """Gerencia cópia completa de execuções anteriores"""

    def __init__(self, pasta_origem, pasta_destino_container):
        self.pasta_origem = Path(pasta_origem)
        self.pasta_destino = Path(pasta_destino_container)
        self.timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
        self.log_detalhado = []
        self.erros = []

    def detectar_estrutura(self):
        """Detecta pastas e dicionários na origem"""
        pastas = [
            p for p in self.pasta_origem.iterdir()
            if p.is_dir() and (
                p.name[:2].isdigit() or
                'dicionario' in p.name.lower()
            )
        ]

        dicionarios = []
        pasta_dict = self.pasta_origem / '05_Dicionarios'

        if pasta_dict.exists():
            dicionarios.extend(list(pasta_dict.glob('*.json')))

        dicionarios.extend(
            list(self.pasta_origem.glob('dicionario*.json'))
        )
        dicionarios = list(set(dicionarios))

        return pastas, dicionarios

    def validar_dicionario(self, arquivo_json):
        """Valida integridade do dicionário JSON"""
        try:
            with open(arquivo_json, 'r', encoding='utf-8') as f:
                data = json.load(f)

            # Verificar estrutura mínima
            if not isinstance(data, dict):
                return False, "JSON não é um dicionário"

            return True, "✅ Válido"
        except json.JSONDecodeError as e:
            return False, f"JSON inválido: {str(e)}"
        except Exception as e:
            return False, f"Erro: {str(e)}"

    def copiar_tudo(self):
        """Copia tudo com tratamento de erros"""
        pastas, dicionarios = self.detectar_estrutura()

        print(f"\n📂 MIGRAÇÃO COMPLETA")
        print("─" * 70)
        print(f"   De: {self.pasta_origem}")
        print(f"   Para: {self.pasta_destino}")
        print(f"   Pastas: {len(pastas)}")
        print(f"   Dicionários: {len(dicionarios)}")
        print()

        if not pastas and not dicionarios:
            print("ℹ️  Nada para copiar")
            return {'migrado': False}

        print("Copiar? (S/N ou Enter=S): ", end='')
        resposta = input().strip().upper()
        if resposta and resposta != 'S':
            print("❌ Migração cancelada")
            return {'migrado': False}

        print(f"\n🔄 Copiando...\n")

        arquivos_copiados = 0
        bytes_copiados = 0
        dicionarios_copiados = []

        # Copiar pastas
        for pasta in sorted(pastas):
            try:
                if pasta.name == '05_Dicionarios':
                    continue

                destino_pasta = self.pasta_destino / pasta.name
                destino_pasta.mkdir(parents=True, exist_ok=True)

                print(f"📁 {pasta.name}", end='')
                arquivos_pasta = 0
                bytes_pasta = 0

                for arquivo in pasta.rglob('*'):
                    if arquivo.is_file():
                        try:
                            destino_arq = (
                                destino_pasta /
                                arquivo.relative_to(pasta)
                            )
                            destino_arq.parent.mkdir(
                                parents=True,
                                exist_ok=True
                            )
                            shutil.copy2(arquivo, destino_arq)
                            arquivos_copiados += 1
                            arquivos_pasta += 1
                            bytes_pasta += arquivo.stat().st_size
                        except Exception as e:
                            self.erros.append({
                                'arquivo': str(arquivo),
                                'erro': str(e)
                            })

                bytes_copiados += bytes_pasta
                tamanho_kb = bytes_pasta/1024
                print(f" → {arquivos_pasta} arquivos ({tamanho_kb:.1f} KB)")

                self.log_detalhado.append({
                    'tipo': 'pasta',
                    'nome': pasta.name,
                    'arquivos': arquivos_pasta,
                    'bytes': bytes_pasta
                })

            except Exception as e:
                print(f" ❌ ERRO: {str(e)}")
                self.erros.append({
                    'pasta': pasta.name,
                    'erro': str(e)
                })

        # Copiar dicionários
        if dicionarios:
            pasta_dict_destino = self.pasta_destino / '05_Dicionarios'
            pasta_dict_destino.mkdir(parents=True, exist_ok=True)

            print(f"\n📚 DICIONÁRIOS ({len(dicionarios)}):")

            for dic in dicionarios:
                try:
                    # Validar antes de copiar
                    valido, msg = self.validar_dicionario(dic)

                    destino_dic = pasta_dict_destino / dic.name
                    shutil.copy2(dic, destino_dic)
                    tamanho = dic.stat().st_size

                    status = "✅" if valido else "⚠️"
                    tamanho_kb = tamanho/1024
                    print(f"   {status} {dic.name} ({tamanho_kb:.1f} KB) - {msg}")

                    dicionarios_copiados.append(str(destino_dic))
                    arquivos_copiados += 1
                    bytes_copiados += tamanho

                    self.log_detalhado.append({
                        'tipo': 'dicionario',
                        'nome': dic.name,
                        'bytes': tamanho,
                        'path': str(destino_dic),
                        'validado': valido
                    })

                except Exception as e:
                    print(f"   ❌ {dic.name}: {str(e)}")
                    self.erros.append({
                        'dicionario': dic.name,
                        'erro': str(e)
                    })

        total_mb = bytes_copiados/1024/1024
        print(f"\n✅ TOTAL: {arquivos_copiados} arquivos, {total_mb:.2f} MB")

        if self.erros:
            print(f"⚠️  {len(self.erros)} erros durante cópia (ver log)")

        self._salvar_log_local({
            'migrado': True,
            'arquivos': arquivos_copiados,
            'bytes': bytes_copiados,
            'dicionarios': len(dicionarios_copiados),
            'erros': len(self.erros),
            'detalhes': self.log_detalhado,
            'log_erros': self.erros
        })

        return {
            'migrado': True,
            'arquivos': arquivos_copiados,
            'dicionarios': dicionarios_copiados,
            'erros': self.erros
        }

    def _salvar_log_local(self, info):
        """Salva log detalhado da migração"""
        log_file = self.pasta_destino / 'log_migracoes.json'

        if log_file.exists():
            with open(log_file, 'r', encoding='utf-8') as f:
                historico = json.load(f)
        else:
            historico = {'migracoes': []}

        entrada = {
            'timestamp': self.timestamp,
            'data_hora': datetime.now().isoformat(),
            'pasta_origem': str(self.pasta_origem),
            'pasta_destino': str(self.pasta_destino),
            **info
        }

        historico['migracoes'].append(entrada)
        historico['ultima_migracao'] = self.timestamp

        with open(log_file, 'w', encoding='utf-8') as f:
            json.dump(historico, f, indent=2, ensure_ascii=False)

        print(f"💾 Log: {log_file.name}")

# ═══════════════════════════════════════════════════════════════════
# CLASSE: FileManagerInterativo (GERENCIADOR DE ARQUIVOS)
# ═══════════════════════════════════════════════════════════════════

class FileManagerInterativo:
    """Gerenciador de arquivos e estrutura de pastas"""

    def __init__(self, base_path=None):
        self.base_path = Path(base_path) if base_path else Path.cwd()

        # Estrutura de pastas padrão
        self.pastas = {
            'entrada': self.base_path / '01_Entrada',
            'processados': self.base_path / '02_Processados',
            'outputs': self.base_path / '03_Outputs',
            'logs': self.base_path / '04_Logs',
            'dicionarios': self.base_path / '05_Dicionarios',
            'codigos_integracao': self.base_path / '06_Codigos_Integracao'
        }

        # Criar todas as pastas
        for pasta in self.pastas.values():
            pasta.mkdir(parents=True, exist_ok=True)

        self.timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')

        print(f"✅ FileManager inicializado")
        print(f"   📂 Container: {self.base_path.name}")
        print(f"   🕐 Timestamp: {self.timestamp}")

    def salvar(self, df, nome, tipo='xlsx', pasta='processados'):
        """Salva DataFrame na pasta especificada"""
        arquivo = (
            self.pastas[pasta] /
            f"{nome}_{self.timestamp}.{tipo}"
        )

        if tipo == 'xlsx':
            df.to_excel(arquivo, index=False, engine='openpyxl')
        elif tipo == 'csv':
            df.to_csv(arquivo, index=False, encoding='utf-8-sig')

        return arquivo

    def abrir_pasta(self, pasta):
        """Abre pasta no explorer do sistema"""
        caminho = self.pastas[pasta]
        sistema = platform.system()

        try:
            if sistema == 'Windows':
                os.startfile(caminho)
            elif sistema == 'Darwin':  # macOS
                subprocess.run(['open', caminho])
            else:  # Linux
                subprocess.run(['xdg-open', caminho])
        except Exception as e:
            print(f"⚠️  Não foi possível abrir pasta: {e}")

# ═══════════════════════════════════════════════════════════════════
# FUNÇÃO: gerar_readme
# ═══════════════════════════════════════════════════════════════════

def gerar_readme(pasta_base, versao_bloco1):
    """Gera README.md na pasta container"""
    readme = f"""# 📚 PROCESSADOR DE ARQUIVOS DESCONHECIDOS

**Gerado:** {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}
**Pasta:** {pasta_base}
**Versão BLOCO 1:** {versao_bloco1}

---

## 📁 ESTRUTURA

```
{pasta_base.name}/
├── 01_Entrada/          ← Arquivos originais
├── 02_Processados/      ← Dados limpos
├── 03_Outputs/          ← Resultados finais
├── 04_Logs/             ← Logs de execução ⭐ COMUNICAÇÃO VIA LOG
├── 05_Dicionarios/      ← Mapeamentos DE-PARA
├── 06_Codigos_Integracao/ ← Scripts reutilizáveis
├── README.md            ← Este arquivo
└── log_migracoes.json   ← Histórico de migrações
```

---

## 📚 LOCALIZADOR DE DICIONÁRIO

**Para notebooks consumidores (BLOCO 2+):**

```python
from bloco1 import LocalizadorDicionario

# Obter dicionário atual
dicionario_path = LocalizadorDicionario.obter_dicionario_atual()

# Obter pasta base
pasta_base = LocalizadorDicionario.obter_pasta_base_atual()

# Obter timestamp da execução
timestamp = LocalizadorDicionario.obter_timestamp_atual()

# Carregar dicionário
import json
with open(dicionario_path, 'r', encoding='utf-8') as f:
    dicionario = json.load(f)
```

**Log global:** `~/.processador_dicionario_localizador.json`

---

## 🔗 COMUNICAÇÃO ENTRE BLOCOS (0% MEMÓRIA, 100% LOG)

Todos os blocos seguem o padrão:

1. **LER** do LOG GLOBAL:
   - pasta_base_atual
   - timestamp
   - dicionario_atual (se existir)

2. **RECRIAR** objetos localmente:
   - FileManager(pasta_base)
   - Carregar dicionário de 04_Logs/

3. **PROCESSAR** dados do bloco

4. **SALVAR** estado em 04_Logs/:
   - .bloco_N_state.json
   - Dados específicos do bloco

---

## 🔄 HISTÓRICO DE MIGRAÇÕES

Ver: `log_migracoes.json`

---

## 📋 ESTADO DO BLOCO 1

Ver: `04_Logs/.bloco_1_state.json`

---

## 🆘 SUPORTE

- Erros: `04_Logs/`
- Dicionário perdido: Execute BLOCO 1
- Migração: Consulte `log_migracoes.json`
- Versão do código: `{versao_bloco1}`
"""

    readme_path = pasta_base / 'README.md'
    with open(readme_path, 'w', encoding='utf-8') as f:
        f.write(readme)

    print(f"📖 README: {readme_path.name}")

# ═══════════════════════════════════════════════════════════════════
# EXECUÇÃO PRINCIPAL DO BLOCO 1
# ═══════════════════════════════════════════════════════════════════

print("\n" + "="*70)
print("🔵 ETAPA 1: SELECIONANDO PASTA DESTINO...")
print("="*70)

seletor = SeletorPastaComTimer()
resultado_destino = seletor.selecionar_com_timer()

if not resultado_destino['path']:
    print("❌ Nenhuma pasta selecionada")
    raise ValueError("Execução cancelada")

print(f"\n✅ Destino: {resultado_destino['path']}")
print(f"   Ação: {resultado_destino['acao']}")
seletor.salvar_escolha(resultado_destino['path'])

pasta_root_destino = resultado_destino['path']

# ═══════════════════════════════════════════════════════════════════
print("\n" + "="*70)
print("🔵 ETAPA 2: VERIFICANDO POLUIÇÃO NO ROOT...")
print("="*70)

limpador = LimpadorRoot(pasta_root_destino)
pastas_poluidas = limpador.detectar_poluicao()

acao = None
if pastas_poluidas:
    print(f"\n⚠️  {len(pastas_poluidas)} pastas antigas no root!")
    acao = limpador.perguntar_limpeza(pastas_poluidas)

    if acao == 'DELETAR':
        print("\n🗑️  Deletando...")
        for pasta in pastas_poluidas:
            try:
                shutil.rmtree(pasta)
                print(f"   ✅ {pasta.name}")
            except Exception as e:
                print(f"   ❌ {pasta.name}: {e}")

    elif acao == 'MOVER':
        print("\n📦 Mover será feito após criar container")

    else:
        print("\n⏭️  Ignorando pastas antigas")
else:
    print("✅ Root limpo")

# ═══════════════════════════════════════════════════════════════════
print("\n" + "="*70)
print("🔵 ETAPA 3: CRIANDO PASTA CONTAINER...")
print("="*70)

timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
nome_container = f"PROCESSAR_ARQUIVOS_{timestamp}"
pasta_container = pasta_root_destino / nome_container

pasta_container.mkdir(parents=True, exist_ok=True)
print(f"✅ Container: {nome_container}")

# ═══════════════════════════════════════════════════════════════════
print("\n" + "="*70)
print("🔵 ETAPA 4: COPIAR DE EXECUÇÃO ANTERIOR?")
print("="*70)

seletor_origem = SeletorOrigemComTimer()
resultado_origem = seletor_origem.perguntar_origem()

dicionarios_migrados = []
info_mig = {}

if resultado_origem['copiar'] and resultado_origem['path']:
    print(f"\n📂 Origem: {resultado_origem['path']}")
    gerenciador_mig = GerenciadorMigracao(
        resultado_origem['path'],
        pasta_container
    )
    info_mig = gerenciador_mig.copiar_tudo()

    if info_mig.get('migrado'):
        print(f"\n✅ Migração concluída")
        if info_mig.get('dicionarios'):
            dicionarios_migrados = info_mig['dicionarios']
            print(f"   📚 {len(dicionarios_migrados)} dicionários copiados")
        if info_mig.get('erros'):
            print(f"   ⚠️  {len(info_mig['erros'])} erros (ver log)")
else:
    print("✅ Começando do zero (sem cópia)")

# Mover pastas antigas se solicitado
if pastas_poluidas and acao == 'MOVER':
    print("\n📦 Movendo pastas antigas para container...")
    for pasta in pastas_poluidas:
        try:
            destino = pasta_container / pasta.name
            if destino.exists():
                shutil.rmtree(destino)
            shutil.move(str(pasta), str(destino))
            print(f"   ✅ {pasta.name}")
        except Exception as e:
            print(f"   ❌ {pasta.name}: {e}")

# ═══════════════════════════════════════════════════════════════════
print("\n" + "="*70)
print("🔵 ETAPA 5: INICIALIZANDO FILEMANAGER...")
print("="*70)

fm = FileManagerInterativo(pasta_container)

# Detectar dicionário migrado (se existir)
pasta_dict = fm.pastas['dicionarios']
arquivos_dict = list(pasta_dict.glob('*.json'))

dicionario_existente = None
if arquivos_dict:
    # Usar o mais recente
    dicionario_existente = max(
        arquivos_dict,
        key=lambda p: p.stat().st_mtime
    )
    print(f"📚 Dicionário detectado: {dicionario_existente.name}")

# Registrar no localizador com timestamp E versão
LocalizadorDicionario.registrar_mudanca(
    pasta_base=pasta_container,
    timestamp=timestamp,
    dicionario_path=dicionario_existente,  # None se não existir
    versao_bloco1=VERSAO_BLOCO1
)

# Gerar README
gerar_readme(pasta_container, VERSAO_BLOCO1)

# ═══════════════════════════════════════════════════════════════════
# MELHORIA 1: SALVAR ESTADO LOCAL (NOVO v4.4)
# ═══════════════════════════════════════════════════════════════════

print("\n" + "="*70)
print("🔵 ETAPA 6: SALVANDO ESTADO LOCAL...")
print("="*70)

# Calcular tamanho total do container
tamanho_total = sum(
    f.stat().st_size for f in pasta_container.rglob('*') if f.is_file()
) / 1024 / 1024

estado_bloco1 = {
    'bloco': 1,
    'versao': VERSAO_BLOCO1,
    'versao_codigo': VERSAO_BLOCO1,
    'data_versao': DATA_VERSAO,
    'timestamp_execucao': timestamp,
    'timestamp_registro': datetime.now().isoformat(),
    'status': 'concluido',
    'pasta_container': {
        'nome': pasta_container.name,
        'caminho': str(pasta_container),
        'tamanho_mb': round(tamanho_total, 2)
    },
    'filemanager': {
        'base_path': str(fm.base_path),
        'timestamp': fm.timestamp,
        'pastas_criadas': list(fm.pastas.keys())
    },
    'migracao': {
        'realizada': resultado_origem.get('copiar', False),
        'arquivos_migrados': info_mig.get('arquivos', 0),
        'pasta_origem': str(resultado_origem.get('path', '')) if resultado_origem.get('copiar') else None
    },
    'localizador': {
        'log_file': str(LocalizadorDicionario.LOG_FILE),
        'pasta_base_registrada': str(pasta_container),
        'timestamp_registrado': timestamp,
        'dicionario_registrado': str(dicionario_existente) if dicionario_existente else None
    },
    'poluicao_root': {
        'detectada': len(pastas_poluidas) if pastas_poluidas else 0,
        'acao_tomada': acao if pastas_poluidas else 'NENHUMA'
    },
    'changelog': CHANGELOG_V44
}

arquivo_estado = fm.pastas['logs'] / '.bloco_1_state.json'
with open(arquivo_estado, 'w', encoding='utf-8') as f:
    json.dump(estado_bloco1, f, indent=2, ensure_ascii=False)

print(f"✅ Estado local salvo")
print(f"   📄 {arquivo_estado.name}")
print(f"   📊 Tamanho container: {tamanho_total:.2f} MB")

# ═══════════════════════════════════════════════════════════════════
print("\n" + "="*70)
print("✅ BLOCO 1 v4.4 CONCLUÍDO COM SUCESSO")
print("="*70)
print(f"\n📂 Container: {pasta_container}")
print(f"🕐 Timestamp: {timestamp}")
print(f"📍 Localizador: {LocalizadorDicionario.LOG_FILE}")
print(f"🔖 Versão: {VERSAO_BLOCO1}")
print(f"\n📋 Estrutura criada:")
for nome, pasta in fm.pastas.items():
    print(f"   • {pasta.name}")
print(f"\n💾 Arquivos de estado:")
print(f"   • LOG GLOBAL: {LocalizadorDicionario.LOG_FILE.name}")
print(f"   • Estado local: {arquivo_estado.name}")
print(f"   • README: README.md")
print("\n" + "="*70)
print("\n💡 Próximo: BLOCO 2 vai ler configuração do LOG GLOBAL")
print("="*70)

 🔍 PROCESSADOR DE ARQUIVOS DESCONHECIDOS v4.4
 Versão: 4.4 | Data: 2025-10-17
 Timer | Migração | Dicionários | Validações | Logs | Estado

🔍 Validando dependências...
   ✅ pandas
   ✅ numpy
   ✅ openpyxl
   ✅ xlrd
   ✅ tkinter
✅ Todas as dependências instaladas!

✅ Imports carregados

🔵 ETAPA 1: SELECIONANDO PASTA DESTINO...

✅ Destino: E:\OneDrive - VIBRA\NMCV - Documentos\Indicador\_DataLake\2- Dados Processados (PROCESSED)
   Ação: MANTEVE

🔵 ETAPA 2: VERIFICANDO POLUIÇÃO NO ROOT...
✅ Root limpo

🔵 ETAPA 3: CRIANDO PASTA CONTAINER...
✅ Container: PROCESSAR_ARQUIVOS_20251018_222445

🔵 ETAPA 4: COPIAR DE EXECUÇÃO ANTERIOR?
✅ Começando do zero (sem cópia)

🔵 ETAPA 5: INICIALIZANDO FILEMANAGER...
✅ FileManager inicializado
   📂 Container: PROCESSAR_ARQUIVOS_20251018_222445
   🕐 Timestamp: 20251018_222445

📍 Localizador atualizado:
   Container: PROCESSAR_ARQUIVOS_20251018_222445
   Timestamp: 20251018_222445
   Versão BLOCO 1: 4.4
   Log: C:\Users\fpsou\.processador_dicionario_localiza

In [28]:
# ===================================================================
# BLOCO 2: CLASSES AUXILIARES
# Versao: 4.3 - REVISADO (COMUNICACAO VIA LOG COMPLETA)
# ===================================================================

import pandas as pd
import numpy as np
import tkinter as tk
from tkinter import filedialog, messagebox
from pathlib import Path
import json
import re
from datetime import datetime
import warnings
from collections import Counter
import os
import platform
import subprocess

warnings.filterwarnings('ignore')

print("\n" + "="*70)
print("BLOCO 2: CLASSES AUXILIARES v4.3 REVISADO")
print("="*70)

# ===================================================================
# CLASSE: LocalizadorDicionario (INTEGRADA DO BLOCO 1)
# ===================================================================

class LocalizadorDicionario:
    """
    Sistema de localizacao persistente de dicionarios entre
    sessoes.

    Mantem log global em: ~/.processador_dicionario_localizador.json
    """

    LOG_FILE = Path.home() / '.processador_dicionario_localizador.json'

    @classmethod
    def carregar_log(cls):
        """Carrega log global com fallback para encoding"""
        if cls.LOG_FILE.exists():
            for encoding in ['utf-8', 'utf-8-sig', 'latin-1']:
                try:
                    with open(cls.LOG_FILE, 'r', encoding=encoding) as f:
                        return json.load(f)
                except (UnicodeDecodeError, json.JSONDecodeError):
                    continue
        return {
            'versao': '2.0',
            'dicionario_atual': None,
            'pasta_base_atual': None,
            'historico': []
        }

    @classmethod
    def obter_dicionario_atual(cls):
        """Retorna Path do dicionario atual"""
        log = cls.carregar_log()
        if not log['dicionario_atual']:
            raise FileNotFoundError(
                "Dicionario nao encontrado! Execute BLOCO 1."
            )

        dicionario_path = Path(log['dicionario_atual'])
        if not dicionario_path.exists():
            raise FileNotFoundError(
                f"Dicionario nao existe: {dicionario_path}"
            )

        return dicionario_path

    @classmethod
    def obter_pasta_base_atual(cls):
        """Retorna Path da pasta base atual"""
        log = cls.carregar_log()
        if not log['pasta_base_atual']:
            raise FileNotFoundError(
                "Pasta base nao encontrada! Execute BLOCO 1."
            )

        pasta_base = Path(log['pasta_base_atual'])
        if not pasta_base.exists():
            raise FileNotFoundError(
                f"Pasta base nao existe: {pasta_base}"
            )

        return pasta_base

# ===================================================================
# CLASSE: FileManagerInterativo (INTEGRADA DO BLOCO 1)
# ===================================================================

class FileManagerInterativo:
    """Gerenciador de arquivos e estrutura de pastas"""

    def __init__(self, base_path):
        self.base_path = Path(base_path)

        # Estrutura de pastas padrao
        self.pastas = {
            'entrada': self.base_path / '01_Entrada',
            'processados': self.base_path / '02_Processados',
            'outputs': self.base_path / '03_Outputs',
            'logs': self.base_path / '04_Logs',
            'dicionarios': self.base_path / '05_Dicionarios',
            'codigos_integracao': self.base_path / '06_Codigos_Integracao'
        }

        # Criar todas as pastas
        for pasta in self.pastas.values():
            pasta.mkdir(parents=True, exist_ok=True)

        self.timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')

    def salvar(self, df, nome, tipo='xlsx', pasta='processados'):
        """Salva DataFrame na pasta especificada"""
        arquivo = self.pastas[pasta] / f"{nome}_{self.timestamp}.{tipo}"

        if tipo == 'xlsx':
            df.to_excel(arquivo, index=False, engine='openpyxl')
        elif tipo == 'csv':
            df.to_csv(arquivo, index=False, encoding='utf-8-sig')

        return arquivo

    def abrir_pasta(self, pasta):
        """Abre pasta no explorer do sistema"""
        caminho = self.pastas[pasta]
        sistema = platform.system()

        try:
            if sistema == 'Windows':
                os.startfile(caminho)
            elif sistema == 'Darwin':  # macOS
                subprocess.run(['open', caminho])
            else:  # Linux
                subprocess.run(['xdg-open', caminho])
        except Exception as e:
            print(f"Nao foi possivel abrir pasta: {e}")

# ===================================================================
# CLASSE: SeletorArquivo (GUI COM TIMER E VALIDACOES)
# ===================================================================

class SeletorArquivo:
    """Seletor de arquivo com timer de 10s e validacoes robustas"""

    CONFIG_FILE = Path.home() / '.processador_last_file.json'

    def __init__(self):
        self.resultado = {'path': None, 'acao': None}
        self.timeout_ocorreu = False

    def carregar_ultimo_arquivo(self):
        """Carrega ultimo arquivo usado"""
        if self.CONFIG_FILE.exists():
            try:
                with open(self.CONFIG_FILE, 'r', encoding='utf-8') as f:
                    config = json.load(f)
                ultimo_arquivo = Path(config.get('last_file', ''))
                if ultimo_arquivo.exists():
                    return ultimo_arquivo
            except:
                pass
        return None

    def salvar_escolha(self, arquivo):
        """Salva escolha para proxima execucao"""
        config = {
            'last_file': str(arquivo),
            'timestamp': datetime.now().isoformat()
        }
        with open(self.CONFIG_FILE, 'w', encoding='utf-8') as f:
            json.dump(config, f, indent=2)

    def validar_arquivo(self, arquivo_path):
        """Valida se arquivo e adequado para processamento"""
        arquivo = Path(arquivo_path)

        # Verificar existencia
        if not arquivo.exists():
            return False, "Arquivo nao existe"

        # Verificar se e arquivo (nao diretorio)
        if not arquivo.is_file():
            return False, "Nao e um arquivo"

        # Verificar permissao de leitura
        if not os.access(arquivo, os.R_OK):
            return False, "Sem permissao de leitura"

        # Verificar tamanho (maximo 500MB)
        tamanho_mb = arquivo.stat().st_size / (1024 * 1024)
        if tamanho_mb > 500:
            return False, f"Arquivo muito grande ({tamanho_mb:.1f}MB)"

        # Verificar extensao
        extensoes_validas = {'.xlsx', '.xls', '.csv', '.txt'}
        if arquivo.suffix.lower() not in extensoes_validas:
            return False, f"Extensao invalida ({arquivo.suffix})"

        return True, "Arquivo valido"

    def selecionar_com_timer(self):
        """Exibe GUI com timer de 10s"""
        ultimo_arquivo = self.carregar_ultimo_arquivo()

        root = tk.Tk()
        root.title("Processador - Selecionar Arquivo")
        root.geometry("650x450")

        # Centralizar janela
        x = (root.winfo_screenwidth() // 2) - 325
        y = (root.winfo_screenheight() // 2) - 225
        root.geometry(f"+{x}+{y}")

        frame = tk.Frame(root, padx=20, pady=20, bg='white')
        frame.pack(fill=tk.BOTH, expand=True)

        # Titulo
        tk.Label(
            frame,
            text="Selecionar Arquivo",
            font=('Arial', 14, 'bold'),
            bg='white'
        ).pack(pady=(0, 15))

        # Mensagem
        if ultimo_arquivo:
            msg = f"Timer de 10s para usar:\n\n{ultimo_arquivo.name}"
        else:
            msg = "Primeira execucao - selecione arquivo"

        tk.Label(
            frame,
            text=msg,
            justify=tk.LEFT,
            font=('Arial', 9),
            bg='white',
            wraplength=600
        ).pack(pady=(0, 15))

        # Timer
        contador = [15]
        if ultimo_arquivo:
            label_timer = tk.Label(
                frame,
                text=f"{contador[0]}s",
                font=('Arial', 24, 'bold'),
                fg='#FF4444',
                bg='white'
            )
            label_timer.pack(pady=(5, 20))

            def countdown():
                if contador[0] > 0 and not self.timeout_ocorreu:
                    contador[0] -= 1
                    label_timer.config(text=f"{contador[0]}s")
                    root.after(1000, countdown)
                elif contador[0] == 0:
                    self.timeout_ocorreu = True
                    self.resultado['path'] = ultimo_arquivo
                    self.resultado['acao'] = 'TIMEOUT'
                    root.quit()
                    root.destroy()

            root.after(1000, countdown)

        # Botoes
        def escolher_novo():
            self.timeout_ocorreu = True
            root.withdraw()

            novo_arquivo = filedialog.askopenfilename(
                title="Selecionar Arquivo",
                initialdir=ultimo_arquivo.parent if ultimo_arquivo else None,
                filetypes=[
                    ("Arquivos suportados", "*.xlsx;*.xls;*.csv;*.txt"),
                    ("Excel", "*.xlsx;*.xls"),
                    ("CSV", "*.csv"),
                    ("Todos", "*.*")
                ]
            )

            if novo_arquivo:
                valido, msg = self.validar_arquivo(novo_arquivo)
                if not valido:
                    messagebox.showerror("Arquivo Invalido", msg)
                    self.resultado['path'] = ultimo_arquivo
                    self.resultado['acao'] = 'CANCELADO'
                else:
                    self.resultado['path'] = Path(novo_arquivo)
                    self.resultado['acao'] = 'NOVO'
            else:
                self.resultado['path'] = ultimo_arquivo
                self.resultado['acao'] = 'CANCELADO'

            root.quit()
            root.destroy()

        def usar_ultimo():
            self.timeout_ocorreu = True
            self.resultado['path'] = ultimo_arquivo
            self.resultado['acao'] = 'MANTEVE'
            root.quit()
            root.destroy()

        frame_btns = tk.Frame(frame, bg='white')
        frame_btns.pack(side=tk.BOTTOM, pady=15)

        tk.Button(
            frame_btns,
            text="Novo Arquivo",
            command=escolher_novo,
            width=20,
            height=2,
            font=('Arial', 10, 'bold'),
            bg='#4CAF50',
            fg='white'
        ).pack(side=tk.LEFT, padx=10)

        if ultimo_arquivo:
            tk.Button(
                frame_btns,
                text="Usar Ultimo",
                command=usar_ultimo,
                width=20,
                height=2,
                font=('Arial', 10),
                bg='#2196F3',
                fg='white'
            ).pack(side=tk.LEFT, padx=10)

        root.mainloop()
        return self.resultado

# ===================================================================
# CLASSE: DetectorCabecalho (ANALISE INTELIGENTE COM LOG)
# ===================================================================

class DetectorCabecalho:
    """
    Detecta automaticamente a linha de cabecalho em arquivos.

    Usa sistema de scoring baseado em:
    - Preenchimento (70%+ colunas com dados)
    - Tipo String (80%+ colunas texto)
    - Valores unicos (indicador de rotulos)
    - Palavras-chave tipicas de cabecalho
    - Posicao na planilha (primeiras linhas tem prioridade)
    """

    # CONFIGURACAO EXTERNALIZAVEL
    PALAVRAS_CHAVE_PADRAO = [
        'codigo', 'nome', 'descri', 'data', 'valor', 'quantidade',
        'centro', 'produto', 'material', 'sigla', 'tipo', 'grupo'
    ]

    def __init__(self, df, palavras_chave=None):
        self.df = df
        self.scores = []
        self.log_decisoes = []
        self.palavras_chave = (
            palavras_chave if palavras_chave
            else self.PALAVRAS_CHAVE_PADRAO
        )

    def detectar(self, n_linhas=50):
        """
        Analisa primeiras n linhas e retorna indice do cabecalho.

        Args:
            n_linhas: Numero de linhas a analisar

        Returns:
            dict: {
                'indice': int,  # Linha detectada como cabecalho
                'score': float,  # Confianca da deteccao (0-1)
                'metodo': str,   # Como foi detectado
                'scores_todas_linhas': list,  # Para debug
                'log_decisoes': list  # Historico de analise
            }
        """
        n_linhas = min(n_linhas, len(self.df))

        for i in range(n_linhas):
            linha = self.df.iloc[i]
            score = 0
            detalhes = {'linha': i, 'criterios': {}}

            # Criterio 1: Preenchimento (30 pontos)
            preenchimento = linha.notna().sum() / len(linha)
            if preenchimento >= 0.7:
                pontos = 30 * (preenchimento - 0.7) / 0.3
                score += pontos
                detalhes['criterios']['preenchimento'] = (
                    f"{preenchimento:.1%} (+{pontos:.1f})"
                )

            # Criterio 2: Tipo String (30 pontos)
            tipos_string = sum(isinstance(v, str) for v in linha)
            proporcao_string = tipos_string / len(linha)
            if proporcao_string >= 0.8:
                pontos = 30 * (proporcao_string - 0.8) / 0.2
                score += pontos
                detalhes['criterios']['strings'] = (
                    f"{proporcao_string:.1%} (+{pontos:.1f})"
                )

            # Criterio 3: Valores unicos (20 pontos)
            valores_unicos = len(
                set(str(v) for v in linha if pd.notna(v))
            )
            if valores_unicos >= len(linha) * 0.8:
                pontos = 20
                score += pontos
                detalhes['criterios']['unicos'] = (
                    f"{valores_unicos}/{len(linha)} (+{pontos})"
                )

            # Criterio 4: Palavras-chave (10 pontos)
            texto_linha = ' '.join(
                str(v).lower() for v in linha if pd.notna(v)
            )
            palavras_encontradas = sum(
                1 for p in self.palavras_chave if p in texto_linha
            )
            if palavras_encontradas > 0:
                pontos = min(10, palavras_encontradas * 3)
                score += pontos
                detalhes['criterios']['palavras'] = (
                    f"{palavras_encontradas} palavras (+{pontos})"
                )

            # Criterio 5: Posicao (10 pontos)
            # Primeiras linhas tem vantagem
            if i < 50:
                pontos = 10 * (1 - (i / 50))
                score += pontos
                detalhes['criterios']['posicao'] = (
                    f"linha {i} (+{pontos:.1f})"
                )

            detalhes['score_total'] = score
            self.scores.append(score)
            self.log_decisoes.append(detalhes)

        # Encontrar melhor score
        melhor_indice = self.scores.index(max(self.scores))
        melhor_score = self.scores[melhor_indice]

        # Normalizar score para 0-1
        score_normalizado = min(1.0, melhor_score / 100)

        resultado = {
            'indice': melhor_indice,
            'score': score_normalizado,
            'metodo': 'SCORING_AUTOMATICO',
            'scores_todas_linhas': self.scores,
            'log_decisoes': self.log_decisoes
        }

        return resultado

# ===================================================================
# INICIALIZACAO DO FILEMANAGER (CONECTANDO COM BLOCO 1)
# ===================================================================

print("\n" + "="*70)
print("INICIALIZANDO FILEMANAGER - CONECTANDO COM BLOCO 1")
print("="*70)

try:
    pasta_base = LocalizadorDicionario.obter_pasta_base_atual()

    print(f"Container do BLOCO 1 encontrado!")
    print(f"   {pasta_base}")

    fm = FileManagerInterativo(pasta_base)

except FileNotFoundError as e:
    print(f"\n{e}")
    print("\nATENCAO: Execute o BLOCO 1 primeiro!")
    raise

# ===================================================================
# SALVAR ESTADO DO BLOCO 2 NO LOG
# ===================================================================

estado_bloco2 = {
    'bloco': 2,
    'versao': '4.3',
    'status': 'concluido',
    'timestamp': datetime.now().isoformat(),
    'classes_carregadas': [
        'LocalizadorDicionario',
        'FileManagerInterativo',
        'SeletorArquivo',
        'DetectorCabecalho'
    ],
    'filemanager': {
        'base_path': str(fm.base_path),
        'timestamp': fm.timestamp
    }
}

arquivo_estado = fm.pastas['logs'] / '.bloco_2_state.json'
with open(arquivo_estado, 'w', encoding='utf-8') as f:
    json.dump(estado_bloco2, f, indent=2, ensure_ascii=False)

print("\n" + "="*70)
print("BLOCO 2 CONCLUIDO")
print("="*70)
print("\nClasses carregadas:")
print("   LocalizadorDicionario ........... OK")
print("   FileManagerInterativo ........... OK")
print("   SeletorArquivo .................. OK")
print("   DetectorCabecalho ............... OK")
print("\nFileManager ativo:")
print(f"   Base: {fm.base_path}")
print(f"   Timestamp: {fm.timestamp}")
print("\nEstrutura de pastas:")
for nome, pasta in fm.pastas.items():
    print(f"   {nome.ljust(20)}: {pasta.name}")
print("\nEstado salvo:")
print(f"   {arquivo_estado.name}")
print("\n" + "="*70)
print("Digite 'BLOCO 2 OK' para prosseguir ao BLOCO 3")
print("="*70)


BLOCO 2: CLASSES AUXILIARES v4.3 REVISADO

INICIALIZANDO FILEMANAGER - CONECTANDO COM BLOCO 1
Container do BLOCO 1 encontrado!
   E:\OneDrive - VIBRA\NMCV - Documentos\Indicador\_DataLake\2- Dados Processados (PROCESSED)\PROCESSAR_ARQUIVOS_20251018_222445

BLOCO 2 CONCLUIDO

Classes carregadas:
   LocalizadorDicionario ........... OK
   FileManagerInterativo ........... OK
   SeletorArquivo .................. OK
   DetectorCabecalho ............... OK

FileManager ativo:
   Base: E:\OneDrive - VIBRA\NMCV - Documentos\Indicador\_DataLake\2- Dados Processados (PROCESSED)\PROCESSAR_ARQUIVOS_20251018_222445
   Timestamp: 20251018_222447

Estrutura de pastas:
   entrada             : 01_Entrada
   processados         : 02_Processados
   outputs             : 03_Outputs
   logs                : 04_Logs
   dicionarios         : 05_Dicionarios
   codigos_integracao  : 06_Codigos_Integracao

Estado salvo:
   .bloco_2_state.json

Digite 'BLOCO 2 OK' para prosseguir ao BLOCO 3


In [25]:
# ===================================================================
# BLOCO 3: DICIONÁRIO INTELIGENTE + CLASSE GUI COM TIMER
# Versão: v5.0 - Com persistência completa de variáveis/objetos
# Mudança v4.5 → v5.0: Adiciona recuperação de estado
# ===================================================================

import pandas as pd
import numpy as np
import json
import re
import tkinter as tk
from pathlib import Path
from datetime import datetime
import pickle

print("\n" + "="*70)
print("BLOCO 3: DICIONÁRIO INTELIGENTE + GUI COM TIMER")
print("="*70)

# ===================================================================
# 1. LER CONFIGURAÇÕES DO BLOCO ANTERIOR (VIA LOG)
# ===================================================================

log_global = Path.home() / '.processador_dicionario_localizador.json'

if not log_global.exists():
    raise FileNotFoundError(
        "❌ LOG GLOBAL não encontrado!\n"
        "   Execute BLOCO 1 primeiro."
    )

with open(log_global, 'r', encoding='utf-8') as f:
    config = json.load(f)

pasta_base = Path(config['pasta_base_atual'])
timestamp_execucao = config['timestamp']

print(f"\n✅ CONFIGURAÇÃO CARREGADA DO LOG GLOBAL")
print(f"   📁 Pasta base: {pasta_base.name}")
print(f"   🕐 Timestamp: {timestamp_execucao}")

# ===================================================================
# 2. VALIDAR QUE BLOCO 2 FOI EXECUTADO
# ===================================================================

log_bloco2 = pasta_base / '04_Logs' / '.bloco_2_state.json'

if not log_bloco2.exists():
    raise FileNotFoundError(
        "❌ BLOCO 2 não foi executado!\n"
        "   Execute BLOCO 2 primeiro."
    )

with open(log_bloco2, 'r', encoding='utf-8') as f:
    estado_bloco2 = json.load(f)

print(f"\n✅ BLOCO 2 VALIDADO")
print(f"   Executado em: {estado_bloco2['timestamp']}")
print(f"   Classes: {', '.join(estado_bloco2['classes_carregadas'])}")

# ===================================================================
# 3. RECUPERAR/RECRIAR FILEMANAGER (0% MEMÓRIA, 100% LOG)
# ===================================================================

print("\n" + "="*70)
print("RECUPERAÇÃO DE ESTADO")
print("="*70)

class FileManagerInterativo:
    """Gerenciador de arquivos"""
    def __init__(self, base_path):
        self.base_path = Path(base_path)
        self.pastas = {
            'entrada': self.base_path / '01_Entrada',
            'processados': self.base_path / '02_Processados',
            'outputs': self.base_path / '03_Outputs',
            'logs': self.base_path / '04_Logs',
            'dicionarios': self.base_path / '05_Dicionarios',
            'codigos': self.base_path / '06_Codigos_Integracao'
        }

# Tentar recuperar fm da memória OU recriar
try:
    _ = fm.base_path
    print("✅ FileManager já existe na memória")
except NameError:
    print("⚠️  FileManager não encontrado - recriando...")
    fm = FileManagerInterativo(pasta_base)
    print(f"✅ FileManager recriado: {fm.base_path.name}")

# ===================================================================
# 4. CLASSE GUI COM TIMER - PERSISTIDA EM ARQUIVO
# ===================================================================

arquivo_gui = pasta_base / '04_Logs' / '.bloco_3_gui_timer.py'

# Salvar código da classe para importação futura
codigo_gui = '''import tkinter as tk

class GUIComTimer:
    """Implementa timer de 10s com countdown visual"""

    @staticmethod
    def criar_janela_com_timer(titulo, largura, altura, tem_timer=True):
        """Cria janela base com timer"""
        root = tk.Tk()
        root.title(titulo)
        root.geometry(f"{largura}x{altura}")
        root.resizable(False, False)

        # Centralizar
        x = (root.winfo_screenwidth() // 2) - (largura // 2)
        y = (root.winfo_screenheight() // 2) - (altura // 2)
        root.geometry(f"+{x}+{y}")
        root.attributes('-topmost', True)
        root.after(100, lambda: root.attributes('-topmost', False))

        frame = tk.Frame(root, padx=20, pady=20, bg='white')
        frame.pack(fill=tk.BOTH, expand=True)

        resultado = {'valor': None, 'cancelado': False, 'timeout': False}
        contador = [10] if tem_timer else [0]

        return root, frame, resultado, contador

    @staticmethod
    def adicionar_timer(frame, root, resultado, contador):
        """Adiciona timer visual"""
        label_timer = tk.Label(
            frame,
            text=f"⏱️  {contador[0]}s",
            font=('Arial', 16, 'bold'),
            fg='#FF4444',
            bg='white'
        )
        label_timer.pack(pady=(5, 15))

        def countdown():
            if contador[0] > 0 and not resultado['cancelado']:
                contador[0] -= 1
                label_timer.config(text=f"⏱️  {contador[0]}s")
                root.after(1000, countdown)
            elif contador[0] == 0 and not resultado['cancelado']:
                resultado['timeout'] = True
                root.quit()
                root.destroy()

        return countdown

    @staticmethod
    def criar_botoes(frame, cmd_principal, cmd_secundario=None,
                     label_principal="Confirmar",
                     label_secundario="Usar Último"):
        """Cria botões padronizados"""
        tk.Frame(frame, height=2, bg='#CCCCCC').pack(
            fill=tk.X, pady=10
        )

        frame_btns = tk.Frame(frame, bg='white')
        frame_btns.pack(side=tk.BOTTOM, pady=10)

        tk.Button(
            frame_btns,
            text=label_principal,
            command=cmd_principal,
            width=18,
            height=2,
            bg='#4CAF50',
            fg='white',
            font=('Arial', 10, 'bold'),
            cursor='hand2'
        ).pack(side=tk.LEFT, padx=5)

        if cmd_secundario:
            tk.Button(
                frame_btns,
                text=label_secundario,
                command=cmd_secundario,
                width=18,
                height=2,
                bg='#2196F3',
                fg='white',
                font=('Arial', 10),
                cursor='hand2'
            ).pack(side=tk.LEFT, padx=5)
'''

# Salvar classe em arquivo
with open(arquivo_gui, 'w', encoding='utf-8') as f:
    f.write(codigo_gui)

# Tentar importar classe OU definir localmente
try:
    import importlib.util
    spec = importlib.util.spec_from_file_location("gui_timer", arquivo_gui)
    modulo_gui = importlib.util.module_from_spec(spec)
    spec.loader.exec_module(modulo_gui)
    GUIComTimer = modulo_gui.GUIComTimer
    print("✅ Classe GUIComTimer importada de arquivo")
except Exception as e:
    print(f"⚠️  Erro ao importar ({e}), definindo localmente...")
    exec(codigo_gui)
    print("✅ Classe GUIComTimer definida localmente")

# ===================================================================
# 5. DICIONÁRIO INTELIGENTE
# ===================================================================

class DicionarioInteligente:
    """Dicionário com detecção avançada"""

    def __init__(self, fm):
        self.fm = fm
        self.arquivo_dict = fm.pastas['logs'] / 'DICT_Dicionario_Persistente.json'
        self.dados = self._carregar_ou_criar()

    def _carregar_ou_criar(self):
        if self.arquivo_dict.exists():
            try:
                with open(self.arquivo_dict, 'r', encoding='utf-8') as f:
                    dados = json.load(f)

                if 'campos_conhecidos' not in dados:
                    dados = self._migrar_formato_antigo(dados)
                    self._salvar(dados)

                n_campos = len(dados['campos_conhecidos'])
                n_arquivos = len(dados.get('historico_arquivos', []))

                print(f"\n✅ DICIONÁRIO PERSISTENTE CARREGADO")
                print(f"   📚 {n_campos} campos conhecidos")
                print(f"   📁 {n_arquivos} arquivos processados")

                return dados

            except Exception as e:
                print(f"\n⚠️  Erro ao carregar: {e}")
                print("   Criando novo dicionário...")
                dados = self._criar_novo()
                self._salvar(dados)
                return dados
        else:
            print(f"\n📝 CRIANDO NOVO DICIONÁRIO...")
            dados = self._criar_novo()
            self._salvar(dados)
            print(f"✅ Dicionário criado: {len(dados['campos_conhecidos'])} campos")
            return dados

    def _migrar_formato_antigo(self, dados_antigos):
        novo = self._criar_novo()
        if 'arquivos_processados' in dados_antigos:
            novo['historico_arquivos'] = dados_antigos['arquivos_processados']
        return novo

    def _criar_novo(self):
        """Dicionário com 22 campos padrão"""
        return {
            'versao': '5.0',
            'criado_em': datetime.now().isoformat(),
            'ultima_atualizacao': datetime.now().isoformat(),
            'config_sistema': {
                'timeout_sessao_minutos': 60,
                'padroes_csv_detectados': []
            },
            'campos_conhecidos': {
                'Centro': {
                    'tipo_dado': 'Codigo_Centro',
                    'regex': r'^[5-9]\d{3}$',
                    'sinonimos': ['Centro', 'Código de Centro', 'Cod Centro',
                                  'Unidade Operacional:Centro'],
                    'exemplos': ['5025', '5065', '5174'],
                    'descricao': 'Código numérico de 4 dígitos',
                    'aprendido_de': 'PADRAO_INICIAL'
                },
                'Sigla': {
                    'tipo_dado': 'Sigla_Base',
                    'regex': r'^[A-Z]{4,10}$',
                    'sinonimos': ['Sigla', 'Sigla Base', 'Sigla Centro', 'Base'],
                    'exemplos': ['BABET', 'BAPLAN', 'AIBET'],
                    'descricao': 'Sigla alfabética (4-10 letras maiúsculas)',
                    'aprendido_de': 'PADRAO_INICIAL'
                },
                'Codigo_Produto': {
                    'tipo_dado': 'Codigo_Material',
                    'regex': r'^\d{1,2}\.\d{3}\.\d{3}$|^\d{7,8}$',
                    'sinonimos': ['Código Produto', 'Código Material',
                                  'Cod Produto', 'Cod Material'],
                    'exemplos': ['10.123.456', '1.234.567', '1234567'],
                    'descricao': 'Código numérico do material/produto',
                    'aprendido_de': 'PADRAO_INICIAL'
                },
                'Codigo_Grupo_Produto': {
                    'tipo_dado': 'Codigo_Grupo',
                    'regex': r'^\d{1,2}\.\d{3}\.\d{3}$|^\d{7,8}$|^[A-Z_]+$',
                    'sinonimos': ['Cód Grupo de produto', 'Código Grupo',
                                  'Grupo Produto', 'Produto:CodGrupoProduto'],
                    'exemplos': ['10.123.456', 'DIESEL_S10_SIMPLES',
                                 'GASOLINA_COMUM'],
                    'descricao': 'Código grupo (numérico OU texto_underscore)',
                    'aprendido_de': 'PADRAO_INICIAL'
                },
                'Desc_Grupo_Produto': {
                    'tipo_dado': 'Texto_Descricao',
                    'regex': r'^[A-Za-z0-9\s\-]+$',
                    'sinonimos': ['Desc. Grupo de Produto', 'Descrição Produto',
                                  'Nome Produto', 'Produto'],
                    'exemplos': ['DIESEL S10', 'GASOLINA COMUM',
                                 'ETANOL HIDRATADO'],
                    'descricao': 'Descrição textual do grupo de produto',
                    'aprendido_de': 'PADRAO_INICIAL'
                },
                'Nome_Pessoa': {
                    'tipo_dado': 'Texto_Nome_Pessoa',
                    'regex': r'^[A-ZÁÀÂÃÉÈÊÍÏÓÔÕÖÚÇÑ][a-záàâãéèêíïóôõöúçñ]+(\s[A-ZÁÀÂÃÉÈÊÍÏÓÔÕÖÚÇÑ][a-záàâãéèêíïóôõöúçñ]+)+$',
                    'sinonimos': ['Criado por', 'Nome', 'Responsável',
                                  'Solicitante'],
                    'exemplos': ['Kenedy Vinícius Rodrigues',
                                 'Joao Carlos Stival'],
                    'descricao': 'Nome completo de pessoa',
                    'aprendido_de': 'PADRAO_INICIAL'
                },
                'Email': {
                    'tipo_dado': 'Texto_Email',
                    'regex': r'^[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}$',
                    'sinonimos': ['Email', 'E-mail', 'Modificado por'],
                    'exemplos': ['usuario@vibraenergia.com.br'],
                    'descricao': 'Endereço de email',
                    'aprendido_de': 'PADRAO_INICIAL'
                },
                'Status_Workflow': {
                    'tipo_dado': 'Texto_Status',
                    'regex': r'^(Em aprovação|Aprovado|Rejeitado|Pendente|Concluído|Ítem Criado|Item Criado)$',
                    'sinonimos': ['Status', 'Status Aprovação', 'Situação'],
                    'exemplos': ['Em aprovação', 'Ítem Criado', 'Aprovado'],
                    'descricao': 'Status de workflow/aprovação',
                    'aprendido_de': 'PADRAO_INICIAL'
                },
                'Booleano_Texto': {
                    'tipo_dado': 'Texto_Booleano',
                    'regex': r'^(Sim|Não|sim|não|SIM|NÃO|Yes|No|TRUE|FALSE)$',
                    'sinonimos': ['Concluído?', 'Ativo?', 'Habilitado?'],
                    'exemplos': ['Sim', 'Não'],
                    'descricao': 'Valor booleano como texto',
                    'aprendido_de': 'PADRAO_INICIAL'
                },
                'Texto_Longo': {
                    'tipo_dado': 'Texto_Justificativa',
                    'regex': r'^.{50,}$',
                    'sinonimos': ['Justificativa', 'Observação', 'Comentário'],
                    'exemplos': ['Solicitamos a revisão do limite...'],
                    'descricao': 'Texto longo (mais de 50 caracteres)',
                    'aprendido_de': 'PADRAO_INICIAL'
                },
                'Path_URL': {
                    'tipo_dado': 'Texto_Caminho',
                    'regex': r'^(teams/|http|https|ftp|\\\\|/).*',
                    'sinonimos': ['Caminho', 'Path', 'URL', 'Link'],
                    'exemplos': ['teams/portaleso/Lists/...',
                                 'https://example.com'],
                    'descricao': 'Caminho de arquivo ou URL',
                    'aprendido_de': 'PADRAO_INICIAL'
                },
                'Sigla_Curta': {
                    'tipo_dado': 'Texto_Sigla_Curta',
                    'regex': r'^[A-Z]{2,4}$',
                    'sinonimos': ['CME', 'Gerência', 'UF', 'Tipo'],
                    'exemplos': ['OPC', 'OPN', 'Norte', 'Sul', 'CME'],
                    'descricao': 'Sigla curta (2-4 letras maiúsculas)',
                    'aprendido_de': 'PADRAO_INICIAL'
                },
                'Tipo_Item': {
                    'tipo_dado': 'Texto_Tipo_Item',
                    'regex': r'^(Item|Documento|Pasta|Arquivo)$',
                    'sinonimos': ['Tipo de Item', 'Tipo'],
                    'exemplos': ['Item'],
                    'descricao': 'Tipo de item em lista SharePoint',
                    'aprendido_de': 'PADRAO_INICIAL'
                },
                'Data_ISO': {
                    'tipo_dado': 'Data_YYYY-MM-DD',
                    'regex': r'^\d{4}-\d{2}-\d{2}$',
                    'sinonimos': ['Data', 'Período',
                                  'Período Início Validade Novo Limite'],
                    'exemplos': ['2025-08-01', '2024-12-31', '2025-01-07'],
                    'descricao': 'Data formato ISO (YYYY-MM-DD)',
                    'aprendido_de': 'PADRAO_INICIAL'
                },
                'Datetime_ISO': {
                    'tipo_dado': 'Datetime_YYYY-MM-DD_HH:MM:SS',
                    'regex': r'^\d{4}-\d{2}-\d{2}\s\d{2}:\d{2}:\d{2}$',
                    'sinonimos': ['Criado', 'Modificado', 'Data Hora',
                                  'Timestamp'],
                    'exemplos': ['2025-08-04 19:22:17',
                                 '2025-08-04 20:45:37'],
                    'descricao': 'Data e hora formato ISO',
                    'aprendido_de': 'PADRAO_INICIAL'
                },
                'Data_BR': {
                    'tipo_dado': 'Data_DD/MM/YYYY',
                    'regex': r'^\d{2}/\d{2}/\d{4}$',
                    'sinonimos': ['Data'],
                    'exemplos': ['15/01/2024', '31/12/2025'],
                    'descricao': 'Data formato brasileiro',
                    'aprendido_de': 'PADRAO_INICIAL'
                },
                'Percentual_Decimal': {
                    'tipo_dado': 'Numero_Percentual_Decimal',
                    'regex': r'^-?\d+(\.\d+)?$',
                    'sinonimos': ['Limite Inferior Atual',
                                  'Limite Superior Atual', 'AVG VI %',
                                  '% VI', 'AVG VI % 2024 SAP'],
                    'exemplos': ['-0.08', '0.08', '-0.14', '0.05', '-0.03'],
                    'descricao': 'Percentual em decimal (pode ser negativo)',
                    'aprendido_de': 'PADRAO_INICIAL'
                },
                'Percentual_Com_Simbolo': {
                    'tipo_dado': 'Numero_Percentual_Simbolo',
                    'regex': r'^-?\d+(\.\d+)?%$',
                    'sinonimos': ['Percentual', '% VI'],
                    'exemplos': ['10.5%', '-5.2%', '100%'],
                    'descricao': 'Percentual com símbolo %',
                    'aprendido_de': 'PADRAO_INICIAL'
                },
                'Numero_Inteiro': {
                    'tipo_dado': 'Numero_Inteiro',
                    'regex': r'^-?\d+$',
                    'sinonimos': ['Quantidade', 'Qtd', 'Total'],
                    'exemplos': ['123', '-456', '1000'],
                    'descricao': 'Número inteiro',
                    'aprendido_de': 'PADRAO_INICIAL'
                },
                'Monetario': {
                    'tipo_dado': 'Numero_Monetario',
                    'regex': r'^R\$\s?-?\d{1,3}(\.\d{3})*(,\d{2})?$|^-?\d+([.,]\d{2})?$',
                    'sinonimos': ['Valor', 'Custo', 'Preço', 'R$'],
                    'exemplos': ['R$ 1.234,56', '1234.56'],
                    'descricao': 'Valor monetário',
                    'aprendido_de': 'PADRAO_INICIAL'
                },
                'Unidade_Operacional_Nome': {
                    'tipo_dado': 'Texto_Unidade_Operacional',
                    'regex': r'^[A-Z]{4,10}\s+Base\s+de\s+.+$',
                    'sinonimos': ['Unidade Operacional', 'Nome Base'],
                    'exemplos': ['BABET Base de Betim',
                                 'BAPLAN Base de Paulínia'],
                    'descricao': 'Nome completo da unidade operacional',
                    'aprendido_de': 'PADRAO_INICIAL'
                },
                'Rotulo_Retencao': {
                    'tipo_dado': 'Texto_Rotulo_Vazio',
                    'regex': r'^(NaN|nan|None|null|)$',
                    'sinonimos': ['Rótulo de retenção Aplicado'],
                    'exemplos': ['NaN'],
                    'descricao': 'Campo de rótulo (geralmente vazio)',
                    'aprendido_de': 'PADRAO_INICIAL'
                }
            },
            'historico_arquivos': []
        }

    def detectar_campo(self, coluna_nome, valores_amostra):
        """Detecção AVANÇADA com múltiplas estratégias"""
        valores_str = [str(v).strip() for v in valores_amostra
                      if pd.notna(v) and str(v).strip() not in
                      ['', 'nan', 'None']]

        if not valores_str:
            return {
                'campo_detectado': 'VAZIO',
                'confianca': 0.0,
                'score_conteudo': 0.0,
                'score_nome': 0.0,
                'matches': 0,
                'total': 0,
                'ambiguidade': False,
                'candidatos': [],
                'metodo': 'VAZIO'
            }

        # Heurísticas específicas
        campo_heuristico = self._detectar_por_heuristica(
            coluna_nome, valores_str
        )

        if campo_heuristico and campo_heuristico.get('confianca', 0) >= 0.85:
            campo_heuristico.setdefault('score_nome', 0.0)
            campo_heuristico.setdefault('score_conteudo',
                                        campo_heuristico.get('confianca', 0.0))
            campo_heuristico.setdefault('matches', len(valores_str))
            campo_heuristico.setdefault('total', len(valores_str))
            campo_heuristico.setdefault('ambiguidade', False)
            campo_heuristico.setdefault('candidatos', [])
            return campo_heuristico

        # Match por regex
        resultados_regex = []
        for nome_campo, info in self.dados['campos_conhecidos'].items():
            matches = sum(1 for v in valores_str
                         if re.match(info['regex'], v))
            score_conteudo = matches / len(valores_str)

            score_nome = 0.0
            for sinonimo in info['sinonimos']:
                if sinonimo.lower() in coluna_nome.lower():
                    score_nome = 0.3
                    break

            score_final = score_conteudo + score_nome

            resultados_regex.append({
                'campo': nome_campo,
                'score': min(score_final, 1.0),
                'score_conteudo': score_conteudo,
                'score_nome': score_nome,
                'matches': matches,
                'total': len(valores_str)
            })

        resultados_regex = sorted(resultados_regex,
                                 key=lambda x: x['score'],
                                 reverse=True)
        melhor_regex = resultados_regex[0]
        segundo_regex = resultados_regex[1] if len(resultados_regex) > 1 else None

        ambiguidade = False
        candidatos = []
        if segundo_regex and abs(melhor_regex['score'] -
                                segundo_regex['score']) < 0.10:
            ambiguidade = True
            candidatos = [segundo_regex['campo']]

        resultado_final = {
            'campo_detectado': melhor_regex['campo'],
            'confianca': melhor_regex['score'],
            'score_conteudo': melhor_regex['score_conteudo'],
            'score_nome': melhor_regex['score_nome'],
            'matches': melhor_regex['matches'],
            'total': melhor_regex['total'],
            'ambiguidade': ambiguidade,
            'candidatos': candidatos,
            'metodo': 'REGEX'
        }

        if resultado_final['confianca'] < 0.50:
            resultado_final['campo_detectado'] = 'DESCONHECIDO'

        return resultado_final

    def _detectar_por_heuristica(self, nome_coluna, valores_str):
        """Detecção por heurísticas específicas"""
        if not valores_str:
            return None

        tamanho_medio = sum(len(v) for v in valores_str) / len(valores_str)
        valores_unicos = set(valores_str)

        if all(re.match(r'^\d{4}-\d{2}-\d{2}$', v) for v in valores_str[:5]):
            return {'campo_detectado': 'Data_ISO', 'confianca': 0.95,
                   'metodo': 'HEURISTICA_DATA_ISO'}

        if all(re.match(r'^\d{4}-\d{2}-\d{2}\s\d{2}:\d{2}:\d{2}', v)
               for v in valores_str[:5]):
            return {'campo_detectado': 'Datetime_ISO', 'confianca': 0.95,
                   'metodo': 'HEURISTICA_DATETIME'}

        if all('@' in v for v in valores_str):
            return {'campo_detectado': 'Email', 'confianca': 0.95,
                   'metodo': 'HEURISTICA_EMAIL'}

        if 'limite' in nome_coluna.lower() or '%' in nome_coluna or 'vi' in nome_coluna.lower():
            try:
                valores_float = [float(v) for v in valores_str
                                if v not in ['nan', '', 'None', 'NaN']]
                if valores_float and all(-1 <= v <= 1 for v in valores_float):
                    return {'campo_detectado': 'Percentual_Decimal',
                           'confianca': 0.90,
                           'metodo': 'HEURISTICA_PERCENTUAL'}
            except:
                pass

        if tamanho_medio > 50:
            return {'campo_detectado': 'Texto_Longo', 'confianca': 0.85,
                   'metodo': 'HEURISTICA_TEXTO_LONGO'}

        if any(v.startswith(('teams/', 'http', 'https', '//', '\\\\'))
               for v in valores_str):
            return {'campo_detectado': 'Path_URL', 'confianca': 0.90,
                   'metodo': 'HEURISTICA_PATH'}

        if valores_unicos <= {'Sim', 'Não', 'sim', 'não'}:
            return {'campo_detectado': 'Booleano_Texto', 'confianca': 0.95,
                   'metodo': 'HEURISTICA_BOOLEANO'}

        if 'por' in nome_coluna.lower() or 'nome' in nome_coluna.lower():
            if all(len(v.split()) >= 2 and '@' not in v
                   for v in valores_str[:5]):
                return {'campo_detectado': 'Nome_Pessoa', 'confianca': 0.85,
                       'metodo': 'HEURISTICA_NOME'}

        palavras_status = {'em aprovação', 'aprovado', 'rejeitado',
                          'ítem criado', 'item criado', 'pendente'}
        if any(v.lower() in palavras_status for v in valores_str):
            return {'campo_detectado': 'Status_Workflow', 'confianca': 0.90,
                   'metodo': 'HEURISTICA_STATUS'}

        if 'base de' in ' '.join(valores_str[:3]).lower():
            return {'campo_detectado': 'Unidade_Operacional_Nome',
                   'confianca': 0.90,
                   'metodo': 'HEURISTICA_UNIDADE_OP'}

        return None

    def atualizar_historico(self, info):
        self.dados['historico_arquivos'].append(info)
        self.dados['ultima_atualizacao'] = datetime.now().isoformat()
        self._salvar(self.dados)

    def _salvar(self, dados):
        with open(self.arquivo_dict, 'w', encoding='utf-8') as f:
            json.dump(dados, f, indent=2, ensure_ascii=False)

# ===================================================================
# 6. RECUPERAR/RECRIAR DICIONÁRIO (0% MEMÓRIA, 100% LOG)
# ===================================================================

print("\n" + "="*70)
print("INICIALIZANDO DICIONÁRIO")
print("="*70)

# Tentar recuperar dicionario da memória OU recriar
try:
    _ = dicionario.dados
    print("✅ DicionarioInteligente já existe na memória")
except NameError:
    print("⚠️  DicionarioInteligente não encontrado - recriando...")
    dicionario = DicionarioInteligente(fm)

# ===================================================================
# 7. SALVAR ESTADO COMPLETO PARA PRÓXIMO BLOCO
# ===================================================================

estado_bloco3 = {
    'bloco': 3,
    'versao': '5.0',
    'timestamp': datetime.now().isoformat(),
    'status': 'concluido',
    'mudancas_v5': [
        'Adiciona recuperação automática de fm',
        'Adiciona recuperação automática de dicionario',
        'Persiste classe GUIComTimer em arquivo .py',
        'Documenta como recriar todos os objetos'
    ],
    'dicionario': {
        'arquivo': str(dicionario.arquivo_dict),
        'campos_conhecidos': len(dicionario.dados['campos_conhecidos']),
        'arquivos_processados': len(dicionario.dados['historico_arquivos']),
        'config_sistema': dicionario.dados.get('config_sistema', {})
    },
    'componentes_carregados': ['DicionarioInteligente', 'GUIComTimer'],
    'persistencia': {
        'gui_timer_arquivo': str(arquivo_gui),
        'como_recriar_fm': {
            'codigo': 'fm = FileManagerInterativo(pasta_base)',
            'depende_de': ['pasta_base do LOG GLOBAL']
        },
        'como_recriar_dicionario': {
            'codigo': 'dicionario = DicionarioInteligente(fm)',
            'depende_de': ['fm', 'DICT_Dicionario_Persistente.json']
        },
        'como_importar_gui': {
            'codigo': [
                'import importlib.util',
                'spec = importlib.util.spec_from_file_location("gui_timer", arquivo_gui)',
                'modulo_gui = importlib.util.module_from_spec(spec)',
                'spec.loader.exec_module(modulo_gui)',
                'GUIComTimer = modulo_gui.GUIComTimer'
            ],
            'depende_de': ['.bloco_3_gui_timer.py']
        }
    }
}

arquivo_estado = fm.pastas['logs'] / '.bloco_3_state.json'
with open(arquivo_estado, 'w', encoding='utf-8') as f:
    json.dump(estado_bloco3, f, indent=2, ensure_ascii=False)

print("\n" + "="*70)
print("✅ BLOCO 3 CONCLUÍDO")
print("="*70)
print("\n📋 Componentes:")
print("   • DicionarioInteligente ........... ✅")
n_campos = len(dicionario.dados['campos_conhecidos'])
n_arquivos = len(dicionario.dados['historico_arquivos'])
print(f"     - Campos: {n_campos}")
print(f"     - Arquivos: {n_arquivos}")
timeout = dicionario.dados.get('config_sistema', {}).get('timeout_sessao_minutos', 60)
print(f"     - Timeout: {timeout}min")
print("   • GUIComTimer ..................... ✅")
print("\n💾 Persistência v5.0:")
print("   • .bloco_3_state.json ............. ✅")
print("   • DICT_Dicionario_Persistente.json  ✅")
print("   • .bloco_3_gui_timer.py ........... ✅ NOVO")
print("\n🔄 Recuperação de estado:")
print("   • fm recriável .................... ✅")
print("   • dicionario recriável ............ ✅")
print("   • GUIComTimer importável .......... ✅")
print("\n💡 Próximo: BLOCO 4 seleciona arquivo de dados")
print("="*70)


BLOCO 3: DICIONÁRIO INTELIGENTE + GUI COM TIMER

✅ CONFIGURAÇÃO CARREGADA DO LOG GLOBAL
   📁 Pasta base: PROCESSAR_ARQUIVOS_20251018_220345
   🕐 Timestamp: 20251018_220345

✅ BLOCO 2 VALIDADO
   Executado em: 2025-10-18T22:03:48.053686
   Classes: LocalizadorDicionario, FileManagerInterativo, SeletorArquivo, DetectorCabecalho

RECUPERAÇÃO DE ESTADO
✅ FileManager já existe na memória
✅ Classe GUIComTimer importada de arquivo

INICIALIZANDO DICIONÁRIO
✅ DicionarioInteligente já existe na memória

✅ BLOCO 3 CONCLUÍDO

📋 Componentes:
   • DicionarioInteligente ........... ✅
     - Campos: 22
     - Arquivos: 0
     - Timeout: 60min
   • GUIComTimer ..................... ✅

💾 Persistência v5.0:
   • .bloco_3_state.json ............. ✅
   • DICT_Dicionario_Persistente.json  ✅
   • .bloco_3_gui_timer.py ........... ✅ NOVO

🔄 Recuperação de estado:
   • fm recriável .................... ✅
   • dicionario recriável ............ ✅
   • GUIComTimer importável .......... ✅

💡 Próximo: BLOCO 4 sel

In [26]:
# ═══════════════════════════════════════════════════════════════════
# BLOCO 4 - SELEÇÃO DE ARQUIVO - SUPORTE MULTI-FORMATO (REVISADO v4.0)
# ═══════════════════════════════════════════════════════════════════
# PRINCÍPIO: 0% MEMÓRIA, 100% LOG
# ═══════════════════════════════════════════════════════════════════
# COMUNICAÇÃO VIA LOG:
# - LÊ: pasta_base (LOG global), timestamp, dicionário persistente
# - RECRIA: FileManager localmente
# - SALVA:
#   1. Estado JSON (.bloco_4_state.json)
#   2. Variáveis PKL (.bloco_4_vars.pkl)
#   3. Configuração arquivo (.ultimo_arquivo.json)
#   4. Log de transformações (.bloco_4_transformacoes.json)
# ═══════════════════════════════════════════════════════════════════

print("="*70)
print("BLOCO 4: SELEÇÃO DE ARQUIVO")
print("="*70)

# ═══════════════════════════════════════════════════════════════════
# 1. LER CONFIGURAÇÕES DO BLOCO ANTERIOR (VIA LOG)
# ═══════════════════════════════════════════════════════════════════

from pathlib import Path
import json
import pickle
from datetime import datetime

log_global = Path.home() / '.processador_dicionario_localizador.json'

if not log_global.exists():
    raise FileNotFoundError(
        "❌ LOG GLOBAL não encontrado!\n"
        "   Execute BLOCO 1 primeiro para criar a estrutura."
    )

with open(log_global, 'r', encoding='utf-8') as f:
    config = json.load(f)

pasta_base = Path(config['pasta_base_atual'])
timestamp_execucao = config['timestamp']

print(f"\n📂 Pasta base carregada: {pasta_base.name}")
print(f"⏰ Timestamp: {timestamp_execucao}")

# ═══════════════════════════════════════════════════════════════════
# 2. RECRIAR OBJETOS NECESSÁRIOS (NÃO ASSUMIR MEMÓRIA)
# ═══════════════════════════════════════════════════════════════════

# Recriar FileManager
fm = FileManagerInterativo(pasta_base)

# Carregar dicionário persistente
dict_file = fm.pastas['logs'] / 'DICT_Dicionario_Persistente.json'

if dict_file.exists():
    with open(dict_file, 'r', encoding='utf-8') as f:
        DICIONARIO_PERSISTENTE = json.load(f)
    print(f"📚 Dicionário carregado: {len(DICIONARIO_PERSISTENTE.get('campos_conhecidos', {}))} campos")
else:
    raise FileNotFoundError(
        "❌ Dicionário não encontrado!\n"
        "   Execute BLOCO 3 primeiro."
    )

# Inicializar log de transformações
LOG_TRANSFORMACOES = {
    'bloco': 4,
    'inicio': datetime.now().isoformat(),
    'operacoes': []
}

def registrar_operacao(tipo, descricao, dados=None):
    """Registra operação no log de transformações"""
    operacao = {
        'timestamp': datetime.now().isoformat(),
        'tipo': tipo,
        'descricao': descricao
    }
    if dados:
        operacao['dados'] = dados
    LOG_TRANSFORMACOES['operacoes'].append(operacao)

# ═══════════════════════════════════════════════════════════════════
# CONSTANTES E CONFIGURAÇÕES
# ═══════════════════════════════════════════════════════════════════

FILETYPES_SUPORTADOS = [
    ("Todos os suportados", "*.xlsx *.xls *.xlsm *.csv *.txt"),
    ("Excel", "*.xlsx *.xls *.xlsm"),
    ("CSV", "*.csv"),
    ("TXT (Tabelas)", "*.txt"),
    ("Todos", "*.*")
]

TIMEOUT_SESSAO_MINUTOS = DICIONARIO_PERSISTENTE.get(
    'config_sistema', {}
).get('timeout_sessao_minutos', 60)

registrar_operacao('config', 'Constantes carregadas', {
    'timeout_minutos': TIMEOUT_SESSAO_MINUTOS,
    'tipos_suportados': len(FILETYPES_SUPORTADOS)
})

# ═══════════════════════════════════════════════════════════════════
# FUNÇÕES AUXILIARES
# ═══════════════════════════════════════════════════════════════════

def detectar_config_csv(arquivo_path):
    """
    Detecta encoding e separador ideal para CSV.

    Returns:
        dict: {'encoding': str, 'sep': str, 'colunas': int}
        None: se falhar
    """
    import pandas as pd

    encodings = ['utf-8', 'latin-1', 'cp1252', 'iso-8859-1']
    separadores = [',', ';', '\t', '|']

    melhor_config = None
    max_colunas = 0

    for encoding in encodings:
        for sep in separadores:
            try:
                df_test = pd.read_csv(
                    arquivo_path,
                    nrows=5,
                    encoding=encoding,
                    sep=sep,
                    on_bad_lines='skip'
                )
                n_cols = len(df_test.columns)

                if n_cols > max_colunas and n_cols > 1:
                    max_colunas = n_cols
                    melhor_config = {
                        'encoding': encoding,
                        'sep': sep,
                        'colunas': n_cols
                    }
            except:
                continue

    return melhor_config


def validar_arquivo_selecionado(arquivo_path):
    """
    Validação básica do arquivo selecionado.

    Raises:
        FileNotFoundError: Se arquivo não existe
        ValueError: Se arquivo inválido

    Returns:
        dict: Informações de validação
    """
    import pandas as pd

    if not arquivo_path.exists():
        raise FileNotFoundError(f"Arquivo não encontrado: {arquivo_path}")

    tamanho = arquivo_path.stat().st_size
    if tamanho == 0:
        raise ValueError(f"Arquivo vazio: {arquivo_path.name}")

    extensao = arquivo_path.suffix.lower()
    extensoes_validas = ['.xlsx', '.xls', '.xlsm', '.csv', '.txt']

    if extensao not in extensoes_validas:
        print(f"   ⚠️ Extensão incomum: {extensao}")

    config_extra = {}

    try:
        if extensao in ['.xlsx', '.xls', '.xlsm']:
            pd.read_excel(arquivo_path, nrows=1)

        elif extensao == '.csv':
            config_csv = detectar_config_csv(arquivo_path)

            if config_csv:
                config_extra['csv'] = config_csv
                print(f"   📊 CSV: {config_csv['colunas']} colunas")
                print(f"   🔤 Encoding: {config_csv['encoding']}")
                print(f"   ➗ Separador: {repr(config_csv['sep'])}")
            else:
                print(f"   ⚠️ Config CSV não detectada automaticamente")

    except Exception as e:
        raise ValueError(f"Arquivo corrompido ou ilegível: {str(e)[:100]}")

    return {
        'valido': True,
        'tamanho_kb': tamanho / 1024,
        'extensao': extensao,
        **config_extra
    }


def carregar_preview_inteligente(arquivo_path, frame_preview):
    """
    Carrega preview do arquivo com tratamento de erro amigável.

    Args:
        arquivo_path: Path do arquivo
        frame_preview: Frame tkinter para mostrar preview
    """
    import tkinter as tk
    import pandas as pd

    extensao = arquivo_path.suffix.lower()

    try:
        if extensao in ['.xlsx', '.xls', '.xlsm']:
            df_quick = pd.read_excel(arquivo_path, nrows=3)

        elif extensao == '.csv':
            config_csv = detectar_config_csv(arquivo_path)
            if config_csv:
                df_quick = pd.read_csv(
                    arquivo_path,
                    nrows=3,
                    encoding=config_csv['encoding'],
                    sep=config_csv['sep']
                )
            else:
                df_quick = pd.read_csv(arquivo_path, nrows=3)

        elif extensao == '.txt':
            for sep in ['\t', ';', '|', ',']:
                try:
                    df_quick = pd.read_csv(arquivo_path, nrows=3, sep=sep)
                    if len(df_quick.columns) > 1:
                        break
                except:
                    continue

        preview_text = f"{len(df_quick)} linhas × {len(df_quick.columns)} colunas"
        tk.Label(
            frame_preview,
            text=preview_text,
            font=('Arial', 8),
            bg='#F5F5F5',
            fg='#666666',
            anchor='w'
        ).pack(fill=tk.X, padx=5, pady=(0, 3))

    except Exception as e:
        tk.Label(
            frame_preview,
            text="⚠️ Preview indisponível",
            font=('Arial', 8),
            bg='#F5F5F5',
            fg='#FF6B6B',
            anchor='w'
        ).pack(fill=tk.X, padx=5, pady=(0, 3))

        print(f"   ⚠️ Preview falhou: {str(e)[:50]}")

# ═══════════════════════════════════════════════════════════════════
# 3. PROCESSAR DADOS DO BLOCO (LÓGICA PRINCIPAL)
# ═══════════════════════════════════════════════════════════════════

import tkinter as tk
from tkinter import filedialog

# Carregar última seleção
config_file = fm.pastas['logs'] / '.ultimo_arquivo.json'
ultimo_arquivo = None
sessao_atual = False

if config_file.exists():
    try:
        with open(config_file, 'r', encoding='utf-8') as f:
            config = json.load(f)

        try:
            ts_config = datetime.fromisoformat(config.get('timestamp', ''))
            ts_agora = datetime.now()
            diff_minutos = (ts_agora - ts_config).total_seconds() / 60

            if diff_minutos < TIMEOUT_SESSAO_MINUTOS:
                caminho_salvo = config.get('caminho')
                if caminho_salvo and Path(caminho_salvo).exists():
                    ultimo_arquivo = Path(caminho_salvo)
                    sessao_atual = True

                    registrar_operacao('sessao', 'Sessão ativa detectada', {
                        'arquivo_anterior': caminho_salvo,
                        'minutos_desde_ultimo': round(diff_minutos, 2)
                    })
        except:
            pass
    except:
        pass

print(f"\n💡 Última seleção: {ultimo_arquivo.name if ultimo_arquivo else 'Nenhuma'}")
print(f"   Mesma sessão: {'Sim' if sessao_atual else 'Não'}")

# CASO 1: Tem arquivo da sessão atual → GUI com timer
if ultimo_arquivo and sessao_atual:
    def selecionar_arquivo_com_timer(ultimo_path):
        """GUI com timer para confirmar ou trocar arquivo"""
        root, frame, resultado, contador = GUIComTimer.criar_janela_com_timer(
            "DETECTOR - Seleção de Arquivo",
            650, 520,
            tem_timer=True
        )

        tk.Label(
            frame,
            text="📂 Seleção de Arquivo",
            font=('Arial', 14, 'bold'),
            bg='white'
        ).pack(pady=(0, 15))

        tk.Label(
            frame,
            text="💡 Último arquivo selecionado nesta sessão:",
            font=('Arial', 10),
            bg='white'
        ).pack(pady=(0, 5))

        extensao = ultimo_path.suffix.lower()
        if extensao in ['.xlsx', '.xls', '.xlsm']:
            tipo_arquivo = "Excel"
            icone = "📊"
        elif extensao == '.csv':
            tipo_arquivo = "CSV"
            icone = "📄"
        elif extensao == '.txt':
            tipo_arquivo = "TXT"
            icone = "📝"
        else:
            tipo_arquivo = "Desconhecido"
            icone = "❓"

        frame_info = tk.Frame(frame, bg='#E3F2FD', relief=tk.SUNKEN, borderwidth=2)
        frame_info.pack(fill=tk.X, pady=(0, 10), padx=10)

        tk.Label(
            frame_info,
            text=f"{icone} {ultimo_path.name}",
            font=('Arial', 10, 'bold'),
            bg='#E3F2FD',
            fg='#1565C0',
            anchor='w'
        ).pack(fill=tk.X, padx=10, pady=(5, 2))

        tamanho_kb = ultimo_path.stat().st_size / 1024
        tk.Label(
            frame_info,
            text=f"📦 Tipo: {tipo_arquivo} | 📏 Tamanho: {tamanho_kb:.1f} KB",
            font=('Arial', 9),
            bg='#E3F2FD',
            fg='#1565C0',
            anchor='w'
        ).pack(fill=tk.X, padx=10, pady=(0, 2))

        tk.Label(
            frame_info,
            text=f"📂 Local: {ultimo_path.parent}",
            font=('Arial', 9),
            bg='#E3F2FD',
            fg='#1565C0',
            anchor='w',
            wraplength=600
        ).pack(fill=tk.X, padx=10, pady=(0, 5))

        countdown = GUIComTimer.adicionar_timer(frame, root, resultado, contador)

        tk.Label(
            frame,
            text="Deseja usar este arquivo ou escolher outro?",
            font=('Arial', 10),
            bg='white'
        ).pack(pady=(10, 10))

        frame_preview = tk.Frame(frame, bg='#F5F5F5', relief=tk.SUNKEN, borderwidth=1)
        frame_preview.pack(fill=tk.X, padx=10, pady=(0, 10))

        tk.Label(
            frame_preview,
            text="📊 Preview (3 primeiras linhas):",
            font=('Arial', 9, 'bold'),
            bg='#F5F5F5',
            anchor='w'
        ).pack(fill=tk.X, padx=5, pady=(3, 2))

        carregar_preview_inteligente(ultimo_path, frame_preview)

        def usar_ultimo():
            resultado['cancelado'] = True
            resultado['valor'] = ultimo_path
            resultado['origem'] = 'sessao_anterior'
            root.quit()
            root.destroy()

        def escolher_novo():
            resultado['cancelado'] = True
            root.withdraw()

            arquivo = filedialog.askopenfilename(
                title="Selecione o arquivo de dados",
                initialdir=ultimo_path.parent,
                filetypes=FILETYPES_SUPORTADOS
            )

            resultado['valor'] = Path(arquivo) if arquivo else ultimo_path
            resultado['origem'] = 'usuario_novo' if arquivo else 'sessao_anterior'
            root.quit()
            root.destroy()

        tk.Frame(frame, height=2, bg='#CCCCCC').pack(fill=tk.X, pady=10)

        frame_btns = tk.Frame(frame, bg='white')
        frame_btns.pack(side=tk.BOTTOM, pady=10)

        tk.Button(
            frame_btns,
            text="Escolher Novo Arquivo",
            command=escolher_novo,
            width=22,
            height=2,
            font=('Arial', 10, 'bold'),
            bg='#4CAF50',
            fg='white',
            cursor='hand2'
        ).pack(side=tk.LEFT, padx=5)

        nome_curto = ultimo_path.name[:15] + '...' if len(ultimo_path.name) > 15 else ultimo_path.name
        tk.Button(
            frame_btns,
            text=f"Usar '{nome_curto}' (10s)",
            command=usar_ultimo,
            width=30,
            height=2,
            font=('Arial', 10),
            bg='#2196F3',
            fg='white',
            cursor='hand2'
        ).pack(side=tk.LEFT, padx=5)

        root.after(1000, countdown)
        root.mainloop()

        if resultado.get('timeout'):
            print(f"   ⏱️ Timeout (10s) - usando último arquivo")
            resultado['origem'] = 'timeout'
            return ultimo_path, 'timeout'

        return resultado['valor'], resultado.get('origem', 'usuario')

    print("\nAbrindo janela...")
    arquivo_selecionado, origem_selecao = selecionar_arquivo_com_timer(ultimo_arquivo)

    registrar_operacao('selecao', 'Arquivo selecionado (sessão ativa)', {
        'arquivo': str(arquivo_selecionado),
        'origem': origem_selecao
    })

# CASO 2: Não tem arquivo da sessão → GUI direta sem timer
else:
    def selecionar_arquivo_direto():
        """GUI direta para primeira seleção"""
        root = tk.Tk()
        root.withdraw()
        root.attributes('-topmost', True)

        arquivo = filedialog.askopenfilename(
            title="Selecione o arquivo de dados",
            initialdir=ultimo_arquivo.parent if ultimo_arquivo else fm.pastas['entrada'],
            filetypes=FILETYPES_SUPORTADOS
        )

        root.destroy()

        if not arquivo:
            raise ValueError("❌ Nenhum arquivo selecionado")

        return Path(arquivo)

    print("\nAbrindo janela de seleção...")
    print("(A janela pode estar atrás do navegador)")
    arquivo_selecionado = selecionar_arquivo_direto()
    origem_selecao = 'primeira_selecao'

    registrar_operacao('selecao', 'Arquivo selecionado (primeira vez)', {
        'arquivo': str(arquivo_selecionado)
    })

# ═══════════════════════════════════════════════════════════════════
# VALIDAÇÃO E DETECÇÃO DE TIPO
# ═══════════════════════════════════════════════════════════════════

print("\n🔍 Validando arquivo...")

try:
    info_validacao = validar_arquivo_selecionado(arquivo_selecionado)
    print("   ✅ Arquivo validado com sucesso")

    registrar_operacao('validacao', 'Arquivo validado', {
        'tamanho_kb': info_validacao['tamanho_kb'],
        'extensao': info_validacao['extensao']
    })
except Exception as e:
    print(f"   ❌ ERRO: {e}")
    registrar_operacao('erro', 'Falha na validação', {'erro': str(e)})
    raise

extensao = arquivo_selecionado.suffix.lower()
if extensao in ['.xlsx', '.xls', '.xlsm']:
    tipo_arquivo = "Excel"
elif extensao == '.csv':
    tipo_arquivo = "CSV"
elif extensao == '.txt':
    tipo_arquivo = "TXT"
else:
    tipo_arquivo = "Desconhecido"

# ═══════════════════════════════════════════════════════════════════
# 4. SALVAR ESTADO PARA PRÓXIMO BLOCO (PRINCÍPIO 100% LOG)
# ═══════════════════════════════════════════════════════════════════

print("\n💾 Salvando estado completo (0% memória, 100% LOG)...")

# 4.1 - Configuração do arquivo selecionado (.ultimo_arquivo.json)
config_salvar = {
    'nome': arquivo_selecionado.name,
    'caminho': str(arquivo_selecionado),
    'tamanho_kb': info_validacao['tamanho_kb'],
    'tipo': tipo_arquivo,
    'extensao': extensao,
    'timestamp': datetime.now().isoformat(),
    'origem_selecao': origem_selecao
}

if 'csv' in info_validacao:
    config_salvar['config_csv'] = info_validacao['csv']

with open(config_file, 'w', encoding='utf-8') as f:
    json.dump(config_salvar, f, indent=2, ensure_ascii=False)

print(f"   ✅ Configuração salva: .ultimo_arquivo.json")

# 4.2 - Estado do BLOCO 4 (.bloco_4_state.json)
estado_bloco = {
    'bloco': 4,
    'status': 'concluido',
    'timestamp_inicio': LOG_TRANSFORMACOES['inicio'],
    'timestamp_fim': datetime.now().isoformat(),
    'arquivo_selecionado': {
        'nome': arquivo_selecionado.name,
        'caminho': str(arquivo_selecionado),
        'tipo': tipo_arquivo,
        'extensao': extensao,
        'tamanho_kb': info_validacao['tamanho_kb'],
        'origem_selecao': origem_selecao
    },
    'validacao': info_validacao,
    'sessao': {
        'tinha_arquivo_anterior': ultimo_arquivo is not None,
        'sessao_ativa': sessao_atual,
        'timeout_minutos': TIMEOUT_SESSAO_MINUTOS
    }
}

estado_file = fm.pastas['logs'] / '.bloco_4_state.json'
with open(estado_file, 'w', encoding='utf-8') as f:
    json.dump(estado_bloco, f, indent=2, ensure_ascii=False)

print(f"   ✅ Estado salvo: .bloco_4_state.json")

# 4.3 - Variáveis Python (.bloco_4_vars.pkl)
variaveis_bloco = {
    'arquivo_selecionado': arquivo_selecionado,
    'info_validacao': info_validacao,
    'tipo_arquivo': tipo_arquivo,
    'extensao': extensao,
    'origem_selecao': origem_selecao,
    'config_salvar': config_salvar,
    'ultimo_arquivo': ultimo_arquivo,
    'sessao_atual': sessao_atual,
    'TIMEOUT_SESSAO_MINUTOS': TIMEOUT_SESSAO_MINUTOS,
    'FILETYPES_SUPORTADOS': FILETYPES_SUPORTADOS,
    'timestamp_bloco': datetime.now().isoformat()
}

vars_file = fm.pastas['logs'] / '.bloco_4_vars.pkl'
with open(vars_file, 'wb') as f:
    pickle.dump(variaveis_bloco, f)

print(f"   ✅ Variáveis salvas: .bloco_4_vars.pkl ({len(variaveis_bloco)} variáveis)")

# 4.4 - Log de transformações (.bloco_4_transformacoes.json)
LOG_TRANSFORMACOES['fim'] = datetime.now().isoformat()
LOG_TRANSFORMACOES['total_operacoes'] = len(LOG_TRANSFORMACOES['operacoes'])
LOG_TRANSFORMACOES['resumo'] = {
    'arquivo_final': arquivo_selecionado.name,
    'tipo': tipo_arquivo,
    'origem': origem_selecao,
    'validado': True
}

transform_file = fm.pastas['logs'] / '.bloco_4_transformacoes.json'
with open(transform_file, 'w', encoding='utf-8') as f:
    json.dump(LOG_TRANSFORMACOES, f, indent=2, ensure_ascii=False)

print(f"   ✅ Transformações salvas: .bloco_4_transformacoes.json ({LOG_TRANSFORMACOES['total_operacoes']} operações)")

# 4.5 - Metadados para recuperação rápida
metadados = {
    'bloco': 4,
    'arquivos_salvos': {
        'estado_json': str(estado_file.name),
        'variaveis_pkl': str(vars_file.name),
        'transformacoes_json': str(transform_file.name),
        'config_arquivo': str(config_file.name)
    },
    'como_recuperar': {
        'estado': 'json.load(open(estado_file))',
        'variaveis': 'pickle.load(open(vars_file, "rb"))',
        'transformacoes': 'json.load(open(transform_file))',
        'arquivo_selecionado': 'Path(estado["arquivo_selecionado"]["caminho"])'
    },
    'timestamp': datetime.now().isoformat()
}

meta_file = fm.pastas['logs'] / '.bloco_4_metadados.json'
with open(meta_file, 'w', encoding='utf-8') as f:
    json.dump(metadados, f, indent=2, ensure_ascii=False)

print(f"   ✅ Metadados salvos: .bloco_4_metadados.json")

# ═══════════════════════════════════════════════════════════════════
# 5. VERIFICAÇÃO DE INTEGRIDADE
# ═══════════════════════════════════════════════════════════════════

print("\n🔍 Verificando integridade dos arquivos salvos...")

arquivos_esperados = [
    estado_file,
    vars_file,
    transform_file,
    config_file,
    meta_file
]

todos_ok = True
for arq in arquivos_esperados:
    if arq.exists():
        tamanho = arq.stat().st_size
        print(f"   ✅ {arq.name} ({tamanho} bytes)")
    else:
        print(f"   ❌ {arq.name} NÃO ENCONTRADO")
        todos_ok = False

if not todos_ok:
    raise RuntimeError("❌ Falha na gravação de arquivos de estado!")

# ═══════════════════════════════════════════════════════════════════
# 6. CONFIRMAÇÃO FINAL
# ═══════════════════════════════════════════════════════════════════

print(f"\n{'='*70}")
print("✅ ARQUIVO SELECIONADO E VALIDADO")
print(f"{'='*70}")
print(f"📄 Nome: {arquivo_selecionado.name}")
print(f"📦 Tipo: {tipo_arquivo}")
print(f"📏 Tamanho: {info_validacao['tamanho_kb']:.1f} KB")
print(f"📂 Pasta: {arquivo_selecionado.parent.name}")
print(f"🔄 Origem: {origem_selecao}")

if 'csv' in info_validacao:
    csv_info = info_validacao['csv']
    print(f"📊 Colunas detectadas: {csv_info['colunas']}")
    print(f"🔤 Encoding: {csv_info['encoding']}")
    print(f"➗ Separador: {repr(csv_info['sep'])}")

print(f"\n{'='*70}")
print("💾 ESTADO PERSISTENTE SALVO (0% MEMÓRIA, 100% LOG)")
print(f"{'='*70}")
print(f"📋 Estado JSON: {estado_file.name}")
print(f"🔧 Variáveis PKL: {vars_file.name}")
print(f"📝 Transformações: {transform_file.name}")
print(f"⚙️ Config arquivo: {config_file.name}")
print(f"📊 Metadados: {meta_file.name}")

print(f"\n{'='*70}")
print("✅ BLOCO 4 CONCLUÍDO")
print(f"{'='*70}")
print(f"\n💾 Estado completo em: {fm.pastas['logs']}")
print(f"📋 Próximo: BLOCO 5 carregará todos os dados via arquivos .pkl/.json")
print(f"\n💡 Exemplo de recuperação:")
print(f"   import pickle, json")
print(f"   vars = pickle.load(open('{vars_file}', 'rb'))")
print(f"   arquivo = vars['arquivo_selecionado']")

BLOCO 4: SELEÇÃO DE ARQUIVO

📂 Pasta base carregada: PROCESSAR_ARQUIVOS_20251018_220345
⏰ Timestamp: 20251018_220345


FileNotFoundError: ❌ Dicionário não encontrado!
   Execute BLOCO 3 primeiro.

In [4]:
# ═══════════════════════════════════════════════════════════════
# BLOCO 5: CARREGAMENTO INTELIGENTE - EXCEL E CSV
# ═══════════════════════════════════════════════════════════════

import json
from datetime import datetime

print("\n" + "="*70)
print("📥 CARREGAMENTO DO ARQUIVO")
print("="*70)

# Detectar tipo de arquivo pela extensão
extensao = arquivo_selecionado.suffix.lower()
print(f"\n🔍 Extensão detectada: {extensao}")

# ═══════════════════════════════════════════════════════════════
# CASO 1: ARQUIVOS EXCEL (.xls, .xlsx, .xlsm)
# ═══════════════════════════════════════════════════════════════

if extensao in ['.xls', '.xlsx', '.xlsm']:
    print(f"📊 Tipo: EXCEL")

    try:
        # Tentar xlrd primeiro (para .xls antigos)
        workbook = xlrd.open_workbook(str(arquivo_selecionado))
        sheets = workbook.sheet_names()
        metodo_carga = 'xlrd'
        print(f"   ✅ Método: xlrd (XLS)")

    except:
        # Se falhar, usar pandas (para .xlsx/.xlsm)
        try:
            workbook = pd.ExcelFile(str(arquivo_selecionado))
            sheets = workbook.sheet_names
            metodo_carga = 'pandas'
            print(f"   ✅ Método: pandas (XLSX/XLSM)")
        except Exception as e:
            print(f"   ❌ ERRO ao abrir Excel: {e}")
            raise

    print(f"\n📋 Sheets encontradas: {len(sheets)}")
    for i, sheet in enumerate(sheets, 1):
        print(f"   {i}. {sheet}")

    tipo_arquivo = 'EXCEL'
    separador_detectado = None
    skiprows_csv = 0

    # Extrair informações adicionais das sheets
    sheets_info = []
    for sheet_name in sheets:
        try:
            if metodo_carga == 'xlrd':
                sheet_obj = workbook.sheet_by_name(sheet_name)
                nrows = sheet_obj.nrows
                ncols = sheet_obj.ncols
            else:  # pandas
                df_temp = pd.read_excel(workbook, sheet_name=sheet_name, nrows=1)
                # Contar linhas do arquivo
                df_full = pd.read_excel(str(arquivo_selecionado), sheet_name=sheet_name)
                nrows = len(df_full) + 1  # +1 para header
                ncols = len(df_temp.columns)

            sheets_info.append({
                'nome': sheet_name,
                'linhas': nrows,
                'colunas': ncols
            })
        except Exception as e:
            sheets_info.append({
                'nome': sheet_name,
                'linhas': 'N/A',
                'colunas': 'N/A',
                'erro': str(e)
            })

    # IMPORTANTE: Fechar o workbook para liberar recursos
    if metodo_carga == 'pandas' and hasattr(workbook, 'close'):
        workbook.close()

# ═══════════════════════════════════════════════════════════════
# CASO 2: ARQUIVOS CSV (.csv)
# ═══════════════════════════════════════════════════════════════

elif extensao == '.csv':
    print(f"📄 Tipo: CSV")

    try:
        # Ler primeira linha para detectar separador
        with open(arquivo_selecionado, 'r', encoding='cp1252') as f:
            primeira_linha = f.readline().strip()

        print(f"\n🔍 Primeira linha: {primeira_linha[:100]}")

        # Detectar separador
        separador_detectado = None

        # Caso 1: Linha explícita com "sep="
        if primeira_linha.lower().startswith('sep='):
            separador_detectado = primeira_linha.split('=')[1]
            skiprows_csv = 1
            print(f"   ✅ Separador explícito: '{separador_detectado}'")

        # Caso 2: Tentar detectar automaticamente
        else:
            for sep in ['^', ';', ',', '\t', '|']:
                df_test = pd.read_csv(
                    arquivo_selecionado,
                    nrows=2,
                    sep=sep,
                    encoding='cp1252',
                    on_bad_lines='skip'
                )
                if len(df_test.columns) > 1:
                    separador_detectado = sep
                    skiprows_csv = 0
                    print(f"   ✅ Separador auto: '{separador_detectado}'")
                    break

        if not separador_detectado:
            raise ValueError("❌ Separador CSV não detectado")

        # Carregar preview
        df_preview = pd.read_csv(
            arquivo_selecionado,
            sep=separador_detectado,
            encoding='cp1252',
            skiprows=skiprows_csv,
            nrows=5
        )

        print(f"\n📊 Estrutura do CSV:")
        print(f"   Colunas: {len(df_preview.columns)}")
        print(f"   Encoding: cp1252")
        print(f"   Separador: '{separador_detectado}'")

        # Contar linhas totais
        with open(arquivo_selecionado, 'r', encoding='cp1252') as f:
            total_linhas = sum(1 for _ in f) - skiprows_csv

        # Simular sheets (CSV = 1 sheet virtual)
        sheets = ['Dados CSV']
        metodo_carga = 'csv'
        workbook = None
        tipo_arquivo = 'CSV'

        sheets_info = [{
            'nome': 'Dados CSV',
            'linhas': total_linhas,
            'colunas': len(df_preview.columns)
        }]

        print(f"\n📋 Sheet virtual: 'Dados CSV'")

    except Exception as e:
        print(f"   ❌ ERRO ao processar CSV: {e}")
        raise

# ═══════════════════════════════════════════════════════════════
# CASO 3: ARQUIVOS TXT (.txt)
# ═══════════════════════════════════════════════════════════════

elif extensao == '.txt':
    print(f"📝 Tipo: TXT")
    print(f"   ℹ️  Processamento similar a CSV")

    try:
        # Ler primeira linha para detectar separador
        with open(arquivo_selecionado, 'r', encoding='cp1252') as f:
            primeira_linha = f.readline().strip()

        print(f"\n🔍 Primeira linha: {primeira_linha[:100]}")

        # Detectar separador
        separador_detectado = None

        # Caso 1: Linha explícita com "sep="
        if primeira_linha.lower().startswith('sep='):
            separador_detectado = primeira_linha.split('=')[1]
            skiprows_csv = 1
            print(f"   ✅ Separador explícito: '{separador_detectado}'")

        # Caso 2: Tentar detectar automaticamente
        else:
            for sep in ['^', ';', ',', '\t', '|']:
                df_test = pd.read_csv(
                    arquivo_selecionado,
                    nrows=2,
                    sep=sep,
                    encoding='cp1252',
                    on_bad_lines='skip'
                )
                if len(df_test.columns) > 1:
                    separador_detectado = sep
                    skiprows_csv = 0
                    print(f"   ✅ Separador auto: '{separador_detectado}'")
                    break

        if not separador_detectado:
            raise ValueError("❌ Separador TXT não detectado")

        # Carregar preview
        df_preview = pd.read_csv(
            arquivo_selecionado,
            sep=separador_detectado,
            encoding='cp1252',
            skiprows=skiprows_csv,
            nrows=5
        )

        print(f"\n📊 Estrutura do TXT:")
        print(f"   Colunas: {len(df_preview.columns)}")
        print(f"   Encoding: cp1252")
        print(f"   Separador: '{separador_detectado}'")

        # Contar linhas totais
        with open(arquivo_selecionado, 'r', encoding='cp1252') as f:
            total_linhas = sum(1 for _ in f) - skiprows_csv

        # Simular sheets (TXT = 1 sheet virtual)
        sheets = ['Dados TXT']
        metodo_carga = 'csv'
        workbook = None
        tipo_arquivo = 'TXT'

        sheets_info = [{
            'nome': 'Dados TXT',
            'linhas': total_linhas,
            'colunas': len(df_preview.columns)
        }]

        print(f"\n📋 Sheet virtual: 'Dados TXT'")

    except Exception as e:
        print(f"   ❌ ERRO ao processar TXT: {e}")
        raise

else:
    raise ValueError(f"❌ Formato não suportado: {extensao}")

# ═══════════════════════════════════════════════════════════════
# SALVAMENTO DE ESTADO NO LOG (APENAS METADADOS)
# ═══════════════════════════════════════════════════════════════

estado_bloco5 = {
    'bloco': 5,
    'status': 'concluido',
    'timestamp': datetime.now().isoformat(),
    'tipo_arquivo': tipo_arquivo,
    'metodo_carga': metodo_carga,
    'extensao': extensao,
    'sheets': sheets,
    'sheets_info': sheets_info,
    'workbook_path': str(arquivo_selecionado),
    'workbook_path_absolute': str(arquivo_selecionado.resolve()),
    'separador_detectado': separador_detectado,
    'skiprows_csv': skiprows_csv,
    'encoding': 'cp1252' if tipo_arquivo in ['CSV', 'TXT'] else None,
    'total_sheets': len(sheets)
}

# Salvar estado em JSON
arquivo_estado = fm.pastas['logs'] / '.bloco_5_state.json'
with open(arquivo_estado, 'w', encoding='utf-8') as f:
    json.dump(estado_bloco5, f, indent=2, ensure_ascii=False)

print(f"\n💾 Estado salvo: {arquivo_estado.name}")

# ═══════════════════════════════════════════════════════════════
# RESUMO DO CARREGAMENTO
# ═══════════════════════════════════════════════════════════════

print("\n" + "─"*70)
print("✅ CARREGAMENTO CONCLUÍDO")
print("─"*70)
print(f"   Tipo: {tipo_arquivo}")
print(f"   Método: {metodo_carga}")
print(f"   Sheets/Tabelas: {len(sheets)}")
if sheets_info:
    print(f"\n📊 Detalhes das Sheets:")
    for info in sheets_info:
        print(f"   • {info['nome']}: {info['linhas']} linhas x {info['colunas']} colunas")

print("\n" + "="*70)
print("✅ BLOCO 5 CONCLUÍDO")
print("="*70)
print(f"💾 Estado salvo: .bloco_5_state.json")
print(f"📋 Próximo: BLOCO 6 selecionará a sheet e fará preview")
print("\n⚠️  NOTA: Workbook NÃO persistido (será reaberto quando necessário)")


📥 CARREGAMENTO DO ARQUIVO

🔍 Extensão detectada: .xlsx
📊 Tipo: EXCEL
   ✅ Método: pandas (XLSX/XLSM)

📋 Sheets encontradas: 2
   1. Grupos de Produto e Desc
   2. Material x Grupos de Produtos

💾 Estado salvo: .bloco_5_state.json

──────────────────────────────────────────────────────────────────────
✅ CARREGAMENTO CONCLUÍDO
──────────────────────────────────────────────────────────────────────
   Tipo: EXCEL
   Método: pandas
   Sheets/Tabelas: 2

📊 Detalhes das Sheets:
   • Grupos de Produto e Desc: 31 linhas x 2 colunas
   • Material x Grupos de Produtos: 213 linhas x 2 colunas

✅ BLOCO 5 CONCLUÍDO
💾 Estado salvo: .bloco_5_state.json
📋 Próximo: BLOCO 6 selecionará a sheet e fará preview

⚠️  NOTA: Workbook NÃO persistido (será reaberto quando necessário)


In [7]:
# ═══════════════════════════════════════════════════════════════════
# BLOCO 6 - SELEÇÃO DE SHEET (COM SUPORTE CSV + PERSISTÊNCIA)
# ═══════════════════════════════════════════════════════════════════

print("\n" + "="*70)
print("📋 SELEÇÃO DE SHEET/TABELA")
print("="*70)

# ───────────────────────────────────────────────────────────────────
# INICIALIZAÇÃO DE VARIÁVEIS DE ESTADO
# ───────────────────────────────────────────────────────────────────

state_file = fm.pastas['logs'] / '.bloco_6_state.json'
config_file = fm.pastas['logs'] / '.ultima_sheet.json'
log_transformacoes = fm.pastas['logs'] / 'bloco_6_transformacoes.log'

# Variáveis que serão persistidas
estado_bloco_6 = {
    'sheet_nome': None,
    'tipo_arquivo': tipo_arquivo,
    'sheets_disponiveis': sheets,
    'arquivo_nome': arquivo_selecionado.name,
    'arquivo_path': str(arquivo_selecionado),
    'total_sheets': len(sheets),
    'metodo_selecao': None,  # 'auto_csv', 'auto_unica', 'gui_timer', 'gui_manual'
    'timestamp_inicio': datetime.now().isoformat(),
    'timestamp_fim': None,
    'arquivo_mudou': True,
    'ultima_sheet_usada': None
}

# ───────────────────────────────────────────────────────────────────
# FUNÇÃO DE LOGGING
# ───────────────────────────────────────────────────────────────────

def log_transformacao(acao, detalhes):
    """Registra todas as ações e decisões do bloco"""
    timestamp = datetime.now().strftime('%Y-%m-%d %H:%M:%S.%f')[:-3]
    log_entry = f"[{timestamp}] {acao}: {detalhes}\n"

    with open(log_transformacoes, 'a', encoding='utf-8') as f:
        f.write(log_entry)

    print(f"   📝 LOG: {acao}")

# ───────────────────────────────────────────────────────────────────
# CARREGAR ÚLTIMA SELEÇÃO E VERIFICAR MUDANÇAS
# ───────────────────────────────────────────────────────────────────

log_transformacao("INICIO_BLOCO_6", f"Arquivo: {arquivo_selecionado.name}, Tipo: {tipo_arquivo}, Sheets: {len(sheets)}")

ultima_sheet = None
arquivo_mudou = True

if config_file.exists():
    try:
        with open(config_file, 'r', encoding='utf-8') as f:
            config = json.load(f)

            log_transformacao("CONFIG_CARREGADA", f"Última configuração encontrada: {config.get('arquivo')}")

            # Verificar se é o mesmo arquivo E timestamp recente (última hora)
            if config.get('arquivo') == arquivo_selecionado.name:
                ultima_sheet = config.get('sheet')
                estado_bloco_6['ultima_sheet_usada'] = ultima_sheet

                # Verificar se timestamp é recente
                try:
                    ts_salvo = datetime.fromisoformat(config.get('timestamp', ''))
                    ts_agora = datetime.now()
                    diff_minutos = (ts_agora - ts_salvo).total_seconds() / 60

                    if diff_minutos < 60:  # Última hora
                        arquivo_mudou = False
                        log_transformacao("ARQUIVO_INALTERADO", f"Mesmo arquivo em < 60min, diff: {diff_minutos:.1f}min")
                    else:
                        log_transformacao("ARQUIVO_ANTIGO", f"Mesmo arquivo mas > 60min, diff: {diff_minutos:.1f}min")
                except Exception as e:
                    log_transformacao("ERRO_TIMESTAMP", f"Erro ao verificar timestamp: {str(e)}")
            else:
                log_transformacao("ARQUIVO_DIFERENTE", f"Arquivo mudou de '{config.get('arquivo')}' para '{arquivo_selecionado.name}'")
    except Exception as e:
        log_transformacao("ERRO_CONFIG", f"Erro ao carregar config: {str(e)}")

estado_bloco_6['arquivo_mudou'] = arquivo_mudou
estado_bloco_6['ultima_sheet_usada'] = ultima_sheet

print(f"\n💡 Última sheet: {ultima_sheet if ultima_sheet else 'Nenhuma'}")
print(f"   Arquivo mudou: {'Sim' if arquivo_mudou else 'Não'}")

# ═══════════════════════════════════════════════════════════════════
# CASO 1: CSV - SELEÇÃO AUTOMÁTICA (apenas 1 sheet virtual)
# ═══════════════════════════════════════════════════════════════════

if tipo_arquivo == 'CSV':
    sheet_nome = sheets[0]  # 'Dados CSV'
    estado_bloco_6['sheet_nome'] = sheet_nome
    estado_bloco_6['metodo_selecao'] = 'auto_csv'

    log_transformacao("SELECAO_AUTO_CSV", f"Sheet virtual automática: '{sheet_nome}'")
    print(f"\n✅ Arquivo CSV - usando sheet virtual automática: '{sheet_nome}'")

# ═══════════════════════════════════════════════════════════════════
# CASO 2: EXCEL - Apenas 1 sheet E arquivo mudou → Usar diretamente
# ═══════════════════════════════════════════════════════════════════

elif len(sheets) == 1 and arquivo_mudou:
    sheet_nome = sheets[0]
    estado_bloco_6['sheet_nome'] = sheet_nome
    estado_bloco_6['metodo_selecao'] = 'auto_unica'

    log_transformacao("SELECAO_AUTO_UNICA", f"Única sheet disponível: '{sheet_nome}'")
    print(f"\n✅ Apenas 1 sheet - selecionando automaticamente: '{sheet_nome}'")

# ═══════════════════════════════════════════════════════════════════
# CASO 3: EXCEL - Mais de 1 sheet OU tem histórico → GUI COM TIMER
# ═══════════════════════════════════════════════════════════════════

else:
    def selecionar_sheet_com_timer(sheets, ultima=None, mostrar_timer=True):
        """GUI com timer para seleção de sheet"""
        log_transformacao("GUI_INICIADA", f"Sheets: {len(sheets)}, Última: {ultima}, Timer: {mostrar_timer}")

        root, frame, resultado, contador = GUIComTimer.criar_janela_com_timer(
            "DETECTOR - Seleção de Sheet",
            600, 450,
            tem_timer=(mostrar_timer and ultima is not None)
        )

        # Título
        tk.Label(
            frame,
            text="📋 Seleção de Sheet",
            font=('Arial', 14, 'bold'),
            bg='white'
        ).pack(pady=(0, 10))

        tk.Label(
            frame,
            text="Selecione a Sheet para processar:",
            font=('Arial', 12, 'bold'),
            bg='white'
        ).pack(pady=(0, 10))

        # Timer (se tem última E timer ativo)
        if ultima and mostrar_timer:
            tk.Label(
                frame,
                text=f"💡 Última sheet usada: '{ultima}'",
                font=('Arial', 10),
                bg='#E3F2FD',
                fg='#1565C0',
                padx=10,
                pady=10
            ).pack(fill=tk.X, pady=(0, 5))

            countdown = GUIComTimer.adicionar_timer(frame, root, resultado, contador)

        # Listbox
        frame_list = tk.Frame(frame, bg='white')
        frame_list.pack(fill=tk.BOTH, expand=True, pady=(0, 10))

        scrollbar = tk.Scrollbar(frame_list)
        scrollbar.pack(side=tk.RIGHT, fill=tk.Y)

        listbox = tk.Listbox(
            frame_list,
            yscrollcommand=scrollbar.set,
            font=('Arial', 10),
            height=8
        )
        listbox.pack(side=tk.LEFT, fill=tk.BOTH, expand=True)
        scrollbar.config(command=listbox.yview)

        for sheet in sheets:
            listbox.insert(tk.END, sheet)

        # Selecionar última ou primeira
        if ultima and ultima in sheets:
            idx = sheets.index(ultima)
            listbox.select_set(idx)
            listbox.see(idx)
        else:
            listbox.select_set(0)

        # Funções
        def nova_selecao():
            resultado['cancelado'] = True
            selecao = listbox.curselection()
            if selecao:
                resultado['valor'] = sheets[selecao[0]]
                log_transformacao("GUI_SELECAO_MANUAL", f"Usuário selecionou: '{sheets[selecao[0]]}'")
            root.quit()
            root.destroy()

        def usar_ultima():
            resultado['cancelado'] = True
            resultado['valor'] = ultima
            log_transformacao("GUI_USAR_ULTIMA", f"Usuário escolheu usar última: '{ultima}'")
            root.quit()
            root.destroy()

        def duplo_clique(event):
            nova_selecao()

        listbox.bind('<Double-Button-1>', duplo_clique)

        # Botões
        GUIComTimer.criar_botoes(
            frame,
            nova_selecao,
            usar_ultima if (ultima and mostrar_timer) else None,
            "Selecionar",
            f"Usar '{ultima}' (10s)" if ultima else None
        )

        # Iniciar timer
        if ultima and mostrar_timer:
            root.after(1000, countdown)

        root.mainloop()

        # Processar resultado
        if resultado.get('timeout') and ultima:
            log_transformacao("GUI_TIMEOUT", f"Timeout (10s) - usando última sheet: '{ultima}'")
            print(f"   ⏱️  Timeout (10s) - usando última sheet")
            return ultima

        return resultado['valor']

    # Executar GUI
    print(f"\nAbrindo janela de seleção...")
    sheet_nome = selecionar_sheet_com_timer(
        sheets,
        ultima_sheet,
        mostrar_timer=(not arquivo_mudou)  # Timer apenas se mesmo arquivo
    )

    estado_bloco_6['sheet_nome'] = sheet_nome
    if ultima_sheet and not arquivo_mudou:
        estado_bloco_6['metodo_selecao'] = 'gui_timer'
    else:
        estado_bloco_6['metodo_selecao'] = 'gui_manual'

# ═══════════════════════════════════════════════════════════════════
# VALIDAÇÃO DA SELEÇÃO
# ═══════════════════════════════════════════════════════════════════

if sheet_nome is None:
    log_transformacao("ERRO_SELECAO", "Nenhuma sheet foi selecionada!")
    raise ValueError("❌ Nenhuma sheet foi selecionada!")

if sheet_nome not in sheets:
    log_transformacao("ERRO_SHEET_INVALIDA", f"Sheet '{sheet_nome}' não existe nas sheets disponíveis")
    raise ValueError(f"❌ Sheet '{sheet_nome}' não encontrada nas sheets disponíveis!")

log_transformacao("SELECAO_VALIDADA", f"Sheet '{sheet_nome}' validada com sucesso")

# ═══════════════════════════════════════════════════════════════════
# SALVAR ESCOLHA (Configuração Última Sheet)
# ═══════════════════════════════════════════════════════════════════

config_data = {
    'arquivo': arquivo_selecionado.name,
    'sheet': sheet_nome,
    'timestamp': datetime.now().isoformat(),
    'tipo_arquivo': tipo_arquivo,
    'metodo_selecao': estado_bloco_6['metodo_selecao']
}

with open(config_file, 'w', encoding='utf-8') as f:
    json.dump(config_data, f, indent=2, ensure_ascii=False)

log_transformacao("CONFIG_SALVA", f"Configuração salva em {config_file.name}")

# ═══════════════════════════════════════════════════════════════════
# SALVAR ESTADO COMPLETO DO BLOCO 6
# ═══════════════════════════════════════════════════════════════════

estado_bloco_6['timestamp_fim'] = datetime.now().isoformat()

# Calcular tempo de execução
ts_inicio = datetime.fromisoformat(estado_bloco_6['timestamp_inicio'])
ts_fim = datetime.fromisoformat(estado_bloco_6['timestamp_fim'])
tempo_execucao = (ts_fim - ts_inicio).total_seconds()

estado_bloco_6['tempo_execucao_segundos'] = tempo_execucao

# Adicionar metadados
estado_bloco_6['metadata'] = {
    'bloco': 'BLOCO_6',
    'descricao': 'Seleção de Sheet/Tabela',
    'versao': '2.0',
    'persistencia': True
}

# Salvar estado
with open(state_file, 'w', encoding='utf-8') as f:
    json.dump(estado_bloco_6, f, indent=2, ensure_ascii=False)

log_transformacao("ESTADO_SALVO", f"Estado completo salvo em {state_file.name}")

print(f"\n✅ Sheet selecionada: '{sheet_nome}'")
print(f"   💾 Estado persistido em: {state_file.name}")
print(f"   📝 Log de transformações em: {log_transformacoes.name}")

# ═══════════════════════════════════════════════════════════════════
# RESUMO
# ═══════════════════════════════════════════════════════════════════

print("\n" + "─"*70)
print("✅ SELEÇÃO CONCLUÍDA")
print("─"*70)
print(f"   Arquivo: {arquivo_selecionado.name}")
print(f"   Sheet: {sheet_nome}")
print(f"   Tipo: {tipo_arquivo}")
print(f"   Método: {estado_bloco_6['metodo_selecao']}")
print(f"   Tempo: {tempo_execucao:.2f}s")
print("─"*70)

log_transformacao("BLOCO_6_CONCLUIDO",
                 f"Sheet='{sheet_nome}', Método={estado_bloco_6['metodo_selecao']}, Tempo={tempo_execucao:.2f}s")

# ═══════════════════════════════════════════════════════════════════
# EXPORTAR VARIÁVEIS CRÍTICAS PARA PRÓXIMOS BLOCOS
# ═══════════════════════════════════════════════════════════════════
# Estas variáveis devem estar disponíveis para os blocos seguintes:
# - sheet_nome: Nome da sheet selecionada
# - tipo_arquivo: 'CSV' ou 'EXCEL'
# - arquivo_selecionado: Path do arquivo
# - sheets: Lista de todas as sheets disponíveis
# ═══════════════════════════════════════════════════════════════════


📋 SELEÇÃO DE SHEET/TABELA
   📝 LOG: INICIO_BLOCO_6
   📝 LOG: CONFIG_CARREGADA
   📝 LOG: ARQUIVO_ANTIGO

💡 Última sheet: Grupos de Produto e Desc
   Arquivo mudou: Sim

Abrindo janela de seleção...
   📝 LOG: GUI_INICIADA
   📝 LOG: GUI_SELECAO_MANUAL
   📝 LOG: SELECAO_VALIDADA
   📝 LOG: CONFIG_SALVA
   📝 LOG: ESTADO_SALVO

✅ Sheet selecionada: 'Grupos de Produto e Desc'
   💾 Estado persistido em: .bloco_6_state.json
   📝 Log de transformações em: bloco_6_transformacoes.log

──────────────────────────────────────────────────────────────────────
✅ SELEÇÃO CONCLUÍDA
──────────────────────────────────────────────────────────────────────
   Arquivo: Grupos de Produtos x Códigos de Materiais.xlsx
   Sheet: Grupos de Produto e Desc
   Tipo: EXCEL
   Método: gui_manual
   Tempo: 4.46s
──────────────────────────────────────────────────────────────────────
   📝 LOG: BLOCO_6_CONCLUIDO


In [12]:
# ═══════════════════════════════════════════════════════════════════
# BLOCO 7 - PREVIEW VISUAL (50 linhas × 20 colunas) - SUPORTE CSV
# ═══════════════════════════════════════════════════════════════════

print("\n" + "="*70)
print("👀 PREVIEW DO ARQUIVO")
print("="*70)

# ═══════════════════════════════════════════════════════════════════
# CASO 1: EXCEL com xlrd (arquivos .xls antigos)
# ═══════════════════════════════════════════════════════════════════

if metodo_carga == 'xlrd':
    print("📊 Método: xlrd")

    sheet = workbook.sheet_by_name(sheet_nome)
    data_preview = []

    for row_idx in range(min(50, sheet.nrows)):
        data_preview.append(sheet.row_values(row_idx))

    df_preview = pd.DataFrame(data_preview)

# ═══════════════════════════════════════════════════════════════════
# CASO 2: EXCEL com pandas (arquivos .xlsx/.xlsm)
# ═══════════════════════════════════════════════════════════════════

elif metodo_carga == 'pandas':
    print("📊 Método: pandas Excel")

    # CORREÇÃO: Ler diretamente do arquivo, não do objeto workbook que pode estar fechado
    df_preview = pd.read_excel(
        arquivo_selecionado,  # ✅ Usar caminho do arquivo
        sheet_name=sheet_nome,
        nrows=50,
        header=None,
        engine='openpyxl'
    )

# ═══════════════════════════════════════════════════════════════════
# CASO 3: CSV 🆕
# ═══════════════════════════════════════════════════════════════════

elif metodo_carga == 'csv':
    print("📄 Método: CSV")

    df_preview = pd.read_csv(
        arquivo_selecionado,
        sep=separador_detectado,
        encoding='cp1252',
        skiprows=skiprows_csv,
        nrows=50,
        header=None  # Sem cabeçalho por enquanto
    )

else:
    raise ValueError(f"❌ Método de carga desconhecido: {metodo_carga}")

# ═══════════════════════════════════════════════════════════════════
# LIMITAR A 20 COLUNAS PARA VISUALIZAÇÃO
# ═══════════════════════════════════════════════════════════════════

df_preview_limitado = df_preview.iloc[:, :20].copy()

# ═══════════════════════════════════════════════════════════════════
# EXIBIR INFORMAÇÕES
# ═══════════════════════════════════════════════════════════════════

print(f"\n📊 Dimensões do preview:")
print(f"   Total: {df_preview.shape[0]} linhas × {df_preview.shape[1]} colunas")
print(f"   Exibindo: {df_preview_limitado.shape[0]} linhas × {df_preview_limitado.shape[1]} colunas")

print(f"\n👁️  Preview (primeiras 50 linhas, até 20 colunas):")
print("─" * 70)

# Usar display ou print dependendo do ambiente
try:
    display(df_preview_limitado)
except NameError:
    print(df_preview_limitado.to_string())

print("─" * 70)

# ═══════════════════════════════════════════════════════════════════
# 💾 PERSISTÊNCIA: SALVAR PREVIEW EM DISCO (Princípio "0% memória, 100% LOG")
# ═══════════════════════════════════════════════════════════════════

try:
    # Importar módulos necessários se ainda não estiverem disponíveis
    import os
    import json
    from datetime import datetime

    # ✅ RECUPERAR pasta_processamento do estado persistido
    if 'pasta_processamento' not in locals() and 'pasta_processamento' not in globals():
        # Tentar recuperar do arquivo de estado
        estado_files = [f for f in os.listdir('..') if f.startswith('.bloco_') and f.endswith('_state.json')]

        if estado_files:
            # Pegar o estado mais recente
            estado_file = sorted(estado_files)[-1]
            with open(estado_file, 'r', encoding='utf-8') as f:
                estado = json.load(f)
                pasta_processamento = estado.get('pasta_processamento')
                if not pasta_processamento:
                    raise ValueError("pasta_processamento não encontrada no estado")
        else:
            # Se não houver estado, criar pasta padrão baseada no arquivo atual
            if 'arquivo_selecionado' in locals() or 'arquivo_selecionado' in globals():
                base_name = os.path.splitext(os.path.basename(arquivo_selecionado))[0]
                pasta_processamento = f'PROCESSAMENTO_{base_name}'
                os.makedirs(pasta_processamento, exist_ok=True)
            else:
                raise ValueError("Não foi possível determinar pasta_processamento")

    # ✅ RECUPERAR log_transformacoes do arquivo persistido ou criar novo
    log_file = os.path.join(pasta_processamento, 'log_transformacoes.json')
    if os.path.exists(log_file):
        with open(log_file, 'r', encoding='utf-8') as f:
            log_transformacoes = json.load(f)
    else:
        log_transformacoes = []

    # Salvar preview completo em Pickle (não requer dependências extras)
    arquivo_preview = os.path.join(pasta_processamento, 'bloco_7_preview.pkl')
    df_preview.to_pickle(arquivo_preview)
    print(f"\n💾 Preview salvo: {arquivo_preview}")

    # Salvar preview limitado também (para referência visual)
    arquivo_preview_limitado = os.path.join(pasta_processamento, 'bloco_7_preview_limitado.pkl')
    df_preview_limitado.to_pickle(arquivo_preview_limitado)
    print(f"💾 Preview limitado salvo: {arquivo_preview_limitado}")

    # Registrar no log de transformações
    log_transformacoes.append({
        'bloco': 7,
        'operacao': 'preview_visual',
        'timestamp': datetime.now().isoformat(),
        'metodo_carga': metodo_carga,
        'linhas_preview': df_preview.shape[0],
        'colunas_preview': df_preview.shape[1],
        'linhas_exibidas': df_preview_limitado.shape[0],
        'colunas_exibidas': df_preview_limitado.shape[1],
        'arquivo_preview': arquivo_preview,
        'arquivo_preview_limitado': arquivo_preview_limitado
    })

    # Salvar log atualizado
    with open(os.path.join(pasta_processamento, 'log_transformacoes.json'), 'w', encoding='utf-8') as f:
        json.dump(log_transformacoes, f, indent=2, ensure_ascii=False)

    print("✅ Log de transformações atualizado")

except Exception as e:
    print(f"⚠️ Aviso: Não foi possível salvar preview em disco: {e}")
    print("   (Continuando sem persistência do preview)")


👀 PREVIEW DO ARQUIVO
📊 Método: pandas Excel

📊 Dimensões do preview:
   Total: 31 linhas × 2 colunas
   Exibindo: 31 linhas × 2 colunas

👁️  Preview (primeiras 50 linhas, até 20 colunas):
──────────────────────────────────────────────────────────────────────


,0,1
0,Cód Grupo de produto,Desc. Grupo de Produto
1,AB9_KG,AB9
2,ACETATO_ETILA_KG,ACETATO DE ETILA
3,ADIT_DIESEL_SIMPLES,ADITIVO DIESEL
4,AGUARRAS_KG,AGUARRAS
5,ANIDRO_SIMPLES,ANIDRO
6,B100_SIMPLES,B100
7,DIESEL_MARÍTIMO_COMP,DIESEL MARÍTIMO
8,DIESEL_MARÍTIMO_SIMP,DIESEL MARÍTIMO
9,DIESEL_S10_ADITIVADO,DIESEL S10 ADITIVADO


──────────────────────────────────────────────────────────────────────

💾 Preview salvo: PROCESSAMENTO_Grupos de Produtos x Códigos de Materiais\bloco_7_preview.pkl
💾 Preview limitado salvo: PROCESSAMENTO_Grupos de Produtos x Códigos de Materiais\bloco_7_preview_limitado.pkl
✅ Log de transformações atualizado


In [15]:
# ═══════════════════════════════════════════════════════════════
# BLOCO 8 - DETECÇÃO E SELEÇÃO AVANÇADA DE CABEÇALHO - COMPLETO
# VERSÃO PERSISTÊNCIA v2.2 - PRINCIPIO "0% MEMÓRIA, 100% LOG"
# Com: Dicionário + Análise Repetição + Multi-linha + Análise Colunas COMPLETA
# Mudanças v2.2:
#   - Salvamento completo de scores em .json
#   - Salvamento de data_para_analise em .json
#   - Salvamento de colunas_analise em .json
#   - Salvamento de blocos_continuos em .json
#   - Salvamento de linha_cabecalho extraída em .json
#   - Estado .bloco_8_state.json com referências a todos arquivos
#   - Log estruturado de transformações
#   - Usa APENAS JSON (sem dependências externas, 100% compatível)
# ═══════════════════════════════════════════════════════════════

from difflib import SequenceMatcher
import re

print("\n" + "="*70)
print("🎯 DETECÇÃO E SELEÇÃO DE CABEÇALHO")
print("="*70)

# ═══════════════════════════════════════════════════════════════
# CARREGAR DICIONÁRIO PERSISTENTE (se não estiver carregado)
# ═══════════════════════════════════════════════════════════════

print("\n🔍 Verificando cabeçalho multi-linha...")

if 'DICIONARIO_PERSISTENTE' not in globals():
    print("\n📚 Carregando dicionário persistente...")

    locais_dicionario = [
        Path.cwd() / 'DICT_Dicionario_Persistente.json',
        fm.pastas['logs'] / 'DICT_Dicionario_Persistente.json',
        Path.cwd().parent / 'DICT_Dicionario_Persistente.json',
    ]

    DICIONARIO_PERSISTENTE = None

    for local in locais_dicionario:
        if local.exists():
            try:
                with open(local, 'r', encoding='utf-8') as f:
                    DICIONARIO_PERSISTENTE = json.load(f)
                print(f"   ✅ Carregado de: {local.name}")
                break
            except:
                continue

    if not DICIONARIO_PERSISTENTE:
        print(f"   ℹ️  Dicionário não encontrado - criando vazio")
        DICIONARIO_PERSISTENTE = {
            'arquivos': {},
            'ultima_atualizacao': None,
            'versao': '1.0'
        }
else:
    print("\n📚 Dicionário persistente já carregado")

# ═══════════════════════════════════════════════════════════════
# FUNÇÃO AVANÇADA DE AVALIAÇÃO DE LINHA
# ═══════════════════════════════════════════════════════════════

def avaliar_linha_cabecalho_avancada(
    linha, idx, total_linhas, df_preview, dicionario_persistente
):
    """Avalia linha como candidata a cabeçalho com múltiplas heurísticas."""
    celulas = [
        str(c).strip() for c in linha
        if str(c).strip() and str(c).strip().lower() not in
        ['nan', 'none', '']
    ]

    if not celulas:
        return {
            'score': 0.0,
            'detalhes': 'Linha vazia',
            'matches_dicionario': []
        }

    score = 0.0
    detalhes = []

    # CRITÉRIO 1: PROPORÇÃO DE CÉLULAS PREENCHIDAS (peso 2.0)
    prop_preenchidas = len(celulas) / len(linha)
    score_preenchidas = prop_preenchidas * 2.0
    score += score_preenchidas
    detalhes.append(
        f"Preench: {prop_preenchidas:.0%} (+{score_preenchidas:.1f})"
    )

    # CRITÉRIO 2: PROPORÇÃO DE TEXTO (peso 2.5)
    tem_texto = sum(1 for c in celulas if re.search(r'[a-zA-Z]', c))
    prop_texto = tem_texto / len(celulas) if celulas else 0
    score_texto = prop_texto * 2.5
    score += score_texto
    detalhes.append(f"Texto: {prop_texto:.0%} (+{score_texto:.1f})")

    # CRITÉRIO 3: MATCH COM DICIONÁRIO PERSISTENTE (peso 4.0)
    bonus_dicionario = 0.0
    matches_dicionario = []

    if dicionario_persistente and 'arquivos' in dicionario_persistente:
        campos_conhecidos = set()
        for arquivo_info in dicionario_persistente.get(
            'arquivos', {}
        ).values():
            if 'campos_mapeados' in arquivo_info:
                for campo_info in arquivo_info[
                    'campos_mapeados'
                ].values():
                    if 'nome_original' in campo_info:
                        campos_conhecidos.add(
                            campo_info['nome_original'].lower()
                        )
                    if 'nome_padrao' in campo_info:
                        campos_conhecidos.add(
                            campo_info['nome_padrao'].lower()
                        )

        for celula in celulas:
            celula_lower = celula.lower()
            if celula_lower in campos_conhecidos:
                bonus_dicionario += 0.8
                matches_dicionario.append(celula[:20])
            elif any(
                conhecido in celula_lower
                for conhecido in campos_conhecidos
            ):
                bonus_dicionario += 0.4
                matches_dicionario.append(f"{celula[:15]}*")

    bonus_dicionario = min(bonus_dicionario, 4.0)
    score += bonus_dicionario

    if bonus_dicionario > 0:
        detalhes.append(
            f"Dict: {len(matches_dicionario)}m (+{bonus_dicionario:.1f})"
        )

    # CRITÉRIO 4: ANÁLISE DE REPETIÇÃO (peso 3.0)
    try:
        linhas_futuras = min(20, total_linhas - idx - 1)

        if linhas_futuras >= 5:
            colunas_com_repeticao = 0
            total_colunas_analisadas = 0

            for col_idx, valor_atual in enumerate(linha):
                valor_atual_str = str(valor_atual).strip()

                if not valor_atual_str or valor_atual_str.lower() in [
                    'nan', 'none', ''
                ]:
                    continue

                total_colunas_analisadas += 1

                repeticoes = 0
                for i in range(1, min(linhas_futuras + 1, 21)):
                    if idx + i < len(df_preview):
                        valor_futuro = str(
                            df_preview.iloc[idx + i, col_idx]
                        ).strip()
                        if valor_futuro == valor_atual_str:
                            repeticoes += 1

                if repeticoes >= 2:
                    colunas_com_repeticao += 1

            if total_colunas_analisadas > 0:
                prop_repeticao = (
                    colunas_com_repeticao / total_colunas_analisadas
                )
                score_repeticao = (1 - prop_repeticao) * 3.0
                score += score_repeticao
                detalhes.append(
                    f"Unic: {(1-prop_repeticao):.0%} "
                    f"(+{score_repeticao:.1f})"
                )
    except:
        pass

    # CRITÉRIO 5: TAMANHO MÉDIO DE STRINGS (peso 1.0)
    tamanho_medio = np.mean([len(c) for c in celulas]) if celulas else 0
    if 5 <= tamanho_medio <= 50:
        score += 1.0
        detalhes.append(f"Tam: {tamanho_medio:.0f} (+1.0)")

    # CRITÉRIO 6: UNICIDADE DENTRO DA LINHA (peso 1.5)
    if len(celulas) == len(set(celulas)):
        score += 1.5
        detalhes.append("Únicos (+1.5)")

    # CRITÉRIO 7: POSIÇÃO NO ARQUIVO (peso 0.5)
    if idx < 50:
        bonus_posicao = (50 - idx) / 100
        score += bonus_posicao
        detalhes.append(f"Pos: {idx+1} (+{bonus_posicao:.2f})")

    # CRITÉRIO 8: ANÁLISE DE DADOS ABAIXO (peso 2.5)
    try:
        if idx + 5 < total_linhas:
            celulas_match_dict = 0
            celulas_numericas_puras = 0
            celulas_com_numeros = 0
            total_celulas_validas = 0

            campos_dict_lower = set()
            if dicionario_persistente and 'arquivos' in dicionario_persistente:
                for arquivo_info in dicionario_persistente.get(
                    'arquivos', {}
                ).values():
                    if 'campos_mapeados' in arquivo_info:
                        for campo_info in arquivo_info[
                            'campos_mapeados'
                        ].values():
                            if 'nome_original' in campo_info:
                                campos_dict_lower.add(
                                    campo_info['nome_original'].lower()
                                )
                            if 'nome_padrao' in campo_info:
                                campos_dict_lower.add(
                                    campo_info['nome_padrao'].lower()
                                )
                            if 'sinonimos' in campo_info:
                                for sin in campo_info['sinonimos']:
                                    campos_dict_lower.add(sin.lower())

            for offset in range(1, 6):
                if idx + offset < len(df_preview):
                    linha_seguinte = df_preview.iloc[idx + offset]

                    for celula in linha_seguinte:
                        celula_str = str(celula).strip()

                        if not celula_str or celula_str.lower() in [
                            'nan', 'none', ''
                        ]:
                            continue

                        total_celulas_validas += 1
                        celula_lower = celula_str.lower()

                        matched = False
                        if campos_dict_lower:
                            if celula_lower in campos_dict_lower:
                                celulas_match_dict += 1
                                matched = True
                            else:
                                for campo_conhecido in campos_dict_lower:
                                    if (campo_conhecido in celula_lower or
                                        celula_lower in campo_conhecido):
                                        if len(campo_conhecido) >= 3:
                                            celulas_match_dict += 1
                                            matched = True
                                            break

                        if not matched:
                            apenas_numeros = re.sub(
                                r'[^0-9.]', '', celula_str
                            )

                            if len(apenas_numeros) > 0:
                                prop_digitos = (
                                    len(apenas_numeros) / len(celula_str)
                                )
                                if prop_digitos > 0.5:
                                    celulas_numericas_puras += 1
                                elif re.search(r'\d', celula_str):
                                    celulas_com_numeros += 1

            if total_celulas_validas > 0:
                prop_dict = celulas_match_dict / total_celulas_validas
                prop_num_puras = (
                    celulas_numericas_puras / total_celulas_validas
                )
                prop_com_num = celulas_com_numeros / total_celulas_validas

                bonus_dados = 0.0
                metodo_usado = None

                if prop_dict > 0.4:
                    bonus_dados = 2.5
                    metodo_usado = f"Dict:{prop_dict:.0%}"
                elif prop_num_puras > 0.6:
                    bonus_dados = 2.0
                    metodo_usado = f"Num:{prop_num_puras:.0%}"
                elif (prop_num_puras + prop_com_num) > 0.7:
                    bonus_dados = 1.0
                    metodo_usado = f"Misto:{(prop_num_puras+prop_com_num):.0%}"

                if bonus_dados > 0:
                    score += bonus_dados
                    detalhes.append(
                        f"DadosAbaixo:{metodo_usado} (+{bonus_dados:.1f})"
                    )
    except:
        pass

    # CRITÉRIO 9: ANTI-DADOS (penalidade)
    try:
        celulas_linha_atual = [
            str(c).strip() for c in linha
            if str(c).strip() and str(c).strip().lower() not in
            ['nan', 'none', '']
        ]

        if celulas_linha_atual:
            num_puras_linha = 0
            for celula in celulas_linha_atual:
                apenas_numeros = re.sub(r'[^0-9.]', '', celula)
                if len(apenas_numeros) > 0:
                    prop_digitos = len(apenas_numeros) / len(celula)
                    if prop_digitos > 0.5:
                        num_puras_linha += 1

            prop_num_linha = num_puras_linha / len(celulas_linha_atual)

            repeticoes_detectadas = 0
            if idx + 5 < total_linhas:
                for col_idx, valor_atual in enumerate(linha):
                    valor_atual_str = str(valor_atual).strip()

                    if (not valor_atual_str or
                        valor_atual_str.lower() in ['nan', 'none', '']):
                        continue

                    for offset in range(1, min(6, total_linhas - idx)):
                        if idx + offset < len(df_preview):
                            valor_seguinte = str(
                                df_preview.iloc[idx + offset, col_idx]
                            ).strip()
                            if valor_seguinte == valor_atual_str:
                                repeticoes_detectadas += 1
                                break

            prop_repeticoes = (
                repeticoes_detectadas / len(celulas_linha_atual)
                if celulas_linha_atual else 0
            )

            penalidade = 0.0

            if prop_num_linha > 0.6 and prop_repeticoes > 0.3:
                penalidade = -3.0
                score += penalidade
                detalhes.append(
                    f"AntiDados:Num{prop_num_linha:.0%}+Rep"
                    f"{prop_repeticoes:.0%} ({penalidade:.1f})"
                )
            elif prop_num_linha > 0.7:
                penalidade = -1.5
                score += penalidade
                detalhes.append(
                    f"AntiDados:Num{prop_num_linha:.0%} ({penalidade:.1f})"
                )
    except:
        pass

    # CRITÉRIO 10: PADRÃO DE RÓTULOS (+4.0 pontos)
    try:
        palavras_rotulo = [
            'centro', 'produto', 'material', 'data', 'valor', 'quantidade',
            'codigo', 'nome', 'descricao', 'tipo', 'categoria', 'grupo',
            'sigla', 'unidade', 'medida', 'periodo', 'mes', 'ano',
            'referencia', 'documento', 'numero', 'id', 'chave', 'hierarq',
            'lucro', 'receita', 'custo', 'despesa', 'saldo', 'total',
            'indice', 'variacao', 'percentual', 'taxa', 'margem'
        ]

        if dicionario_persistente and 'arquivos' in dicionario_persistente:
            for arquivo_info in dicionario_persistente.get(
                'arquivos', {}
            ).values():
                if 'campos_mapeados' in arquivo_info:
                    for campo_info in arquivo_info[
                        'campos_mapeados'
                    ].values():
                        if 'nome_original' in campo_info:
                            palavras_rotulo.append(
                                campo_info['nome_original'].lower()
                            )
                        if 'nome_padrao' in campo_info:
                            palavras_rotulo.append(
                                campo_info['nome_padrao'].lower()
                            )

        palavras_rotulo = set(palavras_rotulo)

        matches_rotulo = 0
        celulas_validas = 0

        for celula in linha:
            celula_str = str(celula).strip()
            if not celula_str or celula_str.lower() in ['nan', 'none', '']:
                continue

            celulas_validas += 1
            celula_lower = celula_str.lower()

            if celula_lower in palavras_rotulo:
                matches_rotulo += 1
            else:
                for palavra in palavras_rotulo:
                    if len(palavra) >= 4 and palavra in celula_lower:
                        matches_rotulo += 1
                        break

        if celulas_validas > 0:
            prop_rotulos = matches_rotulo / celulas_validas

            if prop_rotulos > 0.4:
                bonus_rotulos = 4.0
                score += bonus_rotulos
                detalhes.append(
                    f"Rotulos:{prop_rotulos:.0%} (+{bonus_rotulos:.1f})"
                )
            elif prop_rotulos > 0.25:
                bonus_rotulos = 2.0
                score += bonus_rotulos
                detalhes.append(
                    f"Rotulos:{prop_rotulos:.0%} (+{bonus_rotulos:.1f})"
                )
    except:
        pass

    # CRITÉRIO 11: ANTI-REPETIÇÃO FORTE (-4.0 pontos)
    try:
        if idx + 10 < total_linhas:
            colunas_com_repeticao_forte = 0
            total_colunas_analisadas = 0

            for col_idx, valor_atual in enumerate(linha):
                valor_atual_str = str(valor_atual).strip()

                if not valor_atual_str or valor_atual_str.lower() in [
                    'nan', 'none', ''
                ]:
                    continue

                total_colunas_analisadas += 1

                repeticoes = 0
                for offset in range(1, min(11, total_linhas - idx)):
                    if idx + offset < len(df_preview):
                        valor_seg = str(
                            df_preview.iloc[idx + offset, col_idx]
                        ).strip()
                        if valor_seg == valor_atual_str:
                            repeticoes += 1

                if repeticoes >= 5:
                    colunas_com_repeticao_forte += 1

            if total_colunas_analisadas > 0:
                prop_rep_forte = (
                    colunas_com_repeticao_forte / total_colunas_analisadas
                )

                if prop_rep_forte > 0.3:
                    penalidade_rep = -4.0
                    score += penalidade_rep
                    detalhes.append(
                        f"AntiRep:{prop_rep_forte:.0%} "
                        f"({penalidade_rep:.1f})"
                    )
    except:
        pass

    # CRITÉRIO 12: DENSIDADE DE RÓTULOS DO DICIONÁRIO (+3.0 pontos)
    try:
        if dicionario_persistente and 'arquivos' in dicionario_persistente:
            campos_conhecidos = {}

            for arquivo_info in dicionario_persistente.get(
                'arquivos', {}
            ).values():
                if 'campos_mapeados' in arquivo_info:
                    for campo_info in arquivo_info[
                        'campos_mapeados'
                    ].values():
                        nome_orig = campo_info.get('nome_original', '')
                        nome_pad = campo_info.get('nome_padrao', '')

                        if nome_orig:
                            campos_conhecidos[nome_orig.lower()] = True
                        if nome_pad:
                            campos_conhecidos[nome_pad.lower()] = True

            if campos_conhecidos:
                matches_exatos = 0
                celulas_validas = 0

                for celula in linha:
                    celula_str = str(celula).strip()
                    if not celula_str or celula_str.lower() in [
                        'nan', 'none', ''
                    ]:
                        continue

                    celulas_validas += 1
                    celula_lower = celula_str.lower()

                    if celula_lower in campos_conhecidos:
                        matches_exatos += 1

                if celulas_validas > 0:
                    prop_dict_exato = matches_exatos / celulas_validas

                    if prop_dict_exato > 0.5:
                        bonus_dict_dens = 3.0
                        score += bonus_dict_dens
                        detalhes.append(
                            f"DictDens:{prop_dict_exato:.0%} "
                            f"(+{bonus_dict_dens:.1f})"
                        )
                    elif prop_dict_exato > 0.3:
                        bonus_dict_dens = 1.5
                        score += bonus_dict_dens
                        detalhes.append(
                            f"DictDens:{prop_dict_exato:.0%} "
                            f"(+{bonus_dict_dens:.1f})"
                        )
    except:
        pass

    return {
        'score': score,
        'detalhes': ' | '.join(detalhes),
        'matches_dicionario': matches_dicionario
    }

# ═══════════════════════════════════════════════════════════════
# AVALIAR TODAS AS LINHAS
# ═══════════════════════════════════════════════════════════════

print("\n📊 Analisando linhas para detectar cabeçalho...")

if metodo_carga == 'csv':
    data_para_analise = df_preview.values.tolist()
elif metodo_carga == 'xlrd':
    data_para_analise = []
    sheet = workbook.sheet_by_name(sheet_nome)
    for row_idx in range(min(50, sheet.nrows)):
        data_para_analise.append(sheet.row_values(row_idx))
else:
    data_para_analise = df_preview.values.tolist()

scores = []

for idx, linha in enumerate(data_para_analise):
    resultado = avaliar_linha_cabecalho_avancada(
        linha,
        idx,
        len(data_para_analise),
        df_preview,
        DICIONARIO_PERSISTENTE
    )

    scores.append({
        'linha_excel': idx + 1,
        'indice': idx,
        'score': resultado['score'],
        'detalhes': resultado['detalhes'],
        'matches': resultado['matches_dicionario']
    })

scores = sorted(scores, key=lambda x: x['score'], reverse=True)

# ═══════════════════════════════════════════════════════════════
# 💾 PERSISTÊNCIA: SALVAR SCORES E DATA_PARA_ANALISE
# ═══════════════════════════════════════════════════════════════

print("\n💾 Salvando scores e preview dos dados...")

# Salvar scores como JSON (compatível, sem dependências)
arquivo_scores = fm.pastas['logs'] / '.bloco8_scores.json'
with open(arquivo_scores, 'w', encoding='utf-8') as f:
    json.dump(scores, f, indent=2, ensure_ascii=False)
print(f"   ✅ Scores salvos: {arquivo_scores.name}")

# Salvar data_para_analise como JSON (lista de listas)
# Converter para formato serializável
data_serializada = []
for linha in data_para_analise:
    linha_serializada = []
    for celula in linha:
        # Converter para tipos básicos do Python
        if pd.isna(celula):
            linha_serializada.append(None)
        else:
            linha_serializada.append(str(celula) if not isinstance(celula, (int, float, str, bool, type(None))) else celula)
    data_serializada.append(linha_serializada)

arquivo_data_analise = fm.pastas['logs'] / '.bloco8_data_para_analise.json'
with open(arquivo_data_analise, 'w', encoding='utf-8') as f:
    json.dump({
        'shape': [len(data_para_analise), len(data_para_analise[0]) if data_para_analise else 0],
        'data': data_serializada
    }, f, indent=2, ensure_ascii=False)
print(f"   ✅ Preview salvo: {arquivo_data_analise.name}")

# ═══════════════════════════════════════════════════════════════
# EXIBIR TOP 5 CANDIDATOS
# ═══════════════════════════════════════════════════════════════

print("\n🏆 Top 5 candidatos a cabeçalho:")
print("=" * 70)
print("📍 NUMERAÇÃO: Usamos índice Python (preview inicia em 0)")
print("   • Índice 0 = Linha 1 no Excel")
print("=" * 70)

for i, item in enumerate(scores[:5], 1):
    idx_py = item['indice']
    linha_excel = item['linha_excel']

    print(f"\n   {i}º. Índice {idx_py} (Excel: Linha {linha_excel})")
    print(f"       Score: {item['score']:.2f}/24.5")
    print(f"       {item['detalhes']}")
    if item['matches']:
        matches_str = ', '.join(item['matches'][:5])
        print(f"       Matches: {matches_str}")

melhor = scores[0]
print(f"\n{'='*70}")
print(
    f"🎯 SUGESTÃO AUTOMÁTICA: Índice {melhor['indice']} "
    f"(Excel: Linha {melhor['linha_excel']})"
)
print(f"   Confiança: {melhor['score']:.2f}/24.5")
print("=" * 70)

# ═══════════════════════════════════════════════════════════════
# ANÁLISE DE COLUNAS VÁLIDAS - SISTEMA COMPLETO v2.1
# ═══════════════════════════════════════════════════════════════

def analisar_coluna_valida_COMPLETA(
    col_idx,
    nome_coluna,
    dados_coluna,
    dicionario,
    todas_colunas_info=None
):
    """
    Análise COMPLETA de coluna com 12 critérios avançados.
    Funciona para TABELAS TRANSACIONAIS e RELATÓRIOS BI.
    """
    score = 0.0
    razoes = []
    metodo_usado = None

    valores = [
        str(v).strip() for v in dados_coluna
        if str(v).strip() and str(v).strip().lower() not in ['nan', 'none', '']
    ]

    if not valores:
        return {
            'valida': False,
            'score': 0.0,
            'razoes': ['Coluna vazia'],
            'tipo_detectado': 'VAZIA',
            'confianca': 0.0,
            'metodo': 'VAZIO',
            'prop_preenchimento': 0.0,
            'match_dicionario': None
        }

    # CRITÉRIO 1: SIMILARIDADE COM ALIASES DO DICIONÁRIO (peso 8.0)
    nome_lower = str(nome_coluna).lower().strip()
    melhor_match_alias = None
    melhor_score_alias = 0.0
    campo_matched = None

    if dicionario and 'arquivos' in dicionario:
        aliases_por_campo = {}

        for arq_info in dicionario.get('arquivos', {}).values():
            if 'campos_mapeados' in arq_info:
                for nome_campo, campo_info in arq_info['campos_mapeados'].items():
                    if nome_campo not in aliases_por_campo:
                        aliases_por_campo[nome_campo] = set()

                    if 'nome_original' in campo_info:
                        aliases_por_campo[nome_campo].add(
                            campo_info['nome_original'].lower()
                        )

                    if 'nome_padrao' in campo_info:
                        aliases_por_campo[nome_campo].add(
                            campo_info['nome_padrao'].lower()
                        )

                    if 'sinonimos' in campo_info:
                        for sin in campo_info['sinonimos']:
                            aliases_por_campo[nome_campo].add(sin.lower())

        for campo, aliases in aliases_por_campo.items():
            for alias in aliases:
                if nome_lower == alias:
                    melhor_score_alias = 1.0
                    melhor_match_alias = alias
                    campo_matched = campo
                    break

                similaridade = SequenceMatcher(None, nome_lower, alias).ratio()

                if similaridade > melhor_score_alias:
                    melhor_score_alias = similaridade
                    melhor_match_alias = alias
                    campo_matched = campo

            if melhor_score_alias == 1.0:
                break

    if melhor_score_alias >= 0.95:
        bonus_alias = 8.0
        score += bonus_alias
        razoes.append(f"Alias:Exato({melhor_score_alias:.0%}) +{bonus_alias:.1f}")
        metodo_usado = 'ALIAS_EXATO'

    elif melhor_score_alias >= 0.80:
        bonus_alias = 6.0
        score += bonus_alias
        razoes.append(f"Alias:Similar({melhor_score_alias:.0%}) +{bonus_alias:.1f}")
        metodo_usado = 'ALIAS_SIMILAR'

    elif melhor_score_alias >= 0.60:
        bonus_alias = 3.0
        score += bonus_alias
        razoes.append(f"Alias:Parcial({melhor_score_alias:.0%}) +{bonus_alias:.1f}")
        metodo_usado = 'ALIAS_PARCIAL'

    # CRITÉRIO 2: REGEX NOS CONTEÚDOS vs DICIONÁRIO (peso 7.0)
    melhor_match_regex = None
    melhor_score_regex = 0.0

    if dicionario and 'arquivos' in dicionario:
        padroes_por_campo = {}

        for arq_info in dicionario.get('arquivos', {}).values():
            if 'campos_mapeados' in arq_info:
                for nome_campo, campo_info in arq_info['campos_mapeados'].items():
                    if 'regex' in campo_info:
                        if nome_campo not in padroes_por_campo:
                            padroes_por_campo[nome_campo] = []
                        padroes_por_campo[nome_campo].append(campo_info['regex'])

        for campo, padroes in padroes_por_campo.items():
            for padrao in padroes:
                try:
                    matches = sum(
                        1 for v in valores[:50]
                        if re.match(padrao, v, re.IGNORECASE)
                    )

                    prop_matches = matches / min(len(valores), 50)

                    if prop_matches > melhor_score_regex:
                        melhor_score_regex = prop_matches
                        melhor_match_regex = campo
                except:
                    continue

    if melhor_score_regex >= 0.80:
        bonus_regex = 7.0
        score += bonus_regex
        razoes.append(f"Regex:{melhor_score_regex:.0%} +{bonus_regex:.1f}")
        if not metodo_usado:
            metodo_usado = 'REGEX_CONTEUDO'

    elif melhor_score_regex >= 0.60:
        bonus_regex = 4.0
        score += bonus_regex
        razoes.append(f"Regex:{melhor_score_regex:.0%} +{bonus_regex:.1f}")
        if not metodo_usado:
            metodo_usado = 'REGEX_PARCIAL'

    # CRITÉRIO 3: SIMILARIDADE COM CONTEÚDOS CONHECIDOS (peso 6.0)
    melhor_match_conteudo = None
    melhor_score_conteudo = 0.0

    if dicionario and 'arquivos' in dicionario:
        exemplos_por_campo = {}

        for arq_info in dicionario.get('arquivos', {}).values():
            if 'campos_mapeados' in arq_info:
                for nome_campo, campo_info in arq_info['campos_mapeados'].items():
                    if 'exemplos' in campo_info:
                        if nome_campo not in exemplos_por_campo:
                            exemplos_por_campo[nome_campo] = set()

                        for exemplo in campo_info['exemplos']:
                            exemplos_por_campo[nome_campo].add(
                                str(exemplo).lower().strip()
                            )

        for campo, exemplos in exemplos_por_campo.items():
            matches = 0
            for valor in valores[:50]:
                valor_lower = valor.lower()

                if valor_lower in exemplos:
                    matches += 1
                else:
                    for exemplo in exemplos:
                        sim = SequenceMatcher(None, valor_lower, exemplo).ratio()
                        if sim >= 0.85:
                            matches += 1
                            break

            prop_matches = matches / min(len(valores), 50)

            if prop_matches > melhor_score_conteudo:
                melhor_score_conteudo = prop_matches
                melhor_match_conteudo = campo

    if melhor_score_conteudo >= 0.70:
        bonus_conteudo = 6.0
        score += bonus_conteudo
        razoes.append(f"Conteudo:{melhor_score_conteudo:.0%} +{bonus_conteudo:.1f}")
        if not metodo_usado:
            metodo_usado = 'CONTEUDO_SIMILAR'

    elif melhor_score_conteudo >= 0.50:
        bonus_conteudo = 3.0
        score += bonus_conteudo
        razoes.append(f"Conteudo:{melhor_score_conteudo:.0%} +{bonus_conteudo:.1f}")

    # CRITÉRIO 4: DETECÇÃO DE FÓRMULAS (penalidade -8.0)
    tem_formulas = False

    padroes_formula = [
        r'^=',
        r'^\+',
        r'^SUM\(',
        r'^IF\(',
        r'^VLOOKUP\(',
    ]

    for valor in valores[:20]:
        for padrao in padroes_formula:
            if re.match(padrao, valor, re.IGNORECASE):
                tem_formulas = True
                break
        if tem_formulas:
            break

    if tem_formulas:
        penalidade_formula = -8.0
        score += penalidade_formula
        razoes.append(f"Formula! {penalidade_formula:.1f}")
        if not metodo_usado:
            metodo_usado = 'FORMULA_DETECTADA'

    # CRITÉRIO 5: DIVERSIDADE DE VALORES (peso 4.0)
    valores_unicos = len(set(valores))
    total_valores = len(valores)
    prop_unicos = valores_unicos / total_valores if total_valores else 0

    if prop_unicos > 0.7:
        score += 4.0
        razoes.append(f"Divers:{prop_unicos:.0%} (+4.0)")
    elif prop_unicos > 0.4:
        score += 2.0
        razoes.append(f"Divers:{prop_unicos:.0%} (+2.0)")
    elif prop_unicos < 0.05 and valores_unicos <= 3:
        score -= 3.0
        razoes.append(f"Divers:{prop_unicos:.0%} (-3.0)")

    # CRITÉRIO 6: PADRÃO DE FLAGS (penalidade -4.0)
    flags_comuns = {'true', 'false', 'x', '✓', '0', '1', 'sim', 'não', 'yes', 'no'}
    valores_lower = [v.lower() for v in valores]

    matches_flag = sum(1 for v in valores_lower if v in flags_comuns)
    prop_flags = matches_flag / total_valores if total_valores else 0

    if prop_flags > 0.8:
        score -= 4.0
        razoes.append(f"Flags:{prop_flags:.0%} (-4.0)")

    # CRITÉRIO 7: PALAVRAS-CHAVE DE DADOS (peso 3.0)
    palavras_dados = [
        'centro', 'produto', 'material', 'codigo', 'nome', 'data',
        'valor', 'quantidade', 'preco', 'custo', 'receita',
        'hierarq', 'grupo', 'categoria', 'tipo', 'unidade',
        'periodo', 'mes', 'ano', 'sigla', 'descricao', 'lucro'
    ]

    tem_palavra_chave = any(palavra in nome_lower for palavra in palavras_dados)

    if tem_palavra_chave:
        score += 3.0
        razoes.append(f"Keyword (+3.0)")

    # CRITÉRIO 8: TAMANHO MÉDIO DOS VALORES (peso 2.0)
    tamanho_medio = sum(len(v) for v in valores) / len(valores)

    if 3 <= tamanho_medio <= 100:
        score += 2.0
        razoes.append(f"Tam:{tamanho_medio:.0f} (+2.0)")
    elif tamanho_medio <= 2:
        score -= 2.0
        razoes.append(f"Tam:{tamanho_medio:.0f} (-2.0)")

    # CRITÉRIO 9: MIX NUMÉRICO/ALFABÉTICO (peso 1.0)
    tem_numeros = sum(1 for v in valores if any(c.isdigit() for c in v))
    tem_letras = sum(1 for v in valores if any(c.isalpha() for c in v))

    if tem_numeros > 0 and tem_letras > 0:
        score += 1.0
        razoes.append(f"Mix (+1.0)")

    # CRITÉRIO 10: PREENCHIMENTO PARCIAL (penalidade -5.0)
    prop_preenchimento = len(valores) / len(dados_coluna) if dados_coluna else 0

    if prop_preenchimento < 0.30:
        penalidade_parcial = -5.0
        score += penalidade_parcial
        razoes.append(f"Parcial:{prop_preenchimento:.0%} {penalidade_parcial:.1f}")
        if not metodo_usado:
            metodo_usado = 'PREENCHIMENTO_PARCIAL'

    # CRITÉRIO 11: MUDANÇA ESTRUTURAL (penalidade -6.0)
    if todas_colunas_info and col_idx > 0:
        colunas_anteriores = todas_colunas_info[:col_idx]

        if len(colunas_anteriores) >= 5:
            preench_ultimas_5 = sum(
                c.get('prop_preenchimento', 1.0)
                for c in colunas_anteriores[-5:]
            ) / 5

            if preench_ultimas_5 > 0.80 and prop_preenchimento < 0.50:
                penalidade_estrutural = -6.0
                score += penalidade_estrutural
                razoes.append(
                    f"Estrutural:Queda ({penalidade_estrutural:.1f})"
                )
                if not metodo_usado:
                    metodo_usado = 'MUDANCA_ESTRUTURAL'

    # CRITÉRIO 12: PADRÃO DE NOME "VAZIO" (penalidade -7.0)
    nomes_vazios = ['unnamed', 'column', 'col', 'field', 'nan', 'none']

    nome_eh_vazio = (
        len(nome_lower) == 0 or
        nome_lower in nomes_vazios or
        (len(nome_lower) < 15 and any(vazio in nome_lower for vazio in nomes_vazios if vazio))
    )

    if nome_eh_vazio:
        penalidade_nome = -7.0
        score += penalidade_nome
        razoes.append(f"NomeVazio {penalidade_nome:.1f}")

    # DECISÃO FINAL
    confianca = max(0.0, min(1.0, (score + 10) / 40))

    if score >= 10.0:
        tipo = "DADOS"
        valida = True
    elif score >= 0.0:
        tipo = "INCERTO"
        valida = True
    else:
        tipo = "FLAG/FORMULA/AUXILIAR"
        valida = False

    if not metodo_usado:
        metodo_usado = 'HEURISTICAS_BASICAS'

    return {
        'valida': valida,
        'score': score,
        'razoes': razoes,
        'tipo_detectado': tipo,
        'confianca': confianca,
        'metodo': metodo_usado,
        'prop_preenchimento': prop_preenchimento,
        'match_dicionario': campo_matched or melhor_match_regex or melhor_match_conteudo
    }

# ═══════════════════════════════════════════════════════════════
# EXECUTAR ANÁLISE DE COLUNAS
# ═══════════════════════════════════════════════════════════════

print("\n" + "="*70)
print("🔍 ANALISANDO COLUNAS (Sistema Avançado v2.1)")
print("="*70)
print("Critérios: Similaridade + Regex + Conteúdo + Fórmulas + Estrutura")
print("Funciona para: Tabelas Transacionais e Relatórios BI")
print("="*70)

linha_cabecalho_detectado = data_para_analise[melhor['indice']]

inicio_dados = melhor['indice'] + 1
fim_dados = min(inicio_dados + 50, len(data_para_analise))
dados_para_colunas = data_para_analise[inicio_dados:fim_dados]

print(f"\n📊 Analisando {len(linha_cabecalho_detectado)} colunas...")
print(f"   Amostra de dados: {fim_dados - inicio_dados} linhas")

# PRIMEIRA PASSAGEM: Informações básicas
todas_colunas_info = []

for col_idx, nome_col in enumerate(linha_cabecalho_detectado):
    valores_col = [linha[col_idx] for linha in dados_para_colunas]

    valores_validos = [
        str(v).strip() for v in valores_col
        if str(v).strip() and str(v).strip().lower() not in ['nan', 'none', '']
    ]

    prop_preenchimento = len(valores_validos) / len(valores_col) if valores_col else 0

    todas_colunas_info.append({
        'indice': col_idx,
        'nome': str(nome_col),
        'valores': valores_col,
        'prop_preenchimento': prop_preenchimento
    })

# SEGUNDA PASSAGEM: Análise completa
colunas_analise = []

for col_info in todas_colunas_info:
    col_idx = col_info['indice']
    nome_col = col_info['nome']
    valores_col = col_info['valores']

    analise = analisar_coluna_valida_COMPLETA(
        col_idx,
        nome_col,
        valores_col,
        DICIONARIO_PERSISTENTE,
        todas_colunas_info=todas_colunas_info
    )

    colunas_analise.append({
        'indice': col_idx,
        'excel_col': col_idx + 1,
        'nome': str(nome_col)[:30],
        **analise
    })

colunas_validas = [c for c in colunas_analise if c['valida']]
colunas_invalidas = [c for c in colunas_analise if not c['valida']]

print(f"\n✅ Colunas VÁLIDAS (dados reais): {len(colunas_validas)}")
print(f"❌ Colunas INVÁLIDAS (flags/fórmulas/auxiliares): {len(colunas_invalidas)}")

# ═══════════════════════════════════════════════════════════════
# 💾 PERSISTÊNCIA: SALVAR ANÁLISE DE COLUNAS
# ═══════════════════════════════════════════════════════════════

print("\n💾 Salvando análise de colunas...")

# Preparar dados para salvar (já em formato JSON-compatível)
colunas_analise_salvar = []
for col in colunas_analise:
    col_salvar = col.copy()
    # Converter lista de razoes para string se necessário
    if 'razoes' in col_salvar and isinstance(col_salvar['razoes'], list):
        col_salvar['razoes'] = ' | '.join(col_salvar['razoes'])
    colunas_analise_salvar.append(col_salvar)

arquivo_colunas = fm.pastas['logs'] / '.bloco8_colunas_analise.json'
with open(arquivo_colunas, 'w', encoding='utf-8') as f:
    json.dump(colunas_analise_salvar, f, indent=2, ensure_ascii=False)
print(f"   ✅ Análise de colunas salva: {arquivo_colunas.name}")

# EXIBIR INVÁLIDAS
if colunas_invalidas:
    print(f"\n❌ COLUNAS DETECTADAS COMO INVÁLIDAS:")
    print(f"{'='*70}")

    for col in colunas_invalidas[:15]:
        print(
            f"   Col {col['excel_col']:2d} (idx {col['indice']:2d}): "
            f"{col['nome'][:25]:<25} | "
            f"Score: {col['score']:+6.1f} | "
            f"{col['tipo_detectado']}"
        )

        if col['razoes']:
            razoes_lista = col['razoes'] if isinstance(col['razoes'], list) else [col['razoes']]
            razoes_str = ' | '.join(razoes_lista[:4])
            print(f"      Razões: {razoes_str}")

        if col.get('match_dicionario'):
            print(f"      Match: {col['match_dicionario']}")

# EXIBIR VÁLIDAS
if colunas_validas:
    print(f"\n✅ TOP 10 COLUNAS VÁLIDAS (maiores scores):")
    print(f"{'='*70}")

    colunas_validas_sorted = sorted(
        colunas_validas,
        key=lambda x: x['score'],
        reverse=True
    )

    for col in colunas_validas_sorted[:10]:
        print(
            f"   Col {col['excel_col']:2d} (idx {col['indice']:2d}): "
            f"{col['nome'][:25]:<25} | "
            f"Score: {col['score']:+6.1f} | "
            f"Conf: {col['confianca']:.0%}"
        )

        if col['razoes']:
            razoes_lista = col['razoes'] if isinstance(col['razoes'], list) else [col['razoes']]
            razoes_str = ' | '.join(razoes_lista[:3])
            print(f"      {razoes_str}")

        if col.get('match_dicionario'):
            print(f"      ✓ Match: {col['match_dicionario']}")

# ═══════════════════════════════════════════════════════════════
# DETECTAR MUDANÇAS ESTRUTURAIS E AGRUPAR EM BLOCOS CONTÍNUOS
# v2.1: Tolerância a gaps de até 3 colunas inválidas
# ═══════════════════════════════════════════════════════════════

print(f"\n🔍 DETECTANDO MUDANÇAS ESTRUTURAIS:")
print(f"{'='*70}")

# Coletar índices de colunas válidas
colunas_validas_indices = [c['excel_col'] for c in colunas_analise if c['valida']]

if not colunas_validas_indices:
    blocos_continuos = []
else:
    # Agrupar com tolerância a gaps de até 3 colunas
    blocos_continuos = []
    bloco_atual = [colunas_validas_indices[0]]

    for i in range(1, len(colunas_validas_indices)):
        col_atual = colunas_validas_indices[i]
        col_anterior = colunas_validas_indices[i-1]

        gap = col_atual - col_anterior - 1

        if gap <= 3:
            bloco_atual.append(col_atual)
        else:
            blocos_continuos.append(bloco_atual)
            bloco_atual = [col_atual]

    if bloco_atual:
        blocos_continuos.append(bloco_atual)

# ═══════════════════════════════════════════════════════════════
# 💾 PERSISTÊNCIA: SALVAR BLOCOS CONTÍNUOS
# ═══════════════════════════════════════════════════════════════

blocos_info = []
for i, bloco in enumerate(blocos_continuos, 1):
    primeira = min(bloco)
    ultima = max(bloco)
    tamanho = len(bloco)
    range_completo = ultima - primeira + 1
    gaps_internos = range_completo - tamanho

    blocos_info.append({
        'bloco': i,
        'inicio': primeira,
        'fim': ultima,
        'colunas_validas': tamanho,
        'range_completo': range_completo,
        'gaps': gaps_internos,
        'principal': i == 1
    })

arquivo_blocos = fm.pastas['logs'] / '.bloco8_blocos_continuos.json'
with open(arquivo_blocos, 'w', encoding='utf-8') as f:
    json.dump({
        'total_blocos': len(blocos_continuos),
        'blocos': blocos_info,
        'timestamp': datetime.now().isoformat()
    }, f, indent=2, ensure_ascii=False)
print(f"\n💾 Blocos contínuos salvos: {arquivo_blocos.name}")

if len(blocos_continuos) > 1:
    print(f"\n⚠️  MÚLTIPLAS TABELAS DETECTADAS!")

    for i, bloco in enumerate(blocos_continuos, 1):
        primeira = min(bloco)
        ultima = max(bloco)
        tamanho = len(bloco)

        range_completo = ultima - primeira + 1
        gaps_internos = range_completo - tamanho

        print(f"\n   Tabela {i}:")
        print(f"      Range Excel: {primeira} a {ultima}")
        print(f"      Colunas válidas: {tamanho}")
        if gaps_internos > 0:
            print(f"      Gaps tolerados: {gaps_internos} col(s)")

        if i == 1:
            print(f"      ✓ TABELA PRINCIPAL (use esta!)")
        else:
            print(f"      ⚠️  Tabela auxiliar/complementar")

    primeira_valida = min(blocos_continuos[0])
    ultima_valida = max(blocos_continuos[0])

    print(f"\n💡 RECOMENDAÇÃO:")
    print(f"   Use apenas TABELA PRINCIPAL: colunas {primeira_valida} a {ultima_valida}")

else:
    if colunas_validas:
        primeira_valida = min(c['excel_col'] for c in colunas_validas)
        ultima_valida = max(c['excel_col'] for c in colunas_validas)

        print(f"\n✓ Estrutura contínua detectada")
        print(f"   Colunas válidas: {primeira_valida} a {ultima_valida}")

# DETERMINAR RANGE FINAL
if blocos_continuos:
    col_inicio_sugerido = min(blocos_continuos[0])
    col_fim_sugerido = max(blocos_continuos[0])

    total_range = col_fim_sugerido - col_inicio_sugerido + 1
    total_validas = len(blocos_continuos[0])
    total_gaps = total_range - total_validas
else:
    if colunas_validas:
        col_inicio_sugerido = min(c['excel_col'] for c in colunas_validas)
        col_fim_sugerido = max(c['excel_col'] for c in colunas_validas)
        total_range = col_fim_sugerido - col_inicio_sugerido + 1
        total_validas = len(colunas_validas)
        total_gaps = total_range - total_validas
    else:
        col_inicio_sugerido = 1
        col_fim_sugerido = len(linha_cabecalho_detectado)
        total_range = col_fim_sugerido
        total_validas = 0
        total_gaps = 0

print(f"\n🎯 RANGE FINAL SUGERIDO:")
print(f"{'='*70}")
print(f"   Excel: {col_inicio_sugerido} a {col_fim_sugerido}")
print(f"   Python: {col_inicio_sugerido-1} a {col_fim_sugerido}")
print(f"   Total range: {total_range} colunas")
print(f"   Colunas válidas: {total_validas}")
if total_gaps > 0:
    print(f"   Gaps internos: {total_gaps} (tolerados)")

if col_inicio_sugerido > 1:
    colunas_ignoradas = col_inicio_sugerido - 1
    print(f"\n   ⚠️  Ignorando colunas 1-{colunas_ignoradas}")

if len(blocos_continuos) > 1:
    total_auxiliares = sum(len(bloco) for bloco in blocos_continuos[1:])
    print(f"   ⚠️  Ignorando {total_auxiliares} colunas de tabelas auxiliares")

print("=" * 70)

# SALVAR RELATÓRIO
relatorio_colunas = {
    'timestamp': datetime.now().isoformat(),
    'arquivo': arquivo_selecionado.name,
    'sheet': sheet_nome,
    'total_colunas': len(colunas_analise),
    'colunas_validas': len(colunas_validas),
    'colunas_invalidas': len(colunas_invalidas),
    'blocos_detectados': len(blocos_continuos),
    'range_sugerido': {
        'inicio': col_inicio_sugerido,
        'fim': col_fim_sugerido,
        'total_range': total_range if blocos_continuos else col_fim_sugerido - col_inicio_sugerido + 1,
        'total_validas': total_validas if blocos_continuos else len(colunas_validas),
        'total_gaps': total_gaps if blocos_continuos else 0
    },
    'blocos': blocos_info,
    'detalhes_colunas': [
        {
            'col': c['excel_col'],
            'nome': c['nome'],
            'valida': c['valida'],
            'score': c['score'],
            'tipo': c['tipo_detectado'],
            'metodo': c['metodo'],
            'match': c.get('match_dicionario')
        }
        for c in colunas_analise
    ]
}

with open(
    fm.pastas['logs'] / '.analise_colunas.json',
    'w',
    encoding='utf-8'
) as f:
    json.dump(relatorio_colunas, f, indent=2, ensure_ascii=False)

print(f"\n💾 Relatório salvo: .analise_colunas.json")

# ═══════════════════════════════════════════════════════════════
# DETECTAR MULTI-LINHA
# ═══════════════════════════════════════════════════════════════

if len(scores) > 1:
    segundo = scores[1]

    if (segundo['indice'] == melhor['indice'] + 1 and
        segundo['score'] > (melhor['score'] * 0.5)):

        print(f"   ⚠️  POSSÍVEL CABEÇALHO MULTI-LINHA detectado!")
        print(f"   Linha {melhor['linha_excel']}: Score {melhor['score']:.2f}")
        print(f"   Linha {segundo['linha_excel']}: Score {segundo['score']:.2f}")
        print(f"\n   💡 RECOMENDAÇÃO:")
        print(f"      1. CONCATENAR: Linha1 + ' - ' + Linha2")
        print(f"      2. USAR PRIMEIRA: Linha {melhor['linha_excel']}")
        print(f"      3. PERSONALIZAR via GUI")

        multi_linha_detectado = True
        linha_fim_sugerida = segundo['linha_excel']
    else:
        multi_linha_detectado = False
        linha_fim_sugerida = melhor['linha_excel']
else:
    multi_linha_detectado = False
    linha_fim_sugerida = melhor['linha_excel']

# ═══════════════════════════════════════════════════════════════
# GUI AVANÇADA COM TIMER
# ═══════════════════════════════════════════════════════════════

def selecionar_range_cabecalho_com_timer(
    sugerido_linha,
    sugerido_linha_fim,
    total_linhas,
    total_colunas,
    multi_linha=False,
    col_inicio_sug=1,
    col_fim_sug=None
):
    """GUI avançada para seleção de range de cabeçalho."""

    if col_fim_sug is None:
        col_fim_sug = total_colunas

    config_file = fm.pastas['logs'] / '.ultimo_range_cabecalho.json'
    ultimo_config = None

    if config_file.exists():
        try:
            with open(config_file, 'r', encoding='utf-8') as f:
                cfg = json.load(f)
                if cfg.get('arquivo') == arquivo_selecionado.name:
                    ultimo_config = cfg
        except:
            pass

    root, frame, resultado, contador = GUIComTimer.criar_janela_com_timer(
        "DETECTOR - Seleção Avançada de Cabeçalho",
        650, 850,
        tem_timer=bool(ultimo_config)
    )

    tk.Label(
        frame,
        text="📋 Configuração de Cabeçalho e Dados",
        font=('Arial', 14, 'bold'),
        bg='white'
    ).pack(pady=(0, 10))

    explicacao = tk.Label(
        frame,
        text=(
            "📍 IMPORTANTE: Numeração\n\n"
            "• Índice Python (preview): inicia em 0\n"
            "• Linha Excel (arquivo): inicia em 1\n"
            "• Use LINHA EXCEL nos campos abaixo"
        ),
        font=('Arial', 9),
        bg='#E3F2FD',
        fg='#0D47A1',
        justify=tk.LEFT,
        padx=15,
        pady=10,
        relief=tk.RIDGE,
        borderwidth=2
    )
    explicacao.pack(fill=tk.X, pady=(0, 10))

    idx_sugerido = sugerido_linha - 1
    texto_sugestao = (
        f"🤖 SUGESTÃO AUTOMÁTICA\n"
        f"Cabeçalho: Índice {idx_sugerido} (Excel L{sugerido_linha})\n"
        f"Colunas: {col_inicio_sug} a {col_fim_sug} "
        f"({col_fim_sug - col_inicio_sug + 1} colunas)"
    )

    if multi_linha and sugerido_linha_fim != sugerido_linha:
        idx_fim = sugerido_linha_fim - 1
        texto_sugestao = (
            f"🤖 SUGESTÃO AUTOMÁTICA (MULTI-LINHA)\n"
            f"Cabeçalho: Índices {idx_sugerido}-{idx_fim} "
            f"(Excel L{sugerido_linha}-L{sugerido_linha_fim})\n"
            f"Colunas: {col_inicio_sug} a {col_fim_sug} "
            f"({col_fim_sug - col_inicio_sug + 1} colunas)"
        )

    tk.Label(
        frame,
        text=texto_sugestao,
        font=('Arial', 10, 'bold'),
        bg='#E8F5E9' if not multi_linha else '#FFF9C4',
        fg='#2E7D32' if not multi_linha else '#F57F17',
        padx=15,
        pady=10,
        justify=tk.CENTER
    ).pack(fill=tk.X, pady=(0, 10))

    if col_inicio_sug > 1:
        colunas_ignoradas = col_inicio_sug - 1
        tk.Label(
            frame,
            text=(
                f"⚠️ COLUNAS 1-{colunas_ignoradas} DETECTADAS COMO "
                f"FLAGS/FÓRMULAS\n"
                f"(Serão ignoradas automaticamente)"
            ),
            font=('Arial', 9),
            bg='#FFEBEE',
            fg='#C62828',
            padx=15,
            pady=8,
            justify=tk.CENTER
        ).pack(fill=tk.X, pady=(0, 10))

    if ultimo_config:
        linha_ini = ultimo_config['linha_inicio']
        linha_fim = ultimo_config.get('linha_fim', linha_ini)
        col_ini = ultimo_config['col_inicio']
        col_fim = ultimo_config['col_fim']

        idx_ini = linha_ini - 1
        idx_fim = linha_fim - 1

        texto_ultimo = (
            f"💡 ÚLTIMA CONFIGURAÇÃO USADA\n"
            f"Cabeçalho: Índice {idx_ini}"
        )
        if idx_fim != idx_ini:
            texto_ultimo += f"-{idx_fim}"
        texto_ultimo += f" (Excel L{linha_ini}"
        if linha_fim != linha_ini:
            texto_ultimo += f"-L{linha_fim}"
        texto_ultimo += f") | Colunas {col_ini}-{col_fim}"

        tk.Label(
            frame,
            text=texto_ultimo,
            font=('Arial', 9),
            bg='#FFF3E0',
            fg='#E65100',
            padx=15,
            pady=8,
            justify=tk.CENTER
        ).pack(fill=tk.X, pady=(0, 10))

        countdown = GUIComTimer.adicionar_timer(
            frame, root, resultado, contador
        )

    frame_inputs = tk.Frame(frame, bg='white')
    frame_inputs.pack(fill=tk.X, pady=(10, 0))

    tk.Label(
        frame_inputs,
        text="Linha Cabeçalho INÍCIO (Excel):",
        bg='white',
        font=('Arial', 9)
    ).grid(row=0, column=0, sticky='w', padx=5, pady=5)

    entry_cab_ini = tk.Entry(frame_inputs, width=10, font=('Arial', 10))
    entry_cab_ini.insert(
        0,
        str(ultimo_config['linha_inicio'] if ultimo_config
            else sugerido_linha)
    )
    entry_cab_ini.grid(row=0, column=1, padx=5, pady=5)

    tk.Label(
        frame_inputs,
        text="Linha Cabeçalho FIM (Excel):",
        bg='white',
        font=('Arial', 9)
    ).grid(row=1, column=0, sticky='w', padx=5, pady=5)

    entry_cab_fim = tk.Entry(frame_inputs, width=10, font=('Arial', 10))
    entry_cab_fim.insert(
        0,
        str(ultimo_config.get('linha_fim', sugerido_linha_fim)
            if ultimo_config else sugerido_linha_fim)
    )
    entry_cab_fim.grid(row=1, column=1, padx=5, pady=5)

    tk.Label(
        frame_inputs,
        text="Coluna INÍCIO:",
        bg='white',
        font=('Arial', 9)
    ).grid(row=2, column=0, sticky='w', padx=5, pady=5)

    entry_col_ini = tk.Entry(frame_inputs, width=10, font=('Arial', 10))
    entry_col_ini.insert(
        0,
        str(ultimo_config['col_inicio'] if ultimo_config
            else col_inicio_sug)
    )
    entry_col_ini.grid(row=2, column=1, padx=5, pady=5)

    tk.Label(
        frame_inputs,
        text="Coluna FIM:",
        bg='white',
        font=('Arial', 9)
    ).grid(row=3, column=0, sticky='w', padx=5, pady=5)

    entry_col_fim = tk.Entry(frame_inputs, width=10, font=('Arial', 10))
    entry_col_fim.insert(
        0,
        str(ultimo_config['col_fim'] if ultimo_config
            else col_fim_sug)
    )
    entry_col_fim.grid(row=3, column=1, padx=5, pady=5)

    tk.Label(
        frame_inputs,
        text="Linha DADOS início (Excel):",
        bg='white',
        font=('Arial', 9)
    ).grid(row=4, column=0, sticky='w', padx=5, pady=5)

    entry_dados_ini = tk.Entry(
        frame_inputs, width=10, font=('Arial', 10)
    )

    dados_inicio_default = (
        ultimo_config.get('linha_fim', sugerido_linha_fim)
        if ultimo_config else sugerido_linha_fim
    ) + 1

    entry_dados_ini.insert(
        0,
        str(ultimo_config.get('linha_dados_inicio', dados_inicio_default)
            if ultimo_config else dados_inicio_default)
    )
    entry_dados_ini.grid(row=4, column=1, padx=5, pady=5)

    tk.Label(
        frame,
        text=(
            "💡 Para cabeçalho 1 linha: Início = Fim\n"
            "💡 Para multi-linha: Início < Fim (ex: 3 a 4)"
        ),
        font=('Arial', 8, 'italic'),
        bg='#FFFDE7',
        fg='#F57F17',
        padx=10,
        pady=8,
        justify=tk.LEFT
    ).pack(fill=tk.X, pady=(10, 0))

    def confirmar():
        resultado['cancelado'] = True
        try:
            resultado['valor'] = {
                'linha_inicio': int(entry_cab_ini.get()),
                'linha_fim': int(entry_cab_fim.get()),
                'col_inicio': int(entry_col_ini.get()),
                'col_fim': int(entry_col_fim.get()),
                'linha_dados_inicio': int(entry_dados_ini.get())
            }
        except ValueError:
            resultado['valor'] = None
        root.quit()
        root.destroy()

    def usar_ultima():
        resultado['cancelado'] = True
        resultado['valor'] = {
            'linha_inicio': ultimo_config['linha_inicio'],
            'linha_fim': ultimo_config.get(
                'linha_fim', ultimo_config['linha_inicio']
            ),
            'col_inicio': ultimo_config['col_inicio'],
            'col_fim': ultimo_config['col_fim'],
            'linha_dados_inicio': ultimo_config['linha_dados_inicio']
        }
        root.quit()
        root.destroy()

    GUIComTimer.criar_botoes(
        frame,
        confirmar,
        usar_ultima if ultimo_config else None,
        "✅ Confirmar",
        "⏱️ Usar Última (10s)"
    )

    if ultimo_config:
        root.after(1000, countdown)

    root.mainloop()

    if resultado.get('timeout') and ultimo_config:
        print(f"   ⏱️  Timeout - usando última configuração")
        return {
            'linha_inicio': ultimo_config['linha_inicio'],
            'linha_fim': ultimo_config.get(
                'linha_fim', ultimo_config['linha_inicio']
            ),
            'col_inicio': ultimo_config['col_inicio'],
            'col_fim': ultimo_config['col_fim'],
            'linha_dados_inicio': ultimo_config['linha_dados_inicio']
        }

    return resultado['valor']

# EXECUTAR GUI
print("\nAbrindo janela de configuração...")
config = selecionar_range_cabecalho_com_timer(
    melhor['linha_excel'],
    linha_fim_sugerida,
    len(data_para_analise),
    len(data_para_analise[0]) if data_para_analise else 1,
    multi_linha=multi_linha_detectado,
    col_inicio_sug=col_inicio_sugerido,
    col_fim_sug=col_fim_sugerido
)

if not config:
    raise ValueError("❌ Configuração inválida ou cancelada")

# Salvar configuração
with open(
    fm.pastas['logs'] / '.ultimo_range_cabecalho.json',
    'w',
    encoding='utf-8'
) as f:
    config_salvar = config.copy()
    config_salvar['arquivo'] = arquivo_selecionado.name
    config_salvar['sheet'] = sheet_nome
    config_salvar['timestamp'] = datetime.now().isoformat()
    json.dump(config_salvar, f, indent=2)

# Extrair informações
linha_cabecalho_inicio = config['linha_inicio']
linha_cabecalho_fim = config['linha_fim']
col_inicio = config['col_inicio']
col_fim = config['col_fim']
linha_dados_inicio = config['linha_dados_inicio']

print(f"\n✅ CONFIGURAÇÃO CONFIRMADA:")
print(f"{'='*70}")
print(f"   📋 Cabeçalho (Excel): L{linha_cabecalho_inicio}", end="")
if linha_cabecalho_fim != linha_cabecalho_inicio:
    print(f" a L{linha_cabecalho_fim}")
else:
    print()

print(f"   📊 Colunas (Excel): {col_inicio} a {col_fim}")
print(f"   📈 Dados começam (Excel): L{linha_dados_inicio}")

# Converter para índices Python
idx_cab_inicio = linha_cabecalho_inicio - 1
idx_cab_fim = linha_cabecalho_fim - 1
idx_col_inicio = col_inicio - 1
idx_col_fim = col_fim
idx_dados_inicio = linha_dados_inicio - 1

print(f"\n   🐍 Índices Python (uso interno):")
print(f"      Cabeçalho: idx {idx_cab_inicio}", end="")
if idx_cab_fim != idx_cab_inicio:
    print(f" a {idx_cab_fim}")
else:
    print()
print(f"      Colunas: idx {idx_col_inicio} a {idx_col_fim}")
print(f"      Dados: a partir de idx {idx_dados_inicio}")
print("=" * 70)

# ═══════════════════════════════════════════════════════════════
# 💾 PERSISTÊNCIA: EXTRAIR E SALVAR CABEÇALHO FINAL
# ═══════════════════════════════════════════════════════════════

print("\n💾 Extraindo e salvando cabeçalho final...")

# Extrair linha(s) de cabeçalho
if idx_cab_fim == idx_cab_inicio:
    # Cabeçalho de 1 linha
    linha_cabecalho_final = data_para_analise[idx_cab_inicio][idx_col_inicio:idx_col_fim]
else:
    # Cabeçalho multi-linha - concatenar
    linhas_cab = []
    for i in range(idx_cab_inicio, idx_cab_fim + 1):
        linhas_cab.append(data_para_analise[i][idx_col_inicio:idx_col_fim])

    # Concatenar células correspondentes
    linha_cabecalho_final = []
    for col_idx in range(len(linhas_cab[0])):
        partes = []
        for linha in linhas_cab:
            valor = str(linha[col_idx]).strip()
            if valor and valor.lower() not in ['nan', 'none', '']:
                partes.append(valor)
        linha_cabecalho_final.append(' - '.join(partes) if partes else f'Coluna_{col_idx+1}')

# Salvar cabeçalho como JSON
cabecalho_dados = {
    'colunas': [
        {
            'indice_python': i,
            'excel_col': col_inicio + i,
            'nome_coluna': nome
        }
        for i, nome in enumerate(linha_cabecalho_final)
    ],
    'shape': {
        'total_colunas': len(linha_cabecalho_final),
        'col_inicio': col_inicio,
        'col_fim': col_fim
    }
}

arquivo_cabecalho = fm.pastas['logs'] / '.bloco8_cabecalho_final.json'
with open(arquivo_cabecalho, 'w', encoding='utf-8') as f:
    json.dump(cabecalho_dados, f, indent=2, ensure_ascii=False)
print(f"   ✅ Cabeçalho final salvo: {arquivo_cabecalho.name}")

# ═══════════════════════════════════════════════════════════════
# 💾 PERSISTÊNCIA: LOG DE TRANSFORMAÇÕES
# ═══════════════════════════════════════════════════════════════

log_transformacoes = {
    'timestamp': datetime.now().isoformat(),
    'bloco': 8,
    'arquivo_origem': arquivo_selecionado.name,
    'sheet': sheet_nome,
    'transformacoes': [
        {
            'etapa': 'deteccao_cabecalho',
            'metodo': 'scoring_avancado_v2.2',
            'entrada': {
                'linhas_analisadas': len(data_para_analise),
                'total_colunas': len(data_para_analise[0]) if data_para_analise else 0
            },
            'saida': {
                'linha_selecionada': melhor['indice'],
                'score': melhor['score'],
                'multi_linha': multi_linha_detectado
            }
        },
        {
            'etapa': 'analise_colunas',
            'metodo': 'criterios_12_v2.1',
            'entrada': {
                'total_colunas': len(linha_cabecalho_detectado)
            },
            'saida': {
                'colunas_validas': len(colunas_validas),
                'colunas_invalidas': len(colunas_invalidas),
                'blocos_detectados': len(blocos_continuos)
            }
        },
        {
            'etapa': 'configuracao_usuario',
            'metodo': 'gui_com_timer',
            'entrada': {
                'sugestao_linha': melhor['linha_excel'],
                'sugestao_linha_fim': linha_fim_sugerida,
                'sugestao_col_inicio': col_inicio_sugerido,
                'sugestao_col_fim': col_fim_sugerido
            },
            'saida': config
        },
        {
            'etapa': 'extracao_cabecalho',
            'metodo': 'concatenacao_multi_linha' if idx_cab_fim != idx_cab_inicio else 'linha_unica',
            'entrada': {
                'linhas_cab': list(range(idx_cab_inicio, idx_cab_fim + 1)),
                'colunas': list(range(idx_col_inicio, idx_col_fim))
            },
            'saida': {
                'total_colunas': len(linha_cabecalho_final),
                'amostra': linha_cabecalho_final[:5]
            }
        }
    ]
}

arquivo_log_trans = fm.pastas['logs'] / '.bloco8_log_transformacoes.json'
with open(arquivo_log_trans, 'w', encoding='utf-8') as f:
    json.dump(log_transformacoes, f, indent=2, ensure_ascii=False)
print(f"   ✅ Log de transformações salvo: {arquivo_log_trans.name}")

# ═══════════════════════════════════════════════════════════════
# 💾 PERSISTÊNCIA: ESTADO COMPLETO DO BLOCO 8
# ═══════════════════════════════════════════════════════════════

estado_bloco8 = {
    'bloco': 8,
    'versao': '2.2',
    'status': 'concluido',
    'timestamp': datetime.now().isoformat(),
    'arquivo_origem': arquivo_selecionado.name,
    'sheet': sheet_nome,

    # Configuração final
    'config': config,

    # Índices Python
    'indices_python': {
        'cabecalho_inicio': idx_cab_inicio,
        'cabecalho_fim': idx_cab_fim,
        'col_inicio': idx_col_inicio,
        'col_fim': idx_col_fim,
        'dados_inicio': idx_dados_inicio
    },

    # Detecção
    'deteccao': {
        'metodo': 'scoring_avancado_v2.2',
        'melhor_score': melhor['score'],
        'multi_linha': multi_linha_detectado,
        'total_candidatos': len(scores),
        'correcoes': ['bug_string_vazia', 'bug_diversidade', 'tolerancia_gaps']
    },

    # Análise de colunas
    'analise_colunas': {
        'total': len(colunas_analise),
        'validas': len(colunas_validas),
        'invalidas': len(colunas_invalidas),
        'blocos_detectados': len(blocos_continuos),
        'range_final': {
            'inicio': col_inicio_sugerido,
            'fim': col_fim_sugerido,
            'total': total_range,
            'gaps': total_gaps
        }
    },

    # Arquivos salvos (REFERÊNCIAS)
    'arquivos_gerados': {
        'scores': '.bloco8_scores.json',
        'data_analise': '.bloco8_data_para_analise.json',
        'colunas_analise': '.bloco8_colunas_analise.json',
        'blocos_continuos': '.bloco8_blocos_continuos.json',
        'cabecalho_final': '.bloco8_cabecalho_final.json',
        'log_transformacoes': '.bloco8_log_transformacoes.json',
        'relatorio_colunas': '.analise_colunas.json',
        'ultima_config': '.ultimo_range_cabecalho.json'
    },

    # Metadados para próximos blocos
    'output_para_proximo_bloco': {
        'linha_cabecalho_final': True,
        'idx_cab_inicio': idx_cab_inicio,
        'idx_cab_fim': idx_cab_fim,
        'idx_col_inicio': idx_col_inicio,
        'idx_col_fim': idx_col_fim,
        'idx_dados_inicio': idx_dados_inicio,
        'total_colunas_validas': len(linha_cabecalho_final)
    }
}

arquivo_estado = fm.pastas['logs'] / '.bloco_8_state.json'
with open(arquivo_estado, 'w', encoding='utf-8') as f:
    json.dump(estado_bloco8, f, indent=2, ensure_ascii=False)

print(f"   ✅ Estado do bloco salvo: {arquivo_estado.name}")

print("\n" + "="*70)
print("✅ DETECÇÃO DE CABEÇALHO CONCLUÍDA")
print("="*70)
print("\n📦 ARQUIVOS GERADOS (princípio 0% memória, 100% LOG):")
for nome, arquivo in estado_bloco8['arquivos_gerados'].items():
    print(f"   • {arquivo}")
print("="*70)


🎯 DETECÇÃO E SELEÇÃO DE CABEÇALHO

🔍 Verificando cabeçalho multi-linha...

📚 Dicionário persistente já carregado

📊 Analisando linhas para detectar cabeçalho...

💾 Salvando scores e preview dos dados...
   ✅ Scores salvos: .bloco8_scores.json
   ✅ Preview salvo: .bloco8_data_para_analise.json

🏆 Top 5 candidatos a cabeçalho:
📍 NUMERAÇÃO: Usamos índice Python (preview inicia em 0)
   • Índice 0 = Linha 1 no Excel

   1º. Índice 0 (Excel: Linha 1)
       Score: 14.50/24.5
       Preench: 100% (+2.0) | Texto: 100% (+2.5) | Unic: 100% (+3.0) | Tam: 21 (+1.0) | Únicos (+1.5) | Pos: 1 (+0.50) | Rotulos:100% (+4.0)

   2º. Índice 7 (Excel: Linha 8)
       Score: 11.43/24.5
       Preench: 100% (+2.0) | Texto: 100% (+2.5) | Unic: 100% (+3.0) | Tam: 18 (+1.0) | Únicos (+1.5) | Pos: 8 (+0.43) | DadosAbaixo:Misto:80% (+1.0)

   3º. Índice 8 (Excel: Linha 9)
       Score: 11.42/24.5
       Preench: 100% (+2.0) | Texto: 100% (+2.5) | Unic: 100% (+3.0) | Tam: 18 (+1.0) | Únicos (+1.5) | Pos: 9 (+0.

In [18]:
# ===================================================================
# BLOCO 9 - EXTRAÇÃO DE DADOS (REVISADO v2.1)
# ===================================================================
# MUDANÇAS v2.1:
# + Salvamento de TODAS variáveis necessárias (df, df_bruto) em .parquet
# + Estado completo com paths de recuperação
# + Instruções de recuperação documentadas
# + 100% recuperável de arquivos
# ===================================================================

print("\n" + "="*70)
print("EXTRACAO DE DADOS")
print("="*70)

# ===================================================================
# 1. CONECTAR COM BLOCOS ANTERIORES (0% memoria, 100% LOG)
# ===================================================================

# Carregar LOG GLOBAL
try:
    arquivo_log_global = Path('.logs/.log_global.json')
    with open(arquivo_log_global, 'r', encoding='utf-8') as f:
        log_global = json.load(f)

    pasta_base = Path(log_global['pasta_base'])
    timestamp_execucao = log_global['timestamp']

    print(f"LOG GLOBAL conectado!")

except Exception as e:
    print(f"ERRO: Nao foi possivel conectar ao LOG GLOBAL")
    print(f"   Execute o BLOCO 1 primeiro!")
    raise

# Recriar FileManager
fm = FileManagerInterativo(pasta_base, timestamp_execucao)

# Validar que BLOCOS 4-8 foram executados
arquivo_config = fm.pastas['logs'] / '.config_detectada.json'
if not arquivo_config.exists():
    print(f"\nERRO: Configuracao nao encontrada!")
    print(f"   Execute os BLOCOS 4-8 primeiro!")
    raise FileNotFoundError("Config necessaria")

with open(arquivo_config, 'r', encoding='utf-8') as f:
    config = json.load(f)

print(f"\nConfiguracao carregada:")
print(f"   Arquivo: {config['arquivo_processado']}")
print(f"   Sheet: {config['sheet_nome']}")
print(f"   Cabecalho: Linhas {config['linha_cabecalho_inicio']} a {config['linha_cabecalho_fim']}")
print(f"   Colunas: {config['col_inicio']} a {config['col_fim']}")

# Extrair variaveis necessarias
arquivo_selecionado = Path(config['caminho_completo'])
metodo_carga = config['metodo_carga']
sheet_nome = config['sheet_nome']
linha_cabecalho_inicio = config['linha_cabecalho_inicio']
linha_cabecalho_fim = config['linha_cabecalho_fim']
idx_cab_inicio = config['idx_cab_inicio']
idx_cab_fim = config['idx_cab_fim']
col_inicio = config['col_inicio']
col_fim = config['col_fim']
idx_col_inicio = config['idx_col_inicio']
idx_col_fim = config['idx_col_fim']
linha_dados_inicio = config.get('linha_dados_inicio', linha_cabecalho_inicio + 1)
idx_dados_inicio = linha_dados_inicio - 1

# Carregar workbook (pode estar na memoria ou nao)
if 'workbook' not in globals():
    print(f"\nRecarregando workbook...")
    if metodo_carga == 'xlrd':
        workbook = xlrd.open_workbook(str(arquivo_selecionado))
    elif metodo_carga == 'pandas':
        workbook = pd.ExcelFile(str(arquivo_selecionado))
    else:
        workbook = None  # CSV nao precisa
    print(f"   Workbook recarregado!")

# ===================================================================
# 2. EXTRAÇÃO DE DADOS (3 CASOS: CSV, PANDAS, XLRD)
# ===================================================================

print(f"\nConfiguracao (notacao Excel para usuario):")
print(f"   Cabecalho: Linha(s) {linha_cabecalho_inicio} a {linha_cabecalho_fim}")
print(f"   Colunas: {col_inicio} a {col_fim}")
print(f"   Dados: A partir da linha {linha_dados_inicio}")

print(f"\nIndices Python (interno):")
print(f"   Cabecalho: {idx_cab_inicio} a {idx_cab_fim}")
print(f"   Colunas: {idx_col_inicio} a {idx_col_fim}")
print(f"   Dados: a partir de {idx_dados_inicio}")

# CASO 1: ARQUIVO CSV
if metodo_carga == 'csv':
    print(f"\nMetodo: CSV")
    print(f"   Detectando separador...")

    # Detectar separador
    with open(arquivo_selecionado, 'r', encoding='utf-8') as f:
        sample = f.read(1024)

    sniffer = csv.Sniffer()
    separador = sniffer.sniff(sample).delimiter
    print(f"   Separador detectado: '{separador}'")

    # Carregar CSV
    df = pd.read_csv(
        arquivo_selecionado,
        sep=separador,
        header=idx_cab_inicio,
        usecols=range(idx_col_inicio, idx_col_fim)
    )

    # Pular linhas se necessario
    linhas_pular = idx_dados_inicio - idx_cab_inicio - 1
    if linhas_pular > 0:
        df = df.iloc[linhas_pular:].copy()

    print(f"   Carregado: {len(df):,} registros x {len(df.columns)} colunas")

# CASO 2: ARQUIVO EXCEL (PANDAS) - .xlsx, .xlsm
elif metodo_carga == 'pandas':
    print(f"\nMetodo: pandas (XLSX/XLSM)")

    try:
        # CASO 2A: Cabecalho em 1 linha
        if linha_cabecalho_inicio == linha_cabecalho_fim:
            print(f"   Cabecalho: Linha unica ({linha_cabecalho_inicio})")

            df = pd.read_excel(
                arquivo_selecionado,
                sheet_name=sheet_nome,
                header=idx_cab_inicio,
                usecols=range(idx_col_inicio, idx_col_fim)
            )

            # Remover linhas antes dos dados
            linhas_pular = idx_dados_inicio - idx_cab_inicio - 1
            if linhas_pular > 0:
                print(f"   Pulando {linhas_pular} linha(s) apos cabecalho")
                df = df.iloc[linhas_pular:].copy()

        # CASO 2B: Cabecalho multi-linha
        else:
            print(f"   Cabecalho: Multi-linha ({linha_cabecalho_inicio} a {linha_cabecalho_fim})")

            df_temp = pd.read_excel(
                arquivo_selecionado,
                sheet_name=sheet_nome,
                header=None,
                usecols=range(idx_col_inicio, idx_col_fim)
            )

            # Combinar linhas do cabecalho
            cabecalho_linhas = df_temp.iloc[idx_cab_inicio:idx_cab_fim+1].values
            cab_final = []

            for col_idx in range(cabecalho_linhas.shape[1]):
                partes = [
                    str(linha[col_idx]).strip()
                    for linha in cabecalho_linhas
                    if str(linha[col_idx]).strip() not in ['', 'nan', 'None']
                ]
                cab_final.append(' - '.join(partes) if partes else f'Col_{col_idx}')

            # Extrair dados
            df = df_temp.iloc[idx_dados_inicio:].copy()
            df.columns = cab_final

        print(f"   Carregado: {len(df):,} registros x {len(df.columns)} colunas")

    except Exception as e:
        print(f"   ERRO ao extrair com pandas: {str(e)}")
        print(f"\n   DEBUG:")
        print(f"      Arquivo: {arquivo_selecionado}")
        print(f"      Sheet: {sheet_nome}")
        print(f"      Cabecalho: linhas {idx_cab_inicio} a {idx_cab_fim}")
        print(f"      Colunas: {idx_col_inicio} a {idx_col_fim}")
        raise

# CASO 3: ARQUIVO EXCEL (XLRD) - .xls antigos
elif metodo_carga == 'xlrd':
    print(f"\nMetodo: xlrd (XLS)")

    try:
        sheet = workbook.sheet_by_name(sheet_nome)

        # CASO 3A: Cabecalho em 1 linha
        if linha_cabecalho_inicio == linha_cabecalho_fim:
            print(f"   Cabecalho: Linha unica ({linha_cabecalho_inicio})")

            # Extrair cabecalho
            cabecalho = sheet.row_values(idx_cab_inicio)[idx_col_inicio:idx_col_fim]

            # Extrair dados
            data = []
            for row_idx in range(idx_dados_inicio, sheet.nrows):
                data.append(sheet.row_values(row_idx)[idx_col_inicio:idx_col_fim])

            df = pd.DataFrame(data, columns=cabecalho)

        # CASO 3B: Cabecalho multi-linha
        else:
            print(f"   Cabecalho: Multi-linha ({linha_cabecalho_inicio} a {linha_cabecalho_fim})")

            # Extrair linhas do cabecalho
            cabecalho_linhas = []
            for linha_idx in range(idx_cab_inicio, idx_cab_fim + 1):
                cabecalho_linhas.append(
                    sheet.row_values(linha_idx)[idx_col_inicio:idx_col_fim]
                )

            # Combinar linhas do cabecalho
            cab_final = []
            for col_idx in range(len(cabecalho_linhas[0])):
                partes = [
                    str(linha[col_idx]).strip()
                    for linha in cabecalho_linhas
                    if str(linha[col_idx]).strip() not in ['', 'nan', 'None']
                ]
                cab_final.append(' - '.join(partes) if partes else f'Col_{col_idx}')

            # Extrair dados
            data = []
            for row_idx in range(idx_dados_inicio, sheet.nrows):
                data.append(sheet.row_values(row_idx)[idx_col_inicio:idx_col_fim])

            df = pd.DataFrame(data, columns=cab_final)

        print(f"   Carregado: {len(df):,} registros x {len(df.columns)} colunas")

    except Exception as e:
        print(f"   ERRO ao extrair com xlrd: {str(e)}")
        print(f"\n   DEBUG:")
        print(f"      Sheet: {sheet_nome}")
        print(f"      Cabecalho: linhas {idx_cab_inicio} a {idx_cab_fim}")
        raise

else:
    raise ValueError(f"Metodo de carga desconhecido: {metodo_carga}")

# ===================================================================
# 3. VALIDAÇÕES PÓS-EXTRAÇÃO
# ===================================================================

print("\n" + "-"*70)
print("VALIDACOES")
print("-"*70)

# Reset index
df = df.reset_index(drop=True)

# Validar shape
print(f"\nShape final:")
print(f"   Registros: {len(df):,}")
print(f"   Colunas: {len(df.columns)}")

# Validar colunas
print(f"\nPrimeiras 10 colunas:")
for i, col in enumerate(df.columns[:10], 1):
    print(f"   {i:2d}. {col}")

if len(df.columns) > 10:
    print(f"   ... e mais {len(df.columns) - 10} colunas")

# Primeiras 5 linhas
print(f"\nPrimeiras 5 linhas (amostra):")
print(df.head().to_string())

# Tipos detectados
print(f"\nTipos detectados:")
tipos_count = df.dtypes.value_counts()
for tipo, count in tipos_count.items():
    print(f"   {str(tipo).ljust(15)}: {count} coluna(s)")

# Valores nulos
print(f"\nValores nulos:")
nulos_total = df.isnull().sum().sum()
if nulos_total > 0:
    print(f"   Total: {nulos_total:,} celulas vazias")
    colunas_com_nulos = df.isnull().sum()
    colunas_com_nulos = colunas_com_nulos[colunas_com_nulos > 0].sort_values(ascending=False)
    print(f"\n   Top 5 colunas com nulos:")
    for col, count in colunas_com_nulos.head(5).items():
        pct = (count / len(df)) * 100
        print(f"      {col[:40].ljust(40)}: {count:>6,} ({pct:>5.1f}%)")
else:
    print(f"   Nenhum valor nulo!")

# Memoria
memoria_mb = df.memory_usage(deep=True).sum() / 1024**2
print(f"\nMemoria utilizada: {memoria_mb:.2f} MB")

print("\n" + "="*70)
print("EXTRACAO CONCLUIDA COM SUCESSO")
print("="*70)

# ===================================================================
# 4. CRIAR E SALVAR TODAS AS VARIÁVEIS NECESSÁRIAS
# ===================================================================

print(f"\n" + "-"*70)
print("SALVANDO VARIAVEIS EM ARQUIVO (Principio 0% memoria)")
print("-"*70)

# Criar df_bruto (cópia do dataframe original extraído)
df_bruto = df.copy()

# Dicionário para rastrear todos os arquivos salvos
arquivos_salvos = {}

# 1. SALVAR df_bruto
print(f"\n1. Salvando df_bruto...")
nome_df_bruto = f"DF_BRUTO_{fm.timestamp}.parquet"
arquivo_df_bruto = fm.pastas['dados_processados'] / nome_df_bruto
df_bruto.to_parquet(arquivo_df_bruto, index=False, compression='snappy')
tamanho_df_bruto_mb = arquivo_df_bruto.stat().st_size / 1024**2
arquivos_salvos['df_bruto'] = {
    'arquivo': nome_df_bruto,
    'path_completo': str(arquivo_df_bruto),
    'tamanho_mb': round(tamanho_df_bruto_mb, 2),
    'formato': 'parquet',
    'descricao': 'DataFrame original extraído do arquivo fonte'
}
print(f"   ✓ {nome_df_bruto} ({tamanho_df_bruto_mb:.2f} MB)")

# 2. SALVAR df (inicialmente igual a df_bruto, mas pode divergir em blocos futuros)
print(f"\n2. Salvando df...")
nome_df = f"DF_{fm.timestamp}.parquet"
arquivo_df = fm.pastas['dados_processados'] / nome_df
df.to_parquet(arquivo_df, index=False, compression='snappy')
tamanho_df_mb = arquivo_df.stat().st_size / 1024**2
arquivos_salvos['df'] = {
    'arquivo': nome_df,
    'path_completo': str(arquivo_df),
    'tamanho_mb': round(tamanho_df_mb, 2),
    'formato': 'parquet',
    'descricao': 'DataFrame de trabalho (será modificado em blocos posteriores)'
}
print(f"   ✓ {nome_df} ({tamanho_df_mb:.2f} MB)")

# 3. SALVAR METADADOS DAS COLUNAS (para referência rápida)
print(f"\n3. Salvando metadados das colunas...")
metadados_colunas = {
    'colunas': list(df.columns),
    'total': len(df.columns),
    'tipos': {col: str(dtype) for col, dtype in df.dtypes.items()},
    'nulos_por_coluna': df.isnull().sum().to_dict()
}
nome_metadata = f"METADATA_COLUNAS_{fm.timestamp}.json"
arquivo_metadata = fm.pastas['logs'] / nome_metadata
with open(arquivo_metadata, 'w', encoding='utf-8') as f:
    json.dump(metadados_colunas, f, indent=2, ensure_ascii=False)
arquivos_salvos['metadados_colunas'] = {
    'arquivo': nome_metadata,
    'path_completo': str(arquivo_metadata),
    'descricao': 'Metadados das colunas extraídas (nomes, tipos, nulos)'
}
print(f"   ✓ {nome_metadata}")

print(f"\nTotal de arquivos salvos: {len(arquivos_salvos)}")

# ===================================================================
# 5. SALVAR ESTADO COMPLETO COM INSTRUÇÕES DE RECUPERAÇÃO
# ===================================================================

print(f"\n" + "-"*70)
print("SALVANDO ESTADO COMPLETO")
print("-"*70)

estado_bloco9 = {
    'timestamp': datetime.now().isoformat(),
    'bloco': 9,
    'nome': 'EXTRACAO DE DADOS',
    'status': 'concluido',

    # VARIÁVEIS CRIADAS E ONDE ESTÃO SALVAS
    'variaveis_criadas': {
        'df': {
            'tipo': 'DataFrame',
            'arquivo': arquivos_salvos['df']['arquivo'],
            'path_completo': arquivos_salvos['df']['path_completo'],
            'recuperacao': 'df = pd.read_parquet(Path(estado["variaveis_criadas"]["df"]["path_completo"]))'
        },
        'df_bruto': {
            'tipo': 'DataFrame',
            'arquivo': arquivos_salvos['df_bruto']['arquivo'],
            'path_completo': arquivos_salvos['df_bruto']['path_completo'],
            'recuperacao': 'df_bruto = pd.read_parquet(Path(estado["variaveis_criadas"]["df_bruto"]["path_completo"]))'
        },
        'fm': {
            'tipo': 'FileManagerInterativo',
            'recuperacao': 'fm = FileManagerInterativo(Path(log_global["pasta_base"]), log_global["timestamp"])'
        }
    },

    # ARQUIVOS SALVOS
    'arquivos_salvos': arquivos_salvos,

    # ESTATÍSTICAS DOS DADOS
    'estatisticas': {
        'total_registros': len(df_bruto),
        'total_colunas': len(df_bruto.columns),
        'memoria_mb': round(memoria_mb, 2),
        'valores_nulos': int(nulos_total),
        'tamanho_total_mb': round(tamanho_df_bruto_mb + tamanho_df_mb, 2)
    },

    # CONFIGURAÇÃO DA EXTRAÇÃO
    'configuracao_extracao': {
        'arquivo_processado': arquivo_selecionado.name,
        'metodo_carga': metodo_carga,
        'sheet_nome': sheet_nome,
        'linha_cabecalho_inicio': linha_cabecalho_inicio,
        'linha_cabecalho_fim': linha_cabecalho_fim,
        'linha_dados_inicio': linha_dados_inicio,
        'col_inicio': col_inicio,
        'col_fim': col_fim,
        'total_colunas_extraidas': len(df_bruto.columns),
        'indices_python': {
            'idx_cab_inicio': idx_cab_inicio,
            'idx_cab_fim': idx_cab_fim,
            'idx_dados_inicio': idx_dados_inicio,
            'idx_col_inicio': idx_col_inicio,
            'idx_col_fim': idx_col_fim
        }
    },

    # TIPOS DE DADOS
    'tipos_dados': {
        str(tipo): int(count)
        for tipo, count in tipos_count.items()
    },

    # INSTRUÇÕES DE RECUPERAÇÃO
    'instrucoes_recuperacao': {
        'descricao': 'Como recuperar as variaveis em uma nova sessao',
        'passo_1': 'Carregar log_global: with open(Path(".logs/.log_global.json")) as f: log_global = json.load(f)',
        'passo_2': 'Carregar este estado: with open(Path(".logs/.bloco_9_state.json")) as f: estado_b9 = json.load(f)',
        'passo_3': 'Recriar FileManager: fm = FileManagerInterativo(Path(log_global["pasta_base"]), log_global["timestamp"])',
        'passo_4': 'Carregar df: df = pd.read_parquet(Path(estado_b9["variaveis_criadas"]["df"]["path_completo"]))',
        'passo_5': 'Carregar df_bruto: df_bruto = pd.read_parquet(Path(estado_b9["variaveis_criadas"]["df_bruto"]["path_completo"]))',
        'exemplo_completo': '''
# Recuperacao completa do BLOCO 9:
import pandas as pd
import json
from pathlib import Path

# Carregar logs
with open(Path('.logs/.log_global.json'), 'r') as f:
    log_global = json.load(f)
with open(Path('.logs/.bloco_9_state.json'), 'r') as f:
    estado_b9 = json.load(f)

# Recriar FileManager
fm = FileManagerInterativo(
    Path(log_global['pasta_base']),
    log_global['timestamp']
)

# Recuperar DataFrames
df = pd.read_parquet(Path(estado_b9['variaveis_criadas']['df']['path_completo']))
df_bruto = pd.read_parquet(Path(estado_b9['variaveis_criadas']['df_bruto']['path_completo']))

print(f"Recuperado: df com {len(df):,} linhas e df_bruto com {len(df_bruto):,} linhas")
        '''
    }
}

# Salvar estado
arquivo_estado = fm.pastas['logs'] / '.bloco_9_state.json'
with open(arquivo_estado, 'w', encoding='utf-8') as f:
    json.dump(estado_bloco9, f, indent=2, ensure_ascii=False)

print(f"\n✓ Estado salvo: {arquivo_estado.name}")

# ===================================================================
# 6. VALIDAÇÃO FINAL
# ===================================================================

print(f"\n" + "="*70)
print("VALIDACAO FINAL")
print("="*70)

print(f"\nArquivos criados:")
for var_nome, info in arquivos_salvos.items():
    print(f"   • {var_nome.ljust(20)}: {info['arquivo']} ({info.get('tamanho_mb', 'N/A')} MB)")

print(f"\nVariaveis disponiveis:")
print(f"   • df: {len(df):,} registros x {len(df.columns)} colunas")
print(f"   • df_bruto: {len(df_bruto):,} registros x {len(df_bruto.columns)} colunas")
print(f"   • fm: FileManagerInterativo")

print(f"\n✓ Principio '0% memoria, 100% LOG' aplicado:")
print(f"   - Todas as variaveis salvas em arquivo")
print(f"   - Estado completo salvo com instrucoes de recuperacao")
print(f"   - Sistema totalmente recuperavel de arquivos")

print("\n" + "="*70)
print("BLOCO 9 CONCLUIDO")
print("="*70)
print("\nDigite 'BLOCO 9 OK' para prosseguir ao BLOCO 10")
print("="*70)


EXTRACAO DE DADOS
ERRO: Nao foi possivel conectar ao LOG GLOBAL
   Execute o BLOCO 1 primeiro!


FileNotFoundError: [Errno 2] No such file or directory: '.logs\\.log_global.json'

In [19]:
# ===================================================================
# BLOCO 10 - LIMPEZA DE ESTRUTURA (REVISADO v2.1)
# ===================================================================
# MUDANCAS v2.1 (CORRIGIDO):
# - Removida reconexao desnecessaria (usa fm e df_bruto da memoria)
# + Salvamento de estado no LOG (.bloco_10_state.json)
# + Registro de transformacoes em JSON (LOG_Transformacoes_Limpeza.json)
# + Rastreabilidade completa de todas as operacoes
# ===================================================================

print("\n" + "="*70)
print("LIMPEZA DE ESTRUTURA")
print("="*70)

# ===================================================================
# 1. INICIALIZAR (fm e df_bruto ja existem na memoria!)
# ===================================================================

# Validar que variaveis necessarias existem
if 'fm' not in globals():
    print("ERRO: FileManager (fm) nao encontrado!")
    print("   Execute o BLOCO 1 primeiro!")
    raise NameError("fm nao definido")

if 'df_bruto' not in globals():
    print("ERRO: DataFrame bruto (df_bruto) nao encontrado!")
    print("   Execute os BLOCOS 1-9 primeiro!")
    raise NameError("df_bruto nao definido")

print(f"Conectado ao container: {fm.base_path.name}")
print(f"DataFrame bruto carregado: {len(df_bruto):,} registros")

# ===================================================================
# 2. COPIAR df_bruto (necessario para limpeza)
# ===================================================================

df = df_bruto.copy()
log_limpeza = []
transformacoes_detalhadas = {
    'colunas_removidas': [],
    'linhas_removidas': [],
    'colunas_renomeadas': {},
    'linhas_totais_detectadas': [],
    'padroes_aplicados': []
}

print(f"\nIniciando limpeza...")
print(f"   Registros iniciais: {len(df):,}")
print(f"   Colunas iniciais: {len(df.columns)}")

# ===================================================================
# 3. REMOVER COLUNAS COMPLETAMENTE VAZIAS
# ===================================================================

colunas_vazias = df.columns[df.isna().all()].tolist()
if colunas_vazias:
    print(f"\nRemovendo {len(colunas_vazias)} colunas vazias:")
    for col in colunas_vazias[:5]:
        print(f"   * {col}")
    if len(colunas_vazias) > 5:
        print(f"   ... e mais {len(colunas_vazias) - 5}")

    df = df.drop(columns=colunas_vazias)
    log_limpeza.append(f"Removidas {len(colunas_vazias)} colunas vazias")
    transformacoes_detalhadas['colunas_removidas'] = colunas_vazias
    transformacoes_detalhadas['padroes_aplicados'].append({
        'tipo': 'remocao_colunas_vazias',
        'criterio': 'df.isna().all()',
        'quantidade': len(colunas_vazias)
    })

# ===================================================================
# 4. REMOVER LINHAS COMPLETAMENTE VAZIAS
# ===================================================================

linhas_vazias_antes = len(df)
df = df.dropna(how='all')
linhas_vazias = linhas_vazias_antes - len(df)

if linhas_vazias > 0:
    print(f"\nRemovidas {linhas_vazias} linhas completamente vazias")
    log_limpeza.append(f"Removidas {linhas_vazias} linhas vazias")
    transformacoes_detalhadas['padroes_aplicados'].append({
        'tipo': 'remocao_linhas_vazias',
        'criterio': 'df.dropna(how=all)',
        'quantidade': linhas_vazias
    })

# ===================================================================
# 5. LIMPAR NOMES DE COLUNAS
# ===================================================================

print(f"\nLimpando nomes de colunas...")
colunas_antes = df.columns.tolist()
colunas_limpas = []

for col in df.columns:
    # Limpar
    col_limpo = str(col).strip()
    col_limpo = col_limpo.lstrip("'\"")
    col_limpo = col_limpo.replace('\n', ' ').replace('\r', '')
    col_limpo = ' '.join(col_limpo.split())

    # Remover espacos extras
    col_limpo = re.sub(r'\s+', ' ', col_limpo)

    colunas_limpas.append(col_limpo)

# Contar modificacoes
modificados = sum(1 for orig, limpo in zip(colunas_antes, colunas_limpas)
                  if orig != limpo)
if modificados > 0:
    print(f"   {modificados} nomes limpos")
    log_limpeza.append(f"Limpeza de nomes: {modificados} colunas")
    transformacoes_detalhadas['padroes_aplicados'].append({
        'tipo': 'limpeza_nomes',
        'criterio': 'strip + lstrip + regex',
        'quantidade': modificados
    })

df.columns = colunas_limpas

# ===================================================================
# 6. RENOMEAR COLUNAS DUPLICADAS
# ===================================================================

contagem = Counter(colunas_limpas)
duplicadas = {c: n for c, n in contagem.items() if n > 1}

if duplicadas:
    print(f"\nRenomeando {len(duplicadas)} colunas duplicadas:")
    colunas_finais = []
    contador = {}

    for col in colunas_limpas:
        if col in duplicadas:
            if col not in contador:
                contador[col] = 0
                colunas_finais.append(col)
            else:
                contador[col] += 1
                novo_nome = f"{col}_dup{contador[col]}"
                colunas_finais.append(novo_nome)
                print(f"   '{col}' -> '{novo_nome}'")
                transformacoes_detalhadas['colunas_renomeadas'][col] = novo_nome
        else:
            colunas_finais.append(col)

    df.columns = colunas_finais
    log_limpeza.append(f"Renomeadas {sum(contador.values())} colunas duplicadas")
    transformacoes_detalhadas['padroes_aplicados'].append({
        'tipo': 'renomear_duplicadas',
        'criterio': 'Counter + sufixo _dupN',
        'quantidade': sum(contador.values())
    })

# ===================================================================
# 7. REMOVER LINHAS DE TOTAIS/RESULTADOS
# ===================================================================

padroes_remover = [
    r'(?i)^total',
    r'(?i)^resultado',
    r'(?i)^soma',
    r'(?i)^subtotal',
    r'(?i)^grand total',
    r'(?i)^media',
    r'(?i)^average'
]

linhas_remover = []
for idx, row in df.iterrows():
    primeira_celula = str(row.iloc[0]).strip().lower()
    if any(re.search(p, primeira_celula) for p in padroes_remover):
        linhas_remover.append(idx)
        transformacoes_detalhadas['linhas_totais_detectadas'].append({
            'indice': int(idx),
            'primeira_celula': str(row.iloc[0]).strip()
        })

if linhas_remover:
    print(f"\nRemovendo {len(linhas_remover)} linhas de totais/resultados")
    df = df.drop(index=linhas_remover)
    log_limpeza.append(f"Removidas {len(linhas_remover)} linhas de totais")
    transformacoes_detalhadas['padroes_aplicados'].append({
        'tipo': 'remocao_totais',
        'criterio': '7 padroes regex',
        'quantidade': len(linhas_remover)
    })

# ===================================================================
# 8. RESET INDEX
# ===================================================================

df = df.reset_index(drop=True)

# ===================================================================
# 9. CRIAR COPIA LIMPA
# ===================================================================

df_limpo = df.copy()

# ===================================================================
# 10. RESUMO
# ===================================================================

print(f"\n" + "="*70)
print(f"LIMPEZA CONCLUIDA")
print(f"="*70)
print(f"\nAntes -> Depois:")
print(f"   Registros: {len(df_bruto):,} -> {len(df_limpo):,}")
print(f"   Colunas: {len(df_bruto.columns)} -> {len(df_limpo.columns)}")

if log_limpeza:
    print(f"\nOperacoes realizadas:")
    for i, op in enumerate(log_limpeza, 1):
        print(f"   {i}. {op}")
else:
    print(f"\nNenhuma limpeza necessaria - dados ja estavam limpos!")

print(f"\nPreview dos dados limpos:")
print("-" * 70)
display(df_limpo.head(3))
print("-" * 70)

# ===================================================================
# 11. SALVAR ESTADO NO LOG (NOVO!)
# ===================================================================

estado_bloco10 = {
    'timestamp': datetime.now().isoformat(),
    'bloco': 10,
    'nome': 'LIMPEZA DE ESTRUTURA',
    'status': 'concluido',
    'variaveis_criadas': ['df_limpo', 'log_limpeza'],
    'estatisticas': {
        'registros_antes': len(df_bruto),
        'registros_depois': len(df_limpo),
        'colunas_antes': len(df_bruto.columns),
        'colunas_depois': len(df_limpo.columns),
        'colunas_removidas': len(colunas_vazias) if colunas_vazias else 0,
        'linhas_removidas_vazias': linhas_vazias,
        'linhas_removidas_totais': len(linhas_remover) if linhas_remover else 0,
        'colunas_renomeadas': sum(contador.values()) if duplicadas else 0
    },
    'transformacoes': transformacoes_detalhadas,
    'log_resumido': log_limpeza
}

# Salvar estado
arquivo_estado = fm.pastas['logs'] / '.bloco_10_state.json'
with open(arquivo_estado, 'w', encoding='utf-8') as f:
    json.dump(estado_bloco10, f, indent=2, ensure_ascii=False)

print(f"\nEstado salvo:")
print(f"   {arquivo_estado.name}")

# Salvar transformacoes detalhadas (para rastreabilidade)
arquivo_transformacoes = fm.pastas['logs'] / 'LOG_Transformacoes_Limpeza.json'
with open(arquivo_transformacoes, 'w', encoding='utf-8') as f:
    json.dump(transformacoes_detalhadas, f, indent=2, ensure_ascii=False)

print(f"   {arquivo_transformacoes.name}")

print("\n" + "="*70)
print("BLOCO 10 CONCLUIDO")
print("="*70)
print("\nDigite 'BLOCO 10 OK' para prosseguir ao BLOCO 11")
print("="*70)


LIMPEZA DE ESTRUTURA
ERRO: DataFrame bruto (df_bruto) nao encontrado!
   Execute os BLOCOS 1-9 primeiro!


NameError: df_bruto nao definido

In [ ]:
# ═══════════════════════════════════════════════════════════════════
# TRATAMENTO DE FORMATO BW/BEx - FORWARD FILL
# Baseado em: ETL - ESTOQUE E MOVIMENTAÇÃO (SAP BEx).ipynb - PASSO 4
# Posição: APÓS Limpeza de Estrutura, ANTES de Detecção de Campos
# ═══════════════════════════════════════════════════════════════════

print("\n" + "="*70)
print("🔄 TRATAMENTO DE FORMATO BW/BEx (FORWARD FILL)")
print("="*70)

# ═══════════════════════════════════════════════════════════════════
# ETAPA 1: DETECTAR SE É FORMATO BW
# ═══════════════════════════════════════════════════════════════════

print("""
ℹ️  CONTEXTO:
   Arquivos do SAP Business Warehouse (BW/BEx) usam formatação hierárquica:
   - Dimensões aparecem apenas na PRIMEIRA linha de cada agrupamento
   - Linhas seguintes ficam VAZIAS até mudar o agrupamento

   Este bloco detecta automaticamente e aplica forward fill se necessário.
""")

print("\n" + "─"*70)
print("🔍 ETAPA 1: DETECTANDO FORMATO BW")
print("─"*70)

# Analisar primeiras colunas (geralmente dimensões em BW)
colunas_iniciais = df.columns[:min(6, len(df.columns))]

print(f"\n📊 Analisando {len(colunas_iniciais)} primeiras colunas:")

analise_colunas = []

for col in colunas_iniciais:
    # Contar valores não-nulos
    valores_nao_nulos = df[col].notna()
    count_nao_nulos = valores_nao_nulos.sum()

    # Contar valores únicos (ignorando NaN)
    valores_unicos = df[col].dropna().nunique()

    # Percentual de NaN
    pct_nulos = (len(df) - count_nao_nulos) / len(df) * 100

    # Armazenar análise
    analise_colunas.append({
        'coluna': col,
        'count_nao_nulos': count_nao_nulos,
        'valores_unicos': valores_unicos,
        'pct_nulos': pct_nulos,
        'eh_dimensao_bw': pct_nulos > 30 and valores_unicos < 1000
    })

    emoji = "🔄" if pct_nulos > 30 and valores_unicos < 1000 else "  "
    print(f"   {emoji} {col[:35].ljust(35)} | Únicos: {valores_unicos:>5} | NaN: {pct_nulos:>5.1f}%")

# Contar quantas colunas parecem ser BW
colunas_bw = [a for a in analise_colunas if a['eh_dimensao_bw']]

print(f"\n📊 RESULTADO DA DETECÇÃO:")
print(f"   Colunas com padrão BW: {len(colunas_bw)}/{len(colunas_iniciais)}")

eh_formato_bw = len(colunas_bw) >= 2  # Pelo menos 2 colunas com padrão BW

if eh_formato_bw:
    print(f"   ✅ FORMATO BW DETECTADO - Forward fill será aplicado")
else:
    print(f"   ℹ️  Formato padrão - Forward fill NÃO necessário")

# ═══════════════════════════════════════════════════════════════════
# ETAPA 2: APLICAR FFILL (SE DETECTADO)
# ═══════════════════════════════════════════════════════════════════

if eh_formato_bw:
    print("\n" + "─"*70)
    print("🔄 ETAPA 2: APLICANDO FORWARD FILL")
    print("─"*70)

    # Listar colunas que receberão ffill
    colunas_para_ffill = [a['coluna'] for a in analise_colunas if a['eh_dimensao_bw']]

    print(f"\n📋 Colunas que receberão forward fill ({len(colunas_para_ffill)}):")
    for i, col in enumerate(colunas_para_ffill, 1):
        print(f"   {i}. {col}")

    # Contar NULLs ANTES do ffill
    print(f"\n📊 Contagem de NULLs ANTES do ffill:")
    nulls_antes = {}
    total_nulls_antes = 0

    for col in colunas_para_ffill:
        count = int(df[col].isnull().sum())
        nulls_antes[col] = count
        total_nulls_antes += count
        print(f"   {col[:40].ljust(40)}: {count:>8,} NULLs")

    print(f"   {'TOTAL'.ljust(40)}: {total_nulls_antes:>8,} NULLs")

    # APLICAR FFILL
    print(f"\n🔄 Aplicando forward fill...")
    df[colunas_para_ffill] = df[colunas_para_ffill].ffill()

    # Contar NULLs DEPOIS do ffill
    print(f"\n✅ Contagem de NULLs DEPOIS do ffill:")
    total_nulls_depois = 0

    for col in colunas_para_ffill:
        nulls_depois = int(df[col].isnull().sum())
        total_nulls_depois += nulls_depois

        # Calcular preenchidas
        preenchidas = nulls_antes[col] - nulls_depois

        print(f"   {col[:40].ljust(40)}: {nulls_antes[col]:>8,} → {nulls_depois:>8,} (Δ {preenchidas:>7,})")

    total_preenchidas = total_nulls_antes - total_nulls_depois
    print(f"   {'TOTAL'.ljust(40)}: {total_nulls_antes:>8,} → {total_nulls_depois:>8,} (Δ {total_preenchidas:>7,})")

    # ═══════════════════════════════════════════════════════════════
    # ETAPA 3: VALIDAR RESULTADO
    # ═══════════════════════════════════════════════════════════════

    print("\n" + "─"*70)
    print("✅ ETAPA 3: VALIDAÇÃO DO RESULTADO")
    print("─"*70)

    print(f"\n👁️  COMPARAÇÃO (primeiras 10 linhas):")

    # Criar DataFrame comparativo
    print(f"\n   ANTES do ffill:")
    # Recarregar apenas para mostrar (não aplicar)
    if metodo_carga == 'csv':
        df_antes = pd.read_csv(
            arquivo_selecionado,
            sep=separador_detectado,
            encoding='cp1252',
            skiprows=skiprows_csv if 'skiprows_csv' in locals() else None,
            header=idx_cab_inicio,
            usecols=range(idx_col_inicio, idx_col_fim)
        )
        if idx_dados_inicio > idx_cab_inicio + 1:
            df_antes = df_antes.iloc[idx_dados_inicio - idx_cab_inicio - 1:]
    else:
        df_antes = pd.read_excel(
            arquivo_selecionado,
            sheet_name=sheet_nome if 'sheet_nome' in locals() else 0,
            header=idx_cab_inicio,
            usecols=range(idx_col_inicio, idx_col_fim)
        )
        if idx_dados_inicio > idx_cab_inicio + 1:
            df_antes = df_antes.iloc[idx_dados_inicio - idx_cab_inicio - 1:]

    print(df_antes[colunas_para_ffill].head(10).to_string())

    print(f"\n   DEPOIS do ffill:")
    print(df[colunas_para_ffill].head(10).to_string())

    # Verificar se primeira linha tem valores
    primeira_linha_nulos = df.iloc[0][colunas_para_ffill].isnull().sum()

    if primeira_linha_nulos > 0:
        print(f"\n⚠️  ATENÇÃO: Primeira linha ainda tem {primeira_linha_nulos} NaN(s)!")
        print(f"   Isso pode indicar problema no cabeçalho ou dados")
        print(f"   Recomenda-se revisar a extração de dados")
    else:
        print(f"\n✅ Primeira linha OK - Todos os valores preenchidos")

    print("\n" + "="*70)
    print("✅ FORWARD FILL APLICADO COM SUCESSO")
    print("="*70)

    print(f"\n📊 RESUMO:")
    print(f"   Colunas processadas: {len(colunas_para_ffill)}")
    print(f"   Células preenchidas: {total_preenchidas:,}")
    print(f"   Redução de NaN: {(total_preenchidas / total_nulls_antes * 100):.1f}%")

else:
    print("\n" + "="*70)
    print("ℹ️  FORWARD FILL NÃO NECESSÁRIO")
    print("="*70)
    print(f"\n   Arquivo não possui formatação BW/BEx hierárquica")
    print(f"   Prosseguindo para próxima etapa...")

print("\n" + "="*70)
print("✅ TRATAMENTO BW CONCLUÍDO")
print("="*70)

In [ ]:
# ═══════════════════════════════════════════════════════════════════
# REMOÇÃO DE LINHAS DE TOTALIZAÇÃO (ARQUIVOS BW/BEx)
# ═══════════════════════════════════════════════════════════════════

print("\n" + "="*70)
print("🗑️  REMOÇÃO DE LINHAS DE TOTALIZAÇÃO")
print("="*70)

print("\nℹ️  CONTEXTO:")
print("   Arquivos BW/BEx frequentemente contêm linhas de:")
print("   - Totais gerais")
print("   - Subtotais por agrupamento")
print("   - Resultados parciais")
print("   - Médias agregadas")
print("   Estas linhas INFLAM valores e devem ser removidas.\n")

# ───────────────────────────────────────────────────────────────────
# ETAPA 1: DETECTAR LINHAS DE TOTALIZAÇÃO
# ───────────────────────────────────────────────────────────────────

print("─" * 80)
print("🔍 ETAPA 1: DETECTANDO LINHAS DE TOTALIZAÇÃO")
print("─" * 80)

# Padrões para identificar totalizações
padroes_totalizacao = [
    # Português
    r'(?i)^total\b',
    r'(?i)^resultado\b',
    r'(?i)^soma\b',
    r'(?i)^subtotal\b',
    r'(?i)^parcial\b',
    r'(?i)^grand total',
    r'(?i)^média\b',
    r'(?i)^media\b',
    r'(?i)^consolidado\b',
    r'(?i)^geral\b',
    r'(?i)total geral',
    r'(?i)resultado final',

    # Inglês (comum em exports SAP)
    r'(?i)^overall',
    r'(?i)^average',
    r'(?i)^sum\b',
    r'(?i)^total\s',
    r'(?i)^result\b',

    # Padrões numéricos (ex: "Total 5262", "Resultado 1234")
    r'(?i)^total\s+\d+',
    r'(?i)^resultado\s+\d+',
    r'(?i)^subtotal\s+\d+',
]

# Compilar regex
padroes_compilados = [re.compile(p) for p in padroes_totalizacao]

# Função para verificar se linha é totalização
def eh_linha_totalizacao(row):
    """Verifica se linha é de totalização."""
    # Verificar primeira coluna (mais comum)
    primeira_celula = str(row.iloc[0]).strip()

    if any(padrao.search(primeira_celula) for padrao in padroes_compilados):
        return True

    # Verificar segunda coluna (caso primeira esteja preenchida com código)
    if len(row) > 1:
        segunda_celula = str(row.iloc[1]).strip()
        if any(padrao.search(segunda_celula) for padrao in padroes_compilados):
            return True

    # Verificar se TODAS as colunas categóricas estão vazias
    # (indicativo de linha de totalização em hierarquia BW)
    colunas_categoricas = df.select_dtypes(include=['object']).columns
    if len(colunas_categoricas) > 0:
        valores_cat = row[colunas_categoricas].dropna()
        # Se menos de 30% preenchidos E primeira célula sugere total
        if len(valores_cat) < len(colunas_categoricas) * 0.3:
            primeira_palavra = primeira_celula.lower().split()[0] if primeira_celula else ''
            if primeira_palavra in ['total', 'resultado', 'soma', 'média', 'media', 'geral']:
                return True

    return False

# Identificar linhas a remover
print(f"\n📊 Analisando {len(df):,} linhas...")

linhas_totalizacao = []
for idx, row in df.iterrows():
    if eh_linha_totalizacao(row):
        linhas_totalizacao.append(idx)

print(f"\n🔍 Linhas de totalização detectadas: {len(linhas_totalizacao)}")

if len(linhas_totalizacao) > 0:
    # Mostrar exemplos
    print(f"\n📋 Exemplos de linhas detectadas (primeiras 5):")
    for i, idx in enumerate(linhas_totalizacao[:5], 1):
        primeira_col = df.iloc[idx, 0]
        segunda_col = df.iloc[idx, 1] if len(df.columns) > 1 else 'N/A'
        print(f"   {i}. Linha {idx}: '{primeira_col}' | '{segunda_col}'")

    if len(linhas_totalizacao) > 5:
        print(f"   ... e mais {len(linhas_totalizacao) - 5}")

# ───────────────────────────────────────────────────────────────────
# ETAPA 2: REMOVER LINHAS DETECTADAS
# ───────────────────────────────────────────────────────────────────

if len(linhas_totalizacao) > 0:
    print("\n" + "─" * 80)
    print("🗑️  ETAPA 2: REMOVENDO LINHAS DE TOTALIZAÇÃO")
    print("─" * 80)

    # Backup
    registros_antes = len(df)

    # Remover
    df_sem_totais = df.drop(index=linhas_totalizacao)

    # Reset index
    df_sem_totais = df_sem_totais.reset_index(drop=True)

    registros_depois = len(df_sem_totais)
    removidos = registros_antes - registros_depois
    pct_removido = (removidos / registros_antes) * 100

    print(f"\n✅ Remoção concluída:")
    print(f"   Antes:    {registros_antes:>7,} linhas")
    print(f"   Removido: {removidos:>7,} linhas ({pct_removido:.1f}%)")
    print(f"   Depois:   {registros_depois:>7,} linhas")

    # Validação: verificar se não removemos demais
    if pct_removido > 50:
        print(f"\n⚠️  ALERTA: Mais de 50% das linhas foram removidas!")
        print(f"   Isso pode indicar falso positivo na detecção.")
        print(f"   Recomenda-se revisar os padrões ou validar manualmente.")

        resposta = input("\n   Deseja MANTER a remoção? (S/N): ").strip().upper()
        if resposta != 'S':
            print(f"\n   ℹ️  Remoção CANCELADA - mantendo dados originais")
            df_sem_totais = df.copy()

    # Atualizar DataFrame principal
    df = df_sem_totais

else:
    print(f"\n✅ Nenhuma linha de totalização detectada - dados já estão limpos!")

# ───────────────────────────────────────────────────────────────────
# ETAPA 3: VALIDAÇÃO FINAL
# ───────────────────────────────────────────────────────────────────

print("\n" + "─" * 80)
print("✅ ETAPA 3: VALIDAÇÃO FINAL")
print("─" * 80)

print(f"\n📊 Dataset final:")
print(f"   Registros: {len(df):,}")
print(f"   Colunas: {len(df.columns)}")

print(f"\n👁️  Preview (primeiras 3 linhas):")
print("─" * 80)
print(df.head(3).to_string())
print("─" * 80)

print("\n" + "="*70)
print("✅ REMOÇÃO DE TOTALIZAÇÕES CONCLUÍDA")
print("="*70)

In [ ]:
# ═══════════════════════════════════════════════════════════════════
# DETECÇÃO AUTOMÁTICA DE CAMPOS (COM DICIONÁRIO PERSISTENTE) -> pedir para detectar nomes de campos unnamed com base no conteudo dos registros, exemplo unnamed com siglas , unnamed com codigos de material, unnamed com siglas, etc e tb remover linhas de totalizacoes, resultados, totais, parciais, em qualquer nivel hierarquivco de agregacao ou granularidade, pois costumam aumentar valores em colunas que tem registros linha a linha (apenas para arquivos tipo bw com hierarquias nao repetidas conforme acima)
# ═══════════════════════════════════════════════════════════════════

print("\n" + "="*70)
print("🔍 DETECÇÃO AUTOMÁTICA DE CAMPOS")
print("="*70)

# ═══════════════════════════════════════════════════════════════════
# CLASSE DETECTOR DE CAMPOS
# ═══════════════════════════════════════════════════════════════════

class DetectorCampos:
    """Detecta e mapeia campos automaticamente."""

    def __init__(self, df, dicionario_persistente):
        self.df = df
        self.dicionario = dicionario_persistente
        self.mapeamento = {}
        self.relatorio = {
            'total_campos': len(df.columns),
            'mapeados_dicionario': 0,
            'mapeados_heuristica': 0,
            'desconhecidos': 0,
            'ambiguos': 0,
            'detalhes': []
        }

    def detectar_todos(self):
        """Detecta todos os campos do DataFrame."""
        print(f"\n📊 Analisando {len(self.df.columns)} campos...")

        for col in self.df.columns:
            print(f"\n   🔍 Analisando: '{col}'")

            # Extrair amostra de valores
            valores_amostra = self.df[col].dropna().head(100).astype(str).tolist()

            # Tentar detecção
            resultado = self._detectar_campo(col, valores_amostra)

            # Armazenar
            self.mapeamento[col] = resultado

            # Atualizar relatório
            if resultado['metodo'] == 'DICIONARIO':
                self.relatorio['mapeados_dicionario'] += 1
                emoji = "✅"
            elif resultado['metodo'] == 'HEURISTICA':
                self.relatorio['mapeados_heuristica'] += 1
                emoji = "🤖"
            elif resultado['campo_detectado'] == 'DESCONHECIDO':
                self.relatorio['desconhecidos'] += 1
                emoji = "❓"
            else:
                emoji = "⚠️"

            if resultado.get('ambiguidade', False):
                self.relatorio['ambiguos'] += 1
                emoji += "⚠️"

            # Exibir resultado
            print(f"      {emoji} {resultado['campo_detectado']}")
            print(f"      Confiança: {resultado['confianca']:.0%} | Método: {resultado['metodo']}")

            # Adicionar ao relatório
            self.relatorio['detalhes'].append({
                'coluna_original': col,
                'campo_detectado': resultado['campo_detectado'],
                'confianca': resultado['confianca'],
                'metodo': resultado['metodo'],
                'ambiguidade': resultado.get('ambiguidade', False)
            })

        return self.mapeamento, self.relatorio

    def _detectar_campo(self, nome_coluna, valores):
        """Detecta um único campo."""

        # ESTRATÉGIA 1: Buscar no dicionário persistente (MÁXIMA PRIORIDADE)
        if self.dicionario and 'arquivos' in self.dicionario:
            for arquivo_info in self.dicionario.get('arquivos', {}).values():
                if 'campos_mapeados' in arquivo_info:
                    for campo_orig, campo_info in arquivo_info['campos_mapeados'].items():
                        # Match exato no nome
                        if nome_coluna.lower() == campo_orig.lower():
                            return {
                                'campo_detectado': campo_info.get('nome_padrao', campo_orig),
                                'confianca': 1.0,
                                'metodo': 'DICIONARIO',
                                'ambiguidade': False
                            }

                        # Match parcial
                        if nome_coluna.lower() in campo_orig.lower() or campo_orig.lower() in nome_coluna.lower():
                            return {
                                'campo_detectado': campo_info.get('nome_padrao', campo_orig),
                                'confianca': 0.85,
                                'metodo': 'DICIONARIO',
                                'ambiguidade': False
                            }

        # ESTRATÉGIA 2: Heurísticas (se não achou no dicionário)
        resultado_heuristica = self._heuristica_simples(nome_coluna, valores)

        if resultado_heuristica:
            return resultado_heuristica

        # ESTRATÉGIA 3: Desconhecido
        return {
            'campo_detectado': 'DESCONHECIDO',
            'confianca': 0.0,
            'metodo': 'NENHUM',
            'ambiguidade': False
        }

    def _heuristica_simples(self, nome, valores):
        """Heurísticas avançadas por padrões de conteúdo."""

        if not valores:
            return None

        # Remover valores vazios
        valores = [v for v in valores if v and str(v).strip().lower() not in ['nan', 'none', '']]

        if not valores:
            return None

        # ═══════════════════════════════════════════════════════════
        # HEURÍSTICAS PARA CAMPOS UNNAMED (PRIORIDADE MÁXIMA)
        # ═══════════════════════════════════════════════════════════

        if 'unnamed' in nome.lower():
            print(f"      🔍 Analisando conteúdo de '{nome}'...")

            # Amostra de valores para análise
            amostra = valores[:20]

            # U1: Códigos de Centro (4-5 dígitos numéricos)
            if all(str(v).isdigit() and 3 <= len(str(v)) <= 5 for v in amostra):
                return {
                    'campo_detectado': 'Código Centro',
                    'confianca': 0.90,
                    'metodo': 'HEURISTICA_UNNAMED',
                    'ambiguidade': False,
                    'justificativa': 'Códigos numéricos de 3-5 dígitos'
                }

            # U2: Siglas de Base (2-6 letras maiúsculas)
            if all(str(v).isalpha() and str(v).isupper() and 2 <= len(str(v)) <= 6 for v in amostra):
                return {
                    'campo_detectado': 'Sigla Base',
                    'confianca': 0.90,
                    'metodo': 'HEURISTICA_UNNAMED',
                    'ambiguidade': False,
                    'justificativa': 'Siglas em letras maiúsculas'
                }

            # U3: Códigos de Produto (formato: NN.NNN.NNN)
            padrao_produto = r'^\d{2}\.\d{3}\.\d{3}$'
            if all(re.match(padrao_produto, str(v)) for v in amostra):
                return {
                    'campo_detectado': 'Código Produto',
                    'confianca': 0.95,
                    'metodo': 'HEURISTICA_UNNAMED',
                    'ambiguidade': False,
                    'justificativa': 'Formato de código de produto (XX.XXX.XXX)'
                }

            # U4: Códigos Alfanuméricos (possível código de emissor)
            if all(bool(re.search(r'\d', str(v))) and len(str(v)) >= 4 for v in amostra):
                # Verificar se tem letras também
                tem_letras = any(c.isalpha() for v in amostra for c in str(v))
                if tem_letras:
                    return {
                        'campo_detectado': 'Código Emissor',
                        'confianca': 0.80,
                        'metodo': 'HEURISTICA_UNNAMED',
                        'ambiguidade': False,
                        'justificativa': 'Códigos alfanuméricos'
                    }
                else:
                    return {
                        'campo_detectado': 'Código Identificador',
                        'confianca': 0.75,
                        'metodo': 'HEURISTICA_UNNAMED',
                        'ambiguidade': False,
                        'justificativa': 'Códigos numéricos longos'
                    }

            # U5: Nomes/Descrições (strings longas com espaços)
            tem_espacos = any(' ' in str(v) for v in amostra)
            tamanho_medio = sum(len(str(v)) for v in amostra) / len(amostra)

            if tem_espacos and tamanho_medio > 10:
                # Verificar se é nome de empresa
                palavras_empresa = ['ltda', 'sa', 's.a', 's/a', 'cia', 'company', 'corporation', 'industria', 'comercio']
                eh_empresa = any(any(palavra in str(v).lower() for palavra in palavras_empresa) for v in amostra[:5])

                if eh_empresa:
                    return {
                        'campo_detectado': 'Razão Social',
                        'confianca': 0.85,
                        'metodo': 'HEURISTICA_UNNAMED',
                        'ambiguidade': False,
                        'justificativa': 'Nomes de empresas detectados'
                    }
                else:
                    return {
                        'campo_detectado': 'Descrição',
                        'confianca': 0.75,
                        'metodo': 'HEURISTICA_UNNAMED',
                        'ambiguidade': False,
                        'justificativa': 'Textos descritivos longos'
                    }

            # U6: Códigos curtos (2-3 caracteres)
            if all(2 <= len(str(v)) <= 3 for v in amostra):
                if all(str(v).isalpha() for v in amostra):
                    return {
                        'campo_detectado': 'Código Curto (Categoria)',
                        'confianca': 0.70,
                        'metodo': 'HEURISTICA_UNNAMED',
                        'ambiguidade': True,
                        'justificativa': 'Códigos alfabéticos curtos'
                    }

            # U7: Estados (2 letras maiúsculas) - comum em BW Brasil
            estados_br = ['AC','AL','AP','AM','BA','CE','DF','ES','GO','MA','MT','MS','MG',
                         'PA','PB','PR','PE','PI','RJ','RN','RS','RO','RR','SC','SP','SE','TO']
            if all(str(v).upper() in estados_br for v in amostra[:10]):
                return {
                    'campo_detectado': 'Estado (UF)',
                    'confianca': 0.95,
                    'metodo': 'HEURISTICA_UNNAMED',
                    'ambiguidade': False,
                    'justificativa': 'Siglas de estados brasileiros'
                }

            # Se chegou aqui e é Unnamed, marcar como desconhecido mas com contexto
            print(f"      ⚠️  Padrão não reconhecido em '{nome}'")
            print(f"      Exemplos: {amostra[:3]}")

        # ═══════════════════════════════════════════════════════════
        # HEURÍSTICAS GERAIS (PARA CAMPOS NÃO-UNNAMED)
        # ═══════════════════════════════════════════════════════════

        # H1: Data no formato YYYY-MM-DD ou DD/MM/YYYY
        padrao_data1 = r'^\d{4}-\d{2}-\d{2}$'
        padrao_data2 = r'^\d{2}/\d{2}/\d{4}$'
        padrao_data3 = r'^\d{1,2}\.\d{4}$'  # M.YYYY ou MM.YYYY

        if all(re.match(padrao_data1, str(v)) or re.match(padrao_data2, str(v)) for v in valores[:10]):
            return {
                'campo_detectado': 'Data',
                'confianca': 0.95,
                'metodo': 'HEURISTICA',
                'ambiguidade': False
            }

        # Formato "M.YYYY" comum em exports BW
        if all(re.match(padrao_data3, str(v)) for v in valores[:10]):
            return {
                'campo_detectado': 'Mês de Calendário',
                'confianca': 0.90,
                'metodo': 'HEURISTICA',
                'ambiguidade': False
            }

        # H2: Números de 4 dígitos (provável código de centro)
        if all(re.match(r'^\d{4}$', str(v)) for v in valores[:10]):
            if 'centro' in nome.lower() or 'cod' in nome.lower() or 'base' in nome.lower():
                return {
                    'campo_detectado': 'Código Centro',
                    'confianca': 0.85,
                    'metodo': 'HEURISTICA',
                    'ambiguidade': False
                }

        # H3: Siglas de 5 letras (formato clássico de bases BR)
        if all(len(str(v)) == 5 and str(v).isalpha() and str(v).isupper() for v in valores[:10]):
            return {
                'campo_detectado': 'Sigla Base',
                'confianca': 0.85,
                'metodo': 'HEURISTICA',
                'ambiguidade': False
            }

        # H4: Percentuais
        if 'porcent' in nome.lower() or '%' in nome or 'perc' in nome.lower():
            return {
                'campo_detectado': 'Percentual',
                'confianca': 0.80,
                'metodo': 'HEURISTICA',
                'ambiguidade': False
            }

        # H5: Valores numéricos grandes (volume/estoque)
        try:
            valores_num = []
            for v in valores[:10]:
                v_str = str(v).replace(',', '.').replace(' ', '')
                valores_num.append(float(v_str))

            if all(v > 1000 for v in valores_num):
                # Identificar tipo por nome ou contexto
                if 'volume' in nome.lower() or 'expedicao' in nome.lower() or 'expedição' in nome.lower():
                    return {
                        'campo_detectado': 'Volume Expedição',
                        'confianca': 0.85,
                        'metodo': 'HEURISTICA',
                        'ambiguidade': False
                    }
                elif 'estoque' in nome.lower() or 'saldo' in nome.lower():
                    return {
                        'campo_detectado': 'Estoque',
                        'confianca': 0.85,
                        'metodo': 'HEURISTICA',
                        'ambiguidade': False
                    }
                elif 'consigna' in nome.lower():
                    return {
                        'campo_detectado': 'Estoque Consignado',
                        'confianca': 0.90,
                        'metodo': 'HEURISTICA',
                        'ambiguidade': False
                    }
                else:
                    return {
                        'campo_detectado': 'Quantidade',
                        'confianca': 0.70,
                        'metodo': 'HEURISTICA',
                        'ambiguidade': True
                    }
        except:
            pass

        # H6: Códigos de Produto (por nome + formato)
        if 'produto' in nome.lower() or 'material' in nome.lower():
            # Verificar se tem formato de código
            if all(any(c.isdigit() for c in str(v)) for v in valores[:10]):
                return {
                    'campo_detectado': 'Código Produto',
                    'confianca': 0.80,
                    'metodo': 'HEURISTICA',
                    'ambiguidade': False
                }

        # H7: Nomes de Produtos/Materiais (por nome + conteúdo textual)
        if 'produto' in nome.lower() or 'material' in nome.lower() or 'descri' in nome.lower():
            tem_texto = any(len(str(v)) > 10 and any(c.isalpha() for c in str(v)) for v in valores[:5])
            if tem_texto:
                return {
                    'campo_detectado': 'Descrição Produto',
                    'confianca': 0.80,
                    'metodo': 'HEURISTICA',
                    'ambiguidade': False
                }

        return None

# ═══════════════════════════════════════════════════════════════════
# EXECUTAR DETECÇÃO
# ═══════════════════════════════════════════════════════════════════

detector = DetectorCampos(df, DICIONARIO_PERSISTENTE)
mapeamento_campos, relatorio_deteccao = detector.detectar_todos()

# ═══════════════════════════════════════════════════════════════════
# RELATÓRIO DE DETECÇÃO
# ═══════════════════════════════════════════════════════════════════

print("\n" + "="*70)
print("📋 RELATÓRIO DE DETECÇÃO")
print("="*70)

print(f"\n📊 RESUMO:")
print(f"   Total de campos: {relatorio_deteccao['total_campos']}")
print(f"   ✅ Dicionário: {relatorio_deteccao['mapeados_dicionario']}")
print(f"   🤖 Heurística: {relatorio_deteccao['mapeados_heuristica']}")
print(f"   ❓ Desconhecidos: {relatorio_deteccao['desconhecidos']}")
print(f"   ⚠️  Ambíguos: {relatorio_deteccao['ambiguos']}")

# Taxa de sucesso
taxa_sucesso = ((relatorio_deteccao['mapeados_dicionario'] + relatorio_deteccao['mapeados_heuristica']) /
                relatorio_deteccao['total_campos']) * 100

print(f"\n🎯 Taxa de detecção: {taxa_sucesso:.1f}%")

# Detalhes por confiança
print(f"\n📊 DISTRIBUIÇÃO POR CONFIANÇA:")

alta = sum(1 for d in relatorio_deteccao['detalhes'] if d['confianca'] >= 0.85)
media = sum(1 for d in relatorio_deteccao['detalhes'] if 0.70 <= d['confianca'] < 0.85)
baixa = sum(1 for d in relatorio_deteccao['detalhes'] if 0 < d['confianca'] < 0.70)
zero = sum(1 for d in relatorio_deteccao['detalhes'] if d['confianca'] == 0)

print(f"   🟢 Alta (≥85%): {alta}")
print(f"   🟡 Média (70-85%): {media}")
print(f"   🟠 Baixa (<70%): {baixa}")
print(f"   ⚫ Desconhecidos: {zero}")

# Lista de desconhecidos
desconhecidos = [d for d in relatorio_deteccao['detalhes'] if d['campo_detectado'] == 'DESCONHECIDO']

if desconhecidos:
    print(f"\n❓ CAMPOS DESCONHECIDOS:")
    for item in desconhecidos:
        print(f"   • {item['coluna_original']}")

# ═══════════════════════════════════════════════════════════════════
# APLICAR MAPEAMENTO (CRIAR COLUNAS PADRONIZADAS)
# ═══════════════════════════════════════════════════════════════════

print("\n" + "─"*70)
print("🔄 APLICANDO MAPEAMENTO")
print("─"*70)

# Criar dicionário de rename (apenas campos com confiança ≥70%)
rename_dict = {}
for col_orig, info in mapeamento_campos.items():
    if info['confianca'] >= 0.70 and info['campo_detectado'] != 'DESCONHECIDO':
        # Se nome diferente, mapear
        if col_orig != info['campo_detectado']:
            rename_dict[col_orig] = info['campo_detectado']

print(f"\n📝 Mapeamentos a aplicar: {len(rename_dict)}")

if rename_dict:
    # Exibir primeiros 10
    print(f"\n   Primeiros 10 mapeamentos:")
    for i, (orig, novo) in enumerate(list(rename_dict.items())[:10], 1):
        print(f"   {i:2d}. '{orig}' → '{novo}'")

    if len(rename_dict) > 10:
        print(f"   ... e mais {len(rename_dict) - 10}")

    # Aplicar rename
    df_mapeado = df.rename(columns=rename_dict)

    print(f"\n✅ Mapeamento aplicado!")
else:
    df_mapeado = df.copy()
    print(f"\n   ℹ️  Nenhum mapeamento necessário (nomes já padronizados)")

# ═══════════════════════════════════════════════════════════════════
# SALVAR MAPEAMENTO NO DICIONÁRIO PERSISTENTE
# ═══════════════════════════════════════════════════════════════════

print("\n" + "─"*70)
print("💾 ATUALIZANDO DICIONÁRIO PERSISTENTE")
print("─"*70)

# Identificar fonte
nome_fonte = f"CSV_{arquivo_selecionado.stem}"

# Criar entrada no dicionário
if 'arquivos' not in DICIONARIO_PERSISTENTE:
    DICIONARIO_PERSISTENTE['arquivos'] = {}

DICIONARIO_PERSISTENTE['arquivos'][nome_fonte] = {
    'arquivo_origem': arquivo_selecionado.name,
    'data_processamento': datetime.now().isoformat(),
    'total_campos': len(df.columns),
    'campos_mapeados': {}
}

# Adicionar campos com confiança ≥ 85%
for col_orig, info in mapeamento_campos.items():
    if info['confianca'] >= 0.85:
        DICIONARIO_PERSISTENTE['arquivos'][nome_fonte]['campos_mapeados'][col_orig] = {
            'nome_padrao': info['campo_detectado'],
            'confianca': info['confianca'],
            'metodo': info['metodo']
        }

# Salvar dicionário
dict_path = fm.pastas['logs'] / 'DICT_Dicionario_Persistente.json'
with open(dict_path, 'w', encoding='utf-8') as f:
    json.dump(DICIONARIO_PERSISTENTE, f, indent=2, ensure_ascii=False)

print(f"✅ Dicionário atualizado: {dict_path.name}")
print(f"   Novos campos: {len(DICIONARIO_PERSISTENTE['arquivos'][nome_fonte]['campos_mapeados'])}")

# ═══════════════════════════════════════════════════════════════════
# PREVIEW DOS DADOS MAPEADOS
# ═══════════════════════════════════════════════════════════════════

print("\n" + "="*70)
print("👀 PREVIEW DOS DADOS MAPEADOS")
print("="*70)

print(f"\n📊 Shape: {df_mapeado.shape[0]:,} registros × {df_mapeado.shape[1]} colunas")

print(f"\n📋 Primeiras 10 colunas:")
for i, col in enumerate(df_mapeado.columns[:10], 1):
    original = [k for k, v in rename_dict.items() if v == col]
    if original:
        print(f"   {i:2d}. {col} (era: {original[0]})")
    else:
        print(f"   {i:2d}. {col}")

print(f"\n📈 Primeiras 3 linhas:")
print(df_mapeado.head(3).to_string())

# Substituir df original
df = df_mapeado

print("\n" + "="*70)
print("✅ DETECÇÃO DE CAMPOS CONCLUÍDA")
print("="*70)

In [ ]:
# ═══════════════════════════════════════════════════════════════════
# GUI VISUAL PARA CONFIRMAR TIPOS - INTERFACE INTUITIVA
# ═══════════════════════════════════════════════════════════════════

print("\n" + "="*70)
print("🎯 CONFIRMAÇÃO VISUAL DE TIPOS")
print("="*70)

if not requer_confirmacao:
    print("\n✅ Nenhuma confirmação necessária - todas com confiança ≥ 90%")
else:
    print(f"\n📋 {len(requer_confirmacao)} campos requerem confirmação")
    print(f"   Abrindo interface visual...")

    def confirmar_tipos_visual(campos_confirmar, tipos_detectados, df):
        """
        GUI VISUAL com lista numerada de tipos
        """
        # Preparar lista de tipos disponíveis
        tipos_dict = dicionario.dados['campos_conhecidos']
        tipos_lista = []

        for i, (nome_tipo, info) in enumerate(sorted(tipos_dict.items()), 1):
            tipos_lista.append({
                'numero': i,
                'nome': nome_tipo,
                'descricao': info.get('descricao', ''),
                'exemplos': info.get('exemplos', [])
            })

        # Adicionar opções especiais
        tipos_lista.append({
            'numero': 0,
            'nome': 'DESCONHECIDO',
            'descricao': 'Tipo não identificado',
            'exemplos': []
        })

        confirmacoes = {}
        idx_atual = [0]

        def processar_proximo():
            if idx_atual[0] >= len(campos_confirmar):
                return

            col = campos_confirmar[idx_atual[0]]
            info_campo = tipos_detectados[col]
            valores_exemplo = df[col].dropna().unique()[:5].tolist()

            # Criar janela
            root = tk.Tk()
            root.title(f"DETECTOR - Confirmação ({idx_atual[0]+1}/{len(campos_confirmar)})")
            root.geometry("900x650")

            x = (root.winfo_screenwidth() // 2) - 450
            y = (root.winfo_screenheight() // 2) - 325
            root.geometry(f"+{x}+{y}")

            frame_principal = tk.Frame(root, bg='white')
            frame_principal.pack(fill=tk.BOTH, expand=True, padx=20, pady=20)

            # ═══════════════════════════════════════════════════════════
            # TOPO: Progresso
            # ═══════════════════════════════════════════════════════════
            frame_topo = tk.Frame(frame_principal, bg='#E3F2FD', relief=tk.RAISED, borderwidth=2)
            frame_topo.pack(fill=tk.X, pady=(0, 15))

            tk.Label(
                frame_topo,
                text=f"🎯 Campo {idx_atual[0]+1} de {len(campos_confirmar)}",
                font=('Arial', 12, 'bold'),
                bg='#E3F2FD',
                fg='#1565C0'
            ).pack(pady=8)

            # ═══════════════════════════════════════════════════════════
            # ESQUERDA: Info do campo
            # ═══════════════════════════════════════════════════════════
            frame_conteudo = tk.Frame(frame_principal, bg='white')
            frame_conteudo.pack(fill=tk.BOTH, expand=True)

            # Coluna esquerda
            frame_esquerda = tk.Frame(frame_conteudo, bg='white', width=400)
            frame_esquerda.pack(side=tk.LEFT, fill=tk.BOTH, expand=False, padx=(0, 10))

            tk.Label(
                frame_esquerda,
                text="📋 CAMPO DO ARQUIVO",
                font=('Arial', 11, 'bold'),
                bg='white',
                anchor='w'
            ).pack(fill=tk.X, pady=(0, 10))

            # Box campo
            frame_campo = tk.Frame(frame_esquerda, bg='#FFF9C4', relief=tk.SUNKEN, borderwidth=2)
            frame_campo.pack(fill=tk.X, pady=(0, 10))

            tk.Label(
                frame_campo,
                text=f"🔖 {col}",
                font=('Arial', 10, 'bold'),
                bg='#FFF9C4',
                anchor='w',
                wraplength=380
            ).pack(fill=tk.X, padx=10, pady=(8, 5))

            tk.Label(
                frame_campo,
                text=f"🤖 Detectado: {info_campo['campo_detectado']}",
                font=('Arial', 9),
                bg='#FFF9C4',
                fg='#F57F17',
                anchor='w'
            ).pack(fill=tk.X, padx=10, pady=(0, 3))

            tk.Label(
                frame_campo,
                text=f"📊 Confiança: {info_campo['confianca']:.0%}",
                font=('Arial', 9),
                bg='#FFF9C4',
                fg='#F57F17',
                anchor='w'
            ).pack(fill=tk.X, padx=10, pady=(0, 8))

            # Exemplos
            tk.Label(
                frame_esquerda,
                text="📊 EXEMPLOS DE VALORES",
                font=('Arial', 10, 'bold'),
                bg='white',
                anchor='w'
            ).pack(fill=tk.X, pady=(5, 5))

            frame_exemplos = tk.Frame(frame_esquerda, bg='#F5F5F5', relief=tk.SUNKEN, borderwidth=1)
            frame_exemplos.pack(fill=tk.X)

            for i, val in enumerate(valores_exemplo, 1):
                val_str = str(val)[:50]
                tk.Label(
                    frame_exemplos,
                    text=f"{i}. {val_str}",
                    font=('Arial', 9),
                    bg='#F5F5F5',
                    anchor='w'
                ).pack(fill=tk.X, padx=10, pady=2)

            # ═══════════════════════════════════════════════════════════
            # DIREITA: Lista de tipos
            # ═══════════════════════════════════════════════════════════
            frame_direita = tk.Frame(frame_conteudo, bg='white')
            frame_direita.pack(side=tk.LEFT, fill=tk.BOTH, expand=True)

            tk.Label(
                frame_direita,
                text="🔢 TIPOS DISPONÍVEIS - Digite o número",
                font=('Arial', 11, 'bold'),
                bg='white',
                anchor='w'
            ).pack(fill=tk.X, pady=(0, 10))

            # Canvas + Scrollbar para lista
            frame_lista_outer = tk.Frame(frame_direita, bg='white')
            frame_lista_outer.pack(fill=tk.BOTH, expand=True)

            canvas = tk.Canvas(frame_lista_outer, bg='white', highlightthickness=0)
            scrollbar = tk.Scrollbar(frame_lista_outer, orient="vertical", command=canvas.yview)
            frame_lista = tk.Frame(canvas, bg='white')

            frame_lista.bind(
                "<Configure>",
                lambda e: canvas.configure(scrollregion=canvas.bbox("all"))
            )

            canvas.create_window((0, 0), window=frame_lista, anchor="nw")
            canvas.configure(yscrollcommand=scrollbar.set)

            canvas.pack(side="left", fill="both", expand=True)
            scrollbar.pack(side="right", fill="y")

            # Popular lista
            for tipo_info in tipos_lista:
                num = tipo_info['numero']
                nome = tipo_info['nome']
                desc = tipo_info['descricao']

                # Destacar tipo detectado
                bg_cor = '#E8F5E9' if nome == info_campo['campo_detectado'] else 'white'
                fg_cor = '#2E7D32' if nome == info_campo['campo_detectado'] else 'black'

                frame_item = tk.Frame(frame_lista, bg=bg_cor, relief=tk.GROOVE, borderwidth=1)
                frame_item.pack(fill=tk.X, pady=2, padx=5)

                tk.Label(
                    frame_item,
                    text=f"[{num:2d}]  {nome}",
                    font=('Courier', 9, 'bold'),
                    bg=bg_cor,
                    fg=fg_cor,
                    anchor='w'
                ).pack(fill=tk.X, padx=8, pady=(3, 0))

                if desc:
                    tk.Label(
                        frame_item,
                        text=f"      {desc}",
                        font=('Arial', 8),
                        bg=bg_cor,
                        fg='#666666',
                        anchor='w'
                    ).pack(fill=tk.X, padx=8, pady=(0, 3))

            # ═══════════════════════════════════════════════════════════
            # RODAPÉ: Campo de entrada + botões
            # ═══════════════════════════════════════════════════════════
            frame_rodape = tk.Frame(frame_principal, bg='white')
            frame_rodape.pack(fill=tk.X, pady=(15, 0))

            tk.Frame(frame_rodape, height=2, bg='#CCCCCC').pack(fill=tk.X, pady=(0, 10))

            # Campo de entrada
            frame_entrada = tk.Frame(frame_rodape, bg='white')
            frame_entrada.pack(pady=(0, 10))

            tk.Label(
                frame_entrada,
                text="Digite o número do tipo correto:",
                font=('Arial', 10, 'bold'),
                bg='white'
            ).pack(side=tk.LEFT, padx=(0, 10))

            var_numero = tk.StringVar()
            entry = tk.Entry(
                frame_entrada,
                textvariable=var_numero,
                font=('Arial', 12, 'bold'),
                width=8,
                justify='center'
            )
            entry.pack(side=tk.LEFT)
            entry.focus()

            label_erro = tk.Label(
                frame_entrada,
                text="",
                font=('Arial', 9),
                fg='#FF0000',
                bg='white'
            )
            label_erro.pack(side=tk.LEFT, padx=(10, 0))

            # Botões
            frame_btns = tk.Frame(frame_rodape, bg='white')
            frame_btns.pack()

            def validar_e_confirmar():
                try:
                    num_digitado = int(var_numero.get().strip())

                    # Buscar tipo pelo número
                    tipo_escolhido = None
                    for t in tipos_lista:
                        if t['numero'] == num_digitado:
                            tipo_escolhido = t['nome']
                            break

                    if tipo_escolhido:
                        confirmacoes[col] = tipo_escolhido
                        idx_atual[0] += 1
                        root.destroy()
                        processar_proximo()
                    else:
                        label_erro.config(text=f"❌ Número {num_digitado} inválido!")

                except ValueError:
                    label_erro.config(text="❌ Digite um número!")

            def manter_detectado():
                confirmacoes[col] = info_campo['campo_detectado']
                idx_atual[0] += 1
                root.destroy()
                processar_proximo()

            def pular_todos():
                for campo_restante in campos_confirmar[idx_atual[0]:]:
                    confirmacoes[campo_restante] = tipos_detectados[campo_restante]['campo_detectado']
                root.destroy()

            # Enter = confirmar
            entry.bind('<Return>', lambda e: validar_e_confirmar())

            tk.Button(
                frame_btns,
                text="✅ Confirmar",
                command=validar_e_confirmar,
                width=15,
                height=2,
                bg='#4CAF50',
                fg='white',
                font=('Arial', 10, 'bold'),
                cursor='hand2'
            ).pack(side=tk.LEFT, padx=5)

            tk.Button(
                frame_btns,
                text="➡️  Manter Detectado",
                command=manter_detectado,
                width=18,
                height=2,
                bg='#FF9800',
                fg='white',
                font=('Arial', 10),
                cursor='hand2'
            ).pack(side=tk.LEFT, padx=5)

            tk.Button(
                frame_btns,
                text="⏭️  Pular Todos",
                command=pular_todos,
                width=15,
                height=2,
                bg='#757575',
                fg='white',
                font=('Arial', 10),
                cursor='hand2'
            ).pack(side=tk.LEFT, padx=5)

            root.mainloop()

        # Iniciar processamento
        processar_proximo()
        return confirmacoes

    # Executar GUI
    confirmacoes = confirmar_tipos_visual(requer_confirmacao, tipos_detectados, df_limpo)

    # Aplicar confirmações
    if confirmacoes:
        print(f"\n✅ Confirmações aplicadas:")
        for col, tipo_confirmado in confirmacoes.items():
            if tipo_confirmado != tipos_detectados[col]['campo_detectado']:
                print(f"   ✏️  {col}")
                print(f"       {tipos_detectados[col]['campo_detectado']} → {tipo_confirmado}")
                tipos_detectados[col]['campo_detectado'] = tipo_confirmado
                tipos_detectados[col]['confianca'] = 1.0
                tipos_detectados[col]['metodo'] = 'CONFIRMACAO_USUARIO'

        print(f"\n✅ Total: {len(confirmacoes)} campos confirmados")
    else:
        print(f"\n⏭️  Nenhuma confirmação realizada")

In [ ]:
# ═══════════════════════════════════════════════════════════════════
# VALIDAÇÕES E ESTATÍSTICAS
# ═══════════════════════════════════════════════════════════════════

print("\n" + "="*70)
print("📊 VALIDAÇÕES E ESTATÍSTICAS")
print("="*70)

# 1. Resumo Geral
print("\n📋 RESUMO GERAL:")
print("─" * 80)
print(f"   Registros finais: {len(df_limpo):,}")
print(f"   Colunas finais: {len(df_limpo.columns)}")
print(f"   Memória em uso: {df_limpo.memory_usage(deep=True).sum() / 1024**2:.2f} MB")
print(f"   Linhas duplicadas: {df_limpo.duplicated().sum():,}")

# 2. Valores Nulos
print("\n🔍 ANÁLISE DE VALORES NULOS:")
print("─" * 80)

nulos_por_col = df_limpo.isnull().sum()
colunas_com_nulos = nulos_por_col[nulos_por_col > 0].sort_values(ascending=False)

if len(colunas_com_nulos) > 0:
    print(f"   ⚠️  {len(colunas_com_nulos)} colunas com valores nulos:")
    for col, qtd in colunas_com_nulos.items():
        pct = (qtd / len(df_limpo)) * 100
        barra = "█" * int(pct / 5)
        print(f"      {col:30s} | {qtd:6,} ({pct:5.1f}%) {barra}")
else:
    print(f"   ✅ Nenhum valor nulo!")

# 3. Distribuição de Tipos Detectados
print("\n🔬 DISTRIBUIÇÃO DE TIPOS DETECTADOS:")
print("─" * 80)

tipos_resumo = {}
for info in tipos_detectados.values():
    tipo = info['campo_detectado']  # ✅ CHAVE CORRETA
    tipos_resumo[tipo] = tipos_resumo.get(tipo, 0) + 1

for tipo, count in sorted(tipos_resumo.items(), key=lambda x: x[1], reverse=True):
    barra = "█" * (count * 2)
    print(f"   • {tipo:25s}: {count:2d} colunas {barra}")

# 4. Campos com Alta/Média/Baixa Confiança
print("\n📈 CONFIANÇA NA DETECÇÃO:")
print("─" * 80)

alta = sum(1 for info in tipos_detectados.values() if info['confianca'] >= 0.90)
media = sum(1 for info in tipos_detectados.values() if 0.70 <= info['confianca'] < 0.90)
baixa = sum(1 for info in tipos_detectados.values() if info['confianca'] < 0.70)

print(f"   ✅ Alta (≥90%):   {alta:2d} colunas")
print(f"   ⚠️  Média (70-90%): {media:2d} colunas")
print(f"   ❓ Baixa (<70%):   {baixa:2d} colunas")

# 5. Campos Ambíguos
print("\n⚠️  CAMPOS AMBÍGUOS:")
print("─" * 80)

ambiguos = {col: info for col, info in tipos_detectados.items() if info.get('ambiguidade', False)}

if ambiguos:
    print(f"   {len(ambiguos)} campos com ambiguidade:")
    for col, info in list(ambiguos.items())[:5]:
        print(f"\n   📌 {col}")
        print(f"      Detectado: {info['campo_detectado']}")
        print(f"      Similar a: {', '.join(info.get('candidatos', []))}")

    if len(ambiguos) > 5:
        print(f"\n   ... e mais {len(ambiguos) - 5} campos")
else:
    print(f"   ✅ Nenhum campo ambíguo!")

# 6. Campos Únicos (potenciais IDs)
print("\n🔑 ANÁLISE DE UNICIDADE:")
print("─" * 80)

for col in df_limpo.columns:
    unicos = df_limpo[col].nunique()
    total = len(df_limpo)
    pct_unico = (unicos / total) * 100

    if pct_unico == 100:
        print(f"   🔑 {col:30s} | 100% único (potencial ID)")
    elif pct_unico >= 95:
        print(f"   ⚠️  {col:30s} | {pct_unico:5.1f}% único")

# 7. Cardinalidade (valores únicos)
print("\n📊 CARDINALIDADE:")
print("─" * 80)

for col in df_limpo.columns[:10]:  # Primeiras 10
    unicos = df_limpo[col].nunique()
    total = len(df_limpo)
    pct = (unicos / total) * 100

    if pct <= 10:
        categoria = "Categoria (baixa)"
    elif pct <= 50:
        categoria = "Mista (média)"
    else:
        categoria = "Contínua (alta)"

    print(f"   {col:30s} | {unicos:4d} únicos ({pct:5.1f}%) - {categoria}")

if len(df_limpo.columns) > 10:
    print(f"\n   ... e mais {len(df_limpo.columns) - 10} colunas")

# 8. Tipos de Dados Pandas
print("\n💾 TIPOS DE DADOS (PANDAS):")
print("─" * 80)

dtype_counts = df_limpo.dtypes.value_counts()
for dtype, count in dtype_counts.items():
    print(f"   • {str(dtype):15s}: {count:2d} colunas")

print("\n" + "="*70)
print("✅ VALIDAÇÕES CONCLUÍDAS")
print("="*70)

In [ ]:
# ═══════════════════════════════════════════════════════════════════
# EXPORTAÇÃO DE RESULTADOS - VERSÃO DEFENSIVA
# ═══════════════════════════════════════════════════════════════════

print("\n" + "="*70)
print("💾 EXPORTAÇÃO DE RESULTADOS")
print("="*70)

nome_base = arquivo_selecionado.stem

# 1️⃣  DADOS LIMPOS
print("\n1️⃣  Dados limpos...")
arquivo_limpo = fm.salvar(df_limpo, f"{nome_base}_Limpo", tipo='xlsx', pasta='processados')
print(f"   ✅ {arquivo_limpo.name}")

# 2️⃣  DICIONÁRIO DE CAMPOS
print("\n2️⃣  Dicionário de campos...")
registros_dict = []

for col in df_limpo.columns:
    tipo_info = tipos_detectados.get(col, {})
    valores_exemplo = df_limpo[col].dropna().unique()[:3].tolist()

    registros_dict.append({
        'Coluna': col,
        'Tipo_Detectado': tipo_info.get('campo_detectado', 'DESCONHECIDO'),
        'Confianca_%': tipo_info.get('confianca', 0.0) * 100,
        'Score_Conteudo_%': tipo_info.get('score_conteudo', 0.0) * 100,
        'Score_Nome_%': tipo_info.get('score_nome', 0.0) * 100,
        'Metodo': tipo_info.get('metodo', 'N/A'),
        'Ambiguidade': 'Sim' if tipo_info.get('ambiguidade') else 'Não',
        'Dtype_Pandas': str(df_limpo[col].dtype),
        'Valores_Unicos': df_limpo[col].nunique(),
        'Nulos_Qtd': df_limpo[col].isna().sum(),
        'Nulos_%': (df_limpo[col].isna().sum() / len(df_limpo)) * 100,
        'Exemplo_1': str(valores_exemplo[0]) if len(valores_exemplo) > 0 else None,
        'Exemplo_2': str(valores_exemplo[1]) if len(valores_exemplo) > 1 else None,
        'Exemplo_3': str(valores_exemplo[2]) if len(valores_exemplo) > 2 else None
    })

df_dict = pd.DataFrame(registros_dict)
arquivo_dict = fm.salvar(df_dict, f"DICT_{nome_base}", tipo='xlsx', pasta='outputs')
print(f"   ✅ {arquivo_dict.name}")

# 3️⃣  LOG DE PROCESSAMENTO (COM VERIFICAÇÃO DEFENSIVA)
print("\n3️⃣  Log de processamento...")

# ✅ VERIFICAÇÃO DEFENSIVA - garantir que variáveis existam
try:
    # Tentar usar as variáveis da Célula 8
    _linha_cab_ini = linha_cabecalho_inicio
    _linha_cab_fim = linha_cabecalho_fim
    _idx_cab_ini = idx_cab_inicio
    _idx_cab_fim = idx_cab_fim
    _col_ini = col_inicio
    _col_fim = col_fim
    _idx_col_ini = idx_col_inicio
    _idx_col_fim = idx_col_fim
    _linha_dados_ini_excel = config.get('linha_dados_inicio', linha_cabecalho_inicio + 1)
    _idx_dados_ini = idx_dados_inicio
except NameError:
    # Se não existirem, usar valores padrão/detectados
    print("   ⚠️  Algumas variáveis não encontradas - usando valores padrão")
    _linha_cab_ini = 1
    _linha_cab_fim = 1
    _idx_cab_ini = 0
    _idx_cab_fim = 0
    _col_ini = 1
    _col_fim = len(df_bruto.columns)
    _idx_col_ini = 0
    _idx_col_fim = len(df_bruto.columns)
    _linha_dados_ini_excel = 2
    _idx_dados_ini = 1

log_processamento = {
    'Arquivo_Original': arquivo_selecionado.name,
    'Caminho_Original': str(arquivo_selecionado),
    'Sheet_Processada': sheet_nome,
    'Metodo_Carga': metodo_carga,

    # Cabeçalho
    'Linha_Cabecalho_Inicio_Excel': _linha_cab_ini,
    'Linha_Cabecalho_Fim_Excel': _linha_cab_fim,
    'Indice_Cabecalho_Inicio_Python': _idx_cab_ini,
    'Indice_Cabecalho_Fim_Python': _idx_cab_fim,

    # Colunas
    'Coluna_Inicio_Excel': _col_ini,
    'Coluna_Fim_Excel': _col_fim,
    'Indice_Coluna_Inicio_Python': _idx_col_ini,
    'Indice_Coluna_Fim_Python': _idx_col_fim,

    # Dados
    'Linha_Dados_Inicio_Excel': _linha_dados_ini_excel,
    'Indice_Dados_Inicio_Python': _idx_dados_ini,

    # Contadores
    'Registros_Bruto': len(df_bruto),
    'Colunas_Bruto': len(df_bruto.columns),
    'Registros_Limpo': len(df_limpo),
    'Colunas_Limpo': len(df_limpo.columns),
    'Operacoes_Limpeza': ', '.join(log_limpeza) if 'log_limpeza' in locals() and log_limpeza else 'Nenhuma',

    # Timestamp
    'Timestamp': fm.timestamp,
    'Data_Processamento': datetime.now().isoformat()
}

df_log = pd.DataFrame([log_processamento])
arquivo_log = fm.salvar(df_log, f"LOG_{nome_base}", tipo='xlsx', pasta='logs')
print(f"   ✅ {arquivo_log.name}")

# 4️⃣  CÓDIGO PYTHON PARA REPRODUÇÃO
print("\n4️⃣  Código de reprodução...")
codigo_reprod = f'''# ═══════════════════════════════════════════════════════════════════
# CÓDIGO DE REPRODUÇÃO - Gerado automaticamente
# Arquivo: {arquivo_selecionado.name}
# Sheet: {sheet_nome}
# Data: {datetime.now().strftime("%Y-%m-%d %H:%M:%S")}
# ═══════════════════════════════════════════════════════════════════

import pandas as pd
from pathlib import Path

# ═══════════════════════════════════════════════════════════════════
# CONFIGURAÇÃO
# ═══════════════════════════════════════════════════════════════════

arquivo = Path(r"{arquivo_selecionado}")
sheet = "{sheet_nome}"

# Range de extração (índices Python - começa em 0)
linha_cabecalho_inicio = {_idx_cab_ini}
linha_cabecalho_fim = {_idx_cab_fim}
col_inicio = {_idx_col_ini}
col_fim = {_idx_col_fim}
linha_dados_inicio = {_idx_dados_ini}

# ═══════════════════════════════════════════════════════════════════
# CARREGAMENTO
# ═══════════════════════════════════════════════════════════════════

print(f"📂 Carregando: {{arquivo.name}}")
print(f"📄 Sheet: {{sheet}}")

if linha_cabecalho_inicio == linha_cabecalho_fim:
    # CASO 1: Cabeçalho em 1 linha
    print(f"📋 Cabeçalho: Linha {{linha_cabecalho_inicio + 1}} (Excel)")

    df = pd.read_excel(
        arquivo,
        sheet_name=sheet,
        header=linha_cabecalho_inicio,
        usecols=range(col_inicio, col_fim)
    )

    # Pular linhas entre cabeçalho e dados
    linhas_pular = linha_dados_inicio - linha_cabecalho_inicio - 1
    if linhas_pular > 0:
        print(f"⏭️  Pulando {{linhas_pular}} linhas")
        df = df.iloc[linhas_pular:].copy()
else:
    # CASO 2: Cabeçalho em múltiplas linhas
    print(f"📋 Cabeçalho: Linhas {{linha_cabecalho_inicio + 1}} a {{linha_cabecalho_fim + 1}} (Excel)")

    df_temp = pd.read_excel(
        arquivo,
        sheet_name=sheet,
        header=None,
        usecols=range(col_inicio, col_fim)
    )

    # Combinar linhas do cabeçalho
    cabecalho = df_temp.iloc[linha_cabecalho_inicio:linha_cabecalho_fim+1].values
    cab_final = []

    for col_idx in range(cabecalho.shape[1]):
        partes = [
            str(linha[col_idx]).strip()
            for linha in cabecalho
            if str(linha[col_idx]).strip() not in ['', 'nan', 'None']
        ]
        cab_final.append(' - '.join(partes) if partes else f'Col_{{col_idx}}')

    # Extrair dados
    df = df_temp.iloc[linha_dados_inicio:].copy()
    df.columns = cab_final

# Reset index
df = df.reset_index(drop=True)

print(f"✅ Carregado: {{len(df):,}} registros × {{len(df.columns)}} colunas")

# ═══════════════════════════════════════════════════════════════════
# VALIDAÇÃO
# ═══════════════════════════════════════════════════════════════════

colunas_esperadas = {df_limpo.columns.tolist()}

if df.columns.tolist() == colunas_esperadas:
    print("✅ Estrutura validada - colunas correspondem!")
else:
    print("⚠️  Diferença na estrutura:")

    extras = set(df.columns) - set(colunas_esperadas)
    if extras:
        print(f"   Colunas extras: {{extras}}")

    faltando = set(colunas_esperadas) - set(df.columns)
    if faltando:
        print(f"   Colunas faltando: {{faltando}}")

print(f"\\n📊 Shape: {{df.shape}}")
print(f"💾 Memória: {{df.memory_usage(deep=True).sum() / 1024**2:.2f}} MB")
'''

arquivo_codigo = fm.pastas['codigos_integracao'] / f"REPROD_{nome_base}_{fm.timestamp}.py"
with open(arquivo_codigo, 'w', encoding='utf-8') as f:
    f.write(codigo_reprod)

print(f"   ✅ {arquivo_codigo.name}")

# 5️⃣  ATUALIZAR DICIONÁRIO MASTER
print("\n5️⃣  Atualizando dicionário master...")
dicionario.atualizar_historico({
    'arquivo': arquivo_selecionado.name,
    'sheet': sheet_nome,
    'timestamp': fm.timestamp,
    'colunas': df_limpo.columns.tolist(),
    'registros': len(df_limpo),
    'tipos': {col: info['campo_detectado'] for col, info in tipos_detectados.items()}
})
print(f"   ✅ Histórico atualizado")

# 6️⃣  RESUMO DE ARQUIVOS GERADOS
print("\n" + "="*70)
print("📂 ARQUIVOS GERADOS")
print("="*70)

arquivos_gerados = [
    ('📊 Dados Limpos', arquivo_limpo),
    ('📖 Dicionário de Campos', arquivo_dict),
    ('📋 Log de Processamento', arquivo_log),
    ('🐍 Código de Reprodução', arquivo_codigo)
]

for emoji_desc, path in arquivos_gerados:
    print(f"\n{emoji_desc}")
    print(f"   📁 {path.name}")
    print(f"   📂 {path.parent}")
    print(f"   📏 {path.stat().st_size / 1024:.1f} KB")

print("\n" + "="*70)
print("✅ EXPORTAÇÃO CONCLUÍDA COM SUCESSO!")
print("="*70)

# Variável global para uso posterior
df_resultado = df_limpo.copy()

print(f"\n💡 Dataset disponível em: df_resultado")
print(f"   Shape: {df_resultado.shape}")
print(f"   Memória: {df_resultado.memory_usage(deep=True).sum() / 1024**2:.2f} MB")

In [ ]:
# ═══════════════════════════════════════════════════════════════════
# RELATÓRIO FINAL - RESUMO VISUAL COMPLETO
# ═══════════════════════════════════════════════════════════════════

print("\n")
print("╔" + "="*68 + "╗")
print("║" + " 📋 RELATÓRIO FINAL - PROCESSAMENTO CONCLUÍDO".center(78) + "║")
print("╚" + "="*68 + "╝")

# ═══════════════════════════════════════════════════════════════════
# INFORMAÇÕES DO ARQUIVO
# ═══════════════════════════════════════════════════════════════════

print("\n┏" + "━"*68 + "┓")
print("┃" + " 📁 INFORMAÇÕES DO ARQUIVO".center(78) + "┃")
print("┣" + "━"*68 + "┫")

print(f"┃  Nome: {arquivo_selecionado.name:<68}┃")
print(f"┃  Sheet: {sheet_nome:<67}┃")

# ✅ CÓDIGO DEFENSIVO - verificar se variáveis existem
try:
    _linha_cab_ini = linha_cabecalho_inicio
    _idx_cab_ini = idx_cab_inicio
    _linha_dados = config.get('linha_dados_inicio', linha_cabecalho_inicio + 1)
    print(f"┃  Cabeçalho: Linha {_linha_cab_ini} (Excel) / Índice {_idx_cab_ini} (Python){' '*20}┃")
    print(f"┃  Dados: A partir da linha {_linha_dados} (Excel){' '*37}┃")
except NameError:
    print(f"┃  Cabeçalho: [informação não disponível]{' '*36}┃")

print(f"┃  Método: {metodo_carga:<66}┃")
print("┗" + "━"*68 + "┛")

# ═══════════════════════════════════════════════════════════════════
# ESTATÍSTICAS DE PROCESSAMENTO
# ═══════════════════════════════════════════════════════════════════

print("\n┏" + "━"*68 + "┓")
print("┃" + " 📊 ESTATÍSTICAS DE PROCESSAMENTO".center(78) + "┃")
print("┣" + "━"*68 + "┫")

print(f"┃  Registros originais: {len(df_bruto):>6,}{' '*47}┃")
print(f"┃  Registros finais:    {len(df_limpo):>6,}{' '*47}┃")
print(f"┃  Diferença:           {len(df_bruto) - len(df_limpo):>6,} removidos{' '*39}┃")
print("┃" + "─"*68 + "┃")
print(f"┃  Colunas originais:   {len(df_bruto.columns):>6,}{' '*47}┃")
print(f"┃  Colunas finais:      {len(df_limpo.columns):>6,}{' '*47}┃")
print(f"┃  Diferença:           {len(df_bruto.columns) - len(df_limpo.columns):>6,} removidas{' '*38}┃")
print("┃" + "─"*68 + "┃")

memoria_mb = df_limpo.memory_usage(deep=True).sum() / 1024**2
print(f"┃  Memória em uso:      {memoria_mb:>6.2f} MB{' '*43}┃")

duplicatas = df_limpo.duplicated().sum()
print(f"┃  Linhas duplicadas:   {duplicatas:>6,}{' '*47}┃")

print("┗" + "━"*68 + "┛")

# ═══════════════════════════════════════════════════════════════════
# QUALIDADE DOS DADOS
# ═══════════════════════════════════════════════════════════════════

print("\n┏" + "━"*68 + "┓")
print("┃" + " 🔍 QUALIDADE DOS DADOS".center(78) + "┃")
print("┣" + "━"*68 + "┫")

total_nulos = df_limpo.isnull().sum().sum()
total_celulas = len(df_limpo) * len(df_limpo.columns)
pct_nulos = (total_nulos / total_celulas) * 100

print(f"┃  Total de valores nulos: {total_nulos:>6,} ({pct_nulos:>5.2f}%){' '*35}┃")

colunas_com_nulos = df_limpo.isnull().sum()
colunas_com_nulos = colunas_com_nulos[colunas_com_nulos > 0]

if len(colunas_com_nulos) > 0:
    print(f"┃  Colunas com nulos:      {len(colunas_com_nulos):>6,}{' '*47}┃")
else:
    print(f"┃  ✅ Nenhuma coluna com valores nulos!{' '*39}┃")

print("┗" + "━"*68 + "┛")

# ═══════════════════════════════════════════════════════════════════
# DETECÇÃO DE TIPOS
# ═══════════════════════════════════════════════════════════════════

print("\n┏" + "━"*68 + "┓")
print("┃" + " 🔬 DETECÇÃO DE TIPOS".center(78) + "┃")
print("┣" + "━"*68 + "┫")

alta = sum(1 for info in tipos_detectados.values() if info['confianca'] >= 0.90)
media = sum(1 for info in tipos_detectados.values() if 0.70 <= info['confianca'] < 0.90)
baixa = sum(1 for info in tipos_detectados.values() if info['confianca'] < 0.70)

print(f"┃  ✅ Alta confiança (≥90%):   {alta:>3} colunas{' '*41}┃")
print(f"┃  ⚠️  Média confiança (70-90%): {media:>3} colunas{' '*41}┃")
print(f"┃  ❓ Baixa confiança (<70%):   {baixa:>3} colunas{' '*41}┃")

print("┃" + "─"*68 + "┃")

# Top 5 tipos detectados
tipos_resumo = {}
for info in tipos_detectados.values():
    tipo = info['campo_detectado']
    tipos_resumo[tipo] = tipos_resumo.get(tipo, 0) + 1

print("┃  Top 5 tipos mais comuns:{' '*53}┃")
for i, (tipo, count) in enumerate(sorted(tipos_resumo.items(), key=lambda x: x[1], reverse=True)[:5], 1):
    print(f"┃     {i}. {tipo:<25} ({count:>2} colunas){' '*24}┃")

print("┗" + "━"*68 + "┛")

# ═══════════════════════════════════════════════════════════════════
# OPERAÇÕES DE LIMPEZA
# ═══════════════════════════════════════════════════════════════════

print("\n┏" + "━"*68 + "┓")
print("┃" + " 🧹 OPERAÇÕES DE LIMPEZA REALIZADAS".center(78) + "┃")
print("┣" + "━"*68 + "┫")

if 'log_limpeza' in locals() and log_limpeza:
    for i, operacao in enumerate(log_limpeza, 1):
        # Quebrar linhas longas
        if len(operacao) > 70:
            print(f"┃  {i}. {operacao[:67]}...┃")
        else:
            print(f"┃  {i}. {operacao:<73}┃")
else:
    print(f"┃  ✅ Nenhuma limpeza necessária - dados já estavam limpos!{' '*18}┃")

print("┗" + "━"*68 + "┛")

# ═══════════════════════════════════════════════════════════════════
# ARQUIVOS GERADOS
# ═══════════════════════════════════════════════════════════════════

print("\n┏" + "━"*68 + "┓")
print("┃" + " 📂 ARQUIVOS GERADOS".center(78) + "┃")
print("┣" + "━"*68 + "┫")

arquivos_info = [
    ("Dados Limpos", arquivo_limpo),
    ("Dicionário de Campos", arquivo_dict),
    ("Log de Processamento", arquivo_log),
    ("Código de Reprodução", arquivo_codigo)
]

for descricao, path in arquivos_info:
    tamanho_kb = path.stat().st_size / 1024
    nome_arquivo = path.name

    # Truncar nome se muito longo
    if len(nome_arquivo) > 50:
        nome_arquivo = nome_arquivo[:47] + "..."

    print(f"┃  {descricao}:{' '*(30 - len(descricao))}┃")
    print(f"┃     📄 {nome_arquivo:<68}┃")
    print(f"┃     📏 {tamanho_kb:>6.1f} KB{' '*61}┃")
    print("┃" + "─"*68 + "┃")

# Remover última linha divisória
print("┗" + "━"*68 + "┛")

# ═══════════════════════════════════════════════════════════════════
# PRÓXIMOS PASSOS
# ═══════════════════════════════════════════════════════════════════

print("\n┏" + "━"*68 + "┓")
print("┃" + " 💡 PRÓXIMOS PASSOS RECOMENDADOS".center(78) + "┃")
print("┣" + "━"*68 + "┫")

sugestoes = []

# Sugestões baseadas em qualidade
if len(colunas_com_nulos) > 0:
    sugestoes.append("Tratar valores nulos nas colunas identificadas")

if baixa > 0:
    sugestoes.append(f"Revisar {baixa} campos com baixa confiança na detecção")

if duplicatas > 0:
    sugestoes.append("Investigar e remover linhas duplicadas")

# Sugestões padrão
sugestoes.extend([
    "Revisar o dicionário de campos gerado",
    "Validar tipos detectados conforme necessidade",
    "Utilizar código de reprodução para reprocessar"
])

for i, sugestao in enumerate(sugestoes[:6], 1):  # Máximo 6 sugestões
    print(f"┃  {i}. {sugestao:<73}┃")

print("┗" + "━"*68 + "┛")

# ═══════════════════════════════════════════════════════════════════
# RODAPÉ
# ═══════════════════════════════════════════════════════════════════

print("\n╔" + "="*68 + "╗")
print("║" + " ✅ PROCESSAMENTO CONCLUÍDO COM SUCESSO!".center(78) + "║")
print("╠" + "="*68 + "╣")
print("║" + f" Timestamp: {fm.timestamp}".ljust(78) + "║")
print("║" + f" Dataset disponível em: df_resultado".ljust(78) + "║")
print("║" + f" Shape: {df_resultado.shape}".ljust(78) + "║")
print("╚" + "="*68 + "╝")

print("\n")

In [ ]:
# ═══════════════════════════════════════════════════════════════════
# ABERTURA AUTOMÁTICA DA PASTA DESTINO
# ═══════════════════════════════════════════════════════════════════

# Abrir pasta de outputs automaticamente
fm.abrir_pasta('outputs')

print("\n💡 TIP: Se a pasta não abriu, o caminho está exibido no relatório acima!")

In [ ]:
df_resultado